## 1. Inicializacion

### Librerías

Se importan todas las librerías que se requieren para ejecutar el Notebook. Es importante contar con todas ellas para poder acceder a las funcionalidades de la aplicación

In [3]:
#Poner en comentario el comando pip install (usando #) una vez se haya instalado el paquete graphviz en Visual Studio Code
!pip install graphviz

In [4]:
#Poner en comentario el comando pip install (usando #) una vez se haya instalado el paquete ipywidgets en Visual Studio Code
!pip install ipywidgets==8.0.2

     ---------------------------------------- 0.0/134.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/134.4 kB ? eta -:--:--
     -------- ---------------------------- 30.7/134.4 kB 445.2 kB/s eta 0:00:01
     -------- ---------------------------- 30.7/134.4 kB 445.2 kB/s eta 0:00:01
     ---------------- -------------------- 61.4/134.4 kB 409.6 kB/s eta 0:00:01
     ------------------------------------ 134.4/134.4 kB 663.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/215.0 kB ? eta -:--:--
   ---------------------- ----------------- 122.9/215.0 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 215.0/215.0 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   -- ------------------------------------- 0.1/2.3 MB 7.0 MB/s eta 0:00:01
   ------ --------------------------------- 0.4/2.3 MB 4.8 MB/s eta 0:00:01
   ---------- ----------------------------- 0.6/2.3 MB 4.9 MB/s eta 0:00:01
   --

In [5]:
#Poner en comentario el comando pip install (usando #) una vez se haya instalado el paquete matplotlib en Visual Studio Code
!pip install matplotlib

     ---------------------------------------- 0.0/162.3 kB ? eta -:--:--
     -- ------------------------------------- 10.2/162.3 kB ? eta -:--:--
     --------- --------------------------- 41.0/162.3 kB 495.5 kB/s eta 0:00:01
     --------------------------------- ---- 143.4/162.3 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 162.3/162.3 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB ? eta 0:00:00
   ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
    --------------------------------------- 0.1/7.6 MB ? eta -:--:--
    --------------------------------------- 0.2/7.6 MB 3.5 MB/s eta 0:00:03
   - -------------------------------------- 0.2/7.6 MB 3.0 MB/s eta 0:00:03
   - -------------------------------------- 0.4/7.6 MB 2.8 MB/s eta 0:00:03
   -- ---------------------------

In [6]:
#Poner en comentario el comando pip install (usando #) una vez se haya instalado el paquete ipykernel en Visual Studio Code
!pip install ipykernel

In [7]:
import os
#Este paso aplica en version Windows
#Este paso NO se requiere en version Mac (mantener comentarios)
#Ruta del directorio bin donde se instalo Graphviz: ../Graphviz/Bin
#TODO Quitar comentario de la linea siguiente colocando el directorio donde se instalo Graphviz/bin
#os.environ["PATH"] += os.pathsep + 'C:/Users/turing/Documents/Graphviz/bin/'

# Estructuras de datos que se toman como referencia para validaciones
from referencias.lista_disc import listaEnlazada as referenciaLista
from referencias.arbol_BST_disc import bst as referenciaBST
from referencias.grafo_disc import grafo as referenciaGrafo
from referencias.arreglo_disc import Arreglo as refereneciaArreglo
from referencias.arbol_RBT_disc import RBT as referenciaRBT
from referencias.separate_chaining_disc import SeparateChaining  as referenciaSeparate
from referencias.linear_probing_disc import LinearProbing  as referenciaLinear

# Librerias que permiten crear la interfaz grafica de la aplicacion
import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display, clear_output, SVG, HTML

# Libreria que permite representar graficamente las estructuras de datos trabajadas
from graphviz import Digraph, Graph

# Librerias adicionales para procesos que se requieren durante la ejecución
import matplotlib.pyplot as plt
import random
import string
import json
import  tkinter as tk
from tkinter import filedialog
import shutil
import importlib.util
import functools

### Inicializacion de la Interfaz

Se crean los elementos base que permiten manejar una interfaz grafica. Se crea en forma de Tabs para diferenciar entre estructuras de datos

In [8]:
out, out1, out2, out3, out4, out5, out6, out7, out8, out9 = Output(), Output(), Output(), Output(), Output(), Output(), Output(), Output(), Output(), Output()
tab = Tab(children = [out1, out2, out3, out4, out5, out6, out7, out8, out9],
          layout=Layout(width='99%', height='auto'))
tab.set_title(0, 'Arreglos')
tab.set_title(1, 'L.E. Sencilla')
tab.set_title(2, 'L.E. Doble')
tab.set_title(3, 'BST')
tab.set_title(4, 'G. Dirigido')
tab.set_title(5, 'G. No Dirigido')
tab.set_title(6, 'Hash LP')
tab.set_title(7, 'Hash SC')
tab.set_title(8, 'RBT')

form_item_layout = Layout(
    justify_content='center'
)

### Metodos de soporte

Se definen una serie de funciones que se usan durante la ejecución, y se inicializan algunas variables

In [9]:
VALIDATION_STATES = {0:'WARNING', 1:'SUCCESSFUL', -1:'FAILED'} # Estados resultantes del proceso de validacion
colorPointer = 'grey'   # Color para apuntadores
colorRight = 'blue'     # Color para conexiones a la derecha
colorLeft = 'green'     # Color para conexiones a la izquierda
colorHigh = 'red'       # Color para resaltar un elemento
col = "black"           # Color por defecto

def checkAlgoritmGraph(type, recorrido):
    '''
    Verifica si un algoritmo de grafos se puede aplicar sobre el grafo actual 

    Args:
        type: tipo del grafo actual: 4 (dirigido) 5(no dirigido)
        recorrido: Nombre del recorrido/algoritmo que se va a ejecutar
    
    Returns:
        True si el recorrido se puede aplicar al tipo de grafo actual.
        False de lo contrario
    '''
    both = ['Bellman-Ford', 'DepthFirstSearch', 'BreadhtFirstSearch', 'DepthFirstOrder', 'Dijkstra','KosarajuSCC']
    result = False
    comment = ''
    if recorrido in both:
        result = True
    elif recorrido == 'PrimMST':
        result = type == 5
        comment = 'ERROR: ' + recorrido + ' solo se puede aplicar a Grafos No Dirigidos'
    elif recorrido == 'KosarajuSCC' or recorrido == 'DirectedCycle':
        result = type == 4
        comment = 'ERROR: ' + recorrido + ' solo se puede aplicar a Grafos Dirigidos'
    return result, comment   

def getNodesGivenEdges(edges):
    '''
    Dado una lista de conexiones, retorna una lista con los nodos que se encuentran en las conexiones

    Args:
        edges: Lista de conexiones. 
        Cada conexión es una tupla: (nodo_origen, nodo_destino, peso)
    
    Returns:
        Lista de los valores de los nodos que se encuentran en las conexiones
    '''
    nodes = list()
    for i in edges:
        if i[0] not in nodes:
            nodes.append(i[0])
        if i[1] not in nodes:
            nodes.append(i[1])
    return nodes

def defaultfunction(elem_1, elem_2):
    '''
    Función de comparación entre dos elementos. Si son del mismo tipo de dato se comparan directamente, 
    de lo contrario se transforman ambos elementos a String y se realiza la comparación
    
    Args:
        elem_1: Elemento 1. 
        elem_2: Elemento 2.
    
    Returns:
        -1: Si el elemento 1 es menor al elemento 2
         0: Si el elemento 1 es igual al elemento 2
         1: Si el elemento 1 es mayor al elemento 2
    '''
    if type(elem_1) != type(elem_2):
        elem_1 = str(elem_1)
        elem_2 = str(elem_2)
    if elem_1 > elem_2:
        return 1
    elif elem_1 < elem_2:
        return -1
    return 0

def create_n_random(n):
    '''
    Crea una lista de numeros aleatorios sin repetición de longitud n.
       
    Args:
        n: tamaño de la lista
    
    Returns:
        Lista de n numeros aleatorios sin repeticion
    '''
    nodos = list()
    for i in range(n*n):
        num = random.randint(1,50)
        if num not in nodos:
            nodos.append(num)
        if len(nodos) == n:
            break
    return nodos 

def createTuples(nodos, tipo):
    '''
    Crea una lista de tuplas (id, elemento) creando las instancias adicionales (apuntadores)
    que se tienen en una lista enlazada dependiendo del tipo
       
    Args:
        nodos: elementos de la lista enlazada
        tipo: tipo de lista enlazada. (1: sencilla, 2: doble)
    
    Returns:
        Lista de tuplas de los elementos de nodos con ids unicos
    '''
    tuples = list()
    if tipo == 1:
        ids = range(0,len(nodos)+2)
        tuples.append((ids[0],'First'))
        for i in range(1,len(ids)-1):
            tuples.append((ids[i],nodos[i-1]))
        if len(nodos) > 0:
            tuples.append((ids[len(nodos)+1],'None'))
    else:
        ids = range(0,len(nodos)+3)
        tuples.append((ids[0],'First'))
        if len(nodos) > 0:
            tuples.append((ids[1],'None'))
            for i in range(2,len(ids)-1):
                tuples.append((ids[i],nodos[i-2]))
            tuples.append((ids[len(nodos)+2],'None'))
    return tuples

def get_random_string():
    '''
    Crea una String random de tamaño 20
       
    Args:
    
    Returns:
        String random de 20 caracteres
    '''
    letters = string.ascii_lowercase
    result_str = ''.join(random.choice(letters) for i in range(20))
    return result_str

def create_n_randomEdges(nodos, tipo = 4):
    '''
    Dado una lista de valores de nodos, se crean x conexiones entre ellos, donde
    n es un numero aleatorio entre 10 y la cantidad de nodos. 
       
    Args:
        nodos: lista de los elementos de los nodos de un grafo
        tipo: tipo de grafo (4: Dirigido, 5: No Dirigido)
    
    Returns:
        Lista de tuplas (nodo_origen, nodo_destino, costo)
    '''
    edges = list()
    edgesAux = list()
    ln = random.randint(10, len(nodos))
    for i in range(ln):
        a = random.choice(nodos)
        b = random.choice(nodos)
        while a == b:
            b = random.choice(nodos)
        if (a,b) not in edgesAux:
            if tipo != 4:
                if (b,a) not in edgesAux:
                    num = random.random() + random.randint(1,50)
                    edges.append((a,b,num))
                    edgesAux.append((a,b))
            else:
                num = random.random() + random.randint(1,50)
                edges.append((a,b,num))
                edgesAux.append((a,b))
    return edges

def validarEstructura(valid, tipo):
    '''
    Verifica si el tipo de una estructura de datos se enuentra entre las validas (valid) 
       
    Args:
        valid: lista de tipos de estructuras validas
        tipo: tipo de una estructura
    
    Returns:
        state: True si el tipo se encuentra en la lista de tipos validos, False de los contrario
        comment: Mensaje de error informando cual es la estructura del tipo ingresado
    '''
    if tipo in valid:
        state = True
        comment = ''
    else:
        state = False
        if tipo == 1:
            actual = 'Lista Encadenada Sencilla'
        elif tipo == 2:
            actual = 'Lista Encadenada Doble'
        elif tipo == 3:
            actual = 'Arbol BST'
        elif tipo == 4:
            actual = 'Grafo Dirigido'
        elif tipo == 5:
            actual = 'Arreglo'
        elif tipo == 6:
            actual = 'Tabla Hash LP'
        else:
            actual = 'Grafo NO Dirigido'
        comment = 'ERROR: La estructura actual es ' + actual
    return state, comment


## 2. Metodos Gráficos (Estructuras de datos)   

### Métodos Gráficos - Listas Enlazadas

In [10]:
def displayList(estructura, tipo, nodosX=list()):
    '''
    Grafica en el Canvas la lista que entra por parametro, del tipo dado, y resalta los nodos
    que se encuentren en la lista nodosX
       
    Args:
        estructura: estructura de datos que debe ser una lista enlazada
        tipo: tipo de la estructura (1: Sencilla, 2: doble)
        nodosX: lista de valores de nodos que se resaltan (pintan en otro color)
    
    Returns:
        -
    '''
    try:
        nodos = estructura.getNodeValues()
    except:
        e = '\tProblema en el método getNodeValues()'
        raise Exception(e)
    
    dot = Digraph()
    
    dot.graph_attr = {
        'rankdir': 'LR',
        'center': 'true',
        'size':'14,5',
        'ratio':'fill'
    }
    
    tuples = createTuples(nodos, tipo)
    
    for i,j in tuples:
        if j == 'First' or j == 'None':
            dot.node(name=str(i), label=str(j), shape="square", color='white')
        elif j in nodosX:
            dot.node(name=str(i), label=str(j), shape="square", color=colorHigh)
        else:
            dot.node(name=str(i), label=str(j), shape="square")
    if len(nodos) == 0:
        dot.node(name='-1', label='', shape="square", color='white')
        dot.edge(str(tuples[0][0]), str('-1'), color=colorPointer)
    else:
        for i in range(1,len(tuples)):
            if tuples[i-1][1] == 'First' and tuples[i][1] != 'None':
                dot.edge(str(tuples[i-1][0]), str(tuples[i][0]), color=colorPointer)
            elif tuples[i-1][1] == 'First' and tuples[i][1] == 'None':
                dot.edge(str(tuples[i-1][0]), str(tuples[i+1][0]), color=colorPointer)
            elif tuples[i-1][1] != 'None':
                dot.edge(str(tuples[i-1][0]), str(tuples[i][0]), color=colorRight)
        if tipo == 2:
            for i in range(1, len(tuples)-1):
                if tuples[i][1] != 'None':
                    dot.edge(str(tuples[i][0]), str(tuples[i-1][0]), color = colorLeft)
    
    if tipo == 2:
        dot2 = Digraph()
        dot2.graph_attr = {
            'rankdir': 'DT'
        }
        dot2.node(name='Colores', label='', color = 'black', shape="none", height="0.2", width = '0.01', fontsize='8.0')
        dot2.node(name='indicativoLeft', label='', color = 'black', shape="none", height="0.2", width = '0.01', fontsize='8.0')
        dot2.edge('indicativoLeft', 'Colores', color=colorLeft, minlen = '0.01', label='before', fontsize='8.0')

        dot3 = Digraph()
        dot3.node(name='Colores2', label='', color = 'black', shape="none", height="0.01", width = '0.01', fontsize='8.0')
        dot3.node(name='indicativoRight', label='', color = 'black', shape="none", height="0.01", width = '0.01', fontsize='8.0')
        dot3.edge('indicativoRight', 'Colores2', color=colorRight, minlen = '0.01', label='next', fontsize='8.0')

        dot2.subgraph(dot3)
        display(dot2)
        display(dot)
    else:
        display(dot)


### Métodos Gráficos - Árboles BST

In [11]:
from DISClib.ADT import orderedmap as omap
from DISClib.ADT import list as lt
from DISClib.Algorithms.Trees import traversal as tr

class BST_grafico():
    '''
    Clase auxiliar para graficar un arbol BST, usando la libreria graphviz
    adaptada de https://www.evamariakiss.de/apps/bstlearner_v1.php
    '''
    def __init__(self):
        self.estructura = omap.newMap(omaptype = 'BST', comparefunction = defaultfunction)

    def addNode_byValue(self, infoNodo):    
        self.estructura = omap.put(self.estructura, infoNodo, infoNodo)

    def getNodeValues(self):
        lst = list()
        if not omap.isEmpty(self.estructura):
            iter = lt.iterator(tr.preorder(self.estructura))
            for i in iter:
                lst.append(i)
        return lst

    def visualize(self, lst = []):
        tree = self.getNodeValues()
        if len(tree) > 0:
            tree = omap.get(self.estructura, tree[0])
            # Recursively add nodes and edges
            def add_nodes_edges(tree, dot=None):
                col = "black"
                # Create Graphviz Digraph 
                if dot is None:
                    dot = Digraph()
                    dot.graph_attr = {
                        'rankdir': 'TB',
                        'center': 'true',
                        'size':'14,5',
                        'ratio':'auto'
                    }
                    dot.node(name='root', label='root', color = 'white', shape="circle", fixedsize="True", width="0.4")
                    if (lst != [] and tree['value'] in lst):
                        dot.node(name=str(tree['value']), label=str(tree['value']), color = colorHigh, shape="circle", fixedsize="True", width="0.4")
                    else:
                        dot.node(name=str(tree['value']), label=str(tree['value']), color = col, shape="circle", fixedsize="True", width="0.4")
                    dot.edge('root', str(tree['value']), color=colorPointer)      
                
                # Add nodes recursively
                if tree['left'] != None:
                    if (lst != [] and tree['left']['value'] in lst):
                        col = "red"  
                    dot.node(name=str(tree['left']['value']), label=str(tree['left']['value']),
                            color = col, shape="circle", fixedsize="True", width="0.4")
                    col = "black"
                    dot.edge(str(tree['value']), str(tree['left']['value']),color=colorLeft)
                    dot = add_nodes_edges(tree['left'], dot=dot)
                else:
                    aux = get_random_string()
                    dot.node(name=aux, label='',
                            color = 'white', shape="circle", fixedsize="True", width="0.4")
                    dot.edge(str(tree['value']), aux,color=colorLeft)
                    #dot = add_nodes_edges(tree['left'], dot=dot)
                                
                if tree['right'] != None:
                    if (lst != [] and tree['right']['value'] in lst): 
                        col = "red" 
                    dot.node(name=str(tree['right']['value']), label=str(tree['right']['value']), 
                            color = col, shape="circle", fixedsize="True", width="0.4")
                    col = "black"
                    dot.edge(str(tree['value']), str(tree['right']['value']), color=colorRight)
                    dot = add_nodes_edges(tree['right'], dot=dot)            
                else:
                    aux = get_random_string()
                    dot.node(name=aux, label='',
                    color = 'white', shape="circle", fixedsize="True", width="0.4")
                    dot.edge(str(tree['value']), aux,color=colorRight)
                    #dot = add_nodes_edges(tree['left'], dot=dot)
                return dot        
            return add_nodes_edges(tree)                   

        else:
            dot = Digraph()
            dot.node(name='root', label='root', color = 'white', shape="circle", fixedsize="True", width="0.4")
            dot.node(name='-1', label='', shape="square", color='white')
            dot.edge('root', str(-1), color=colorPointer)
            return dot


In [12]:
def displayBST(estructura, nodosX = []):
    '''
    Grafica en el Canvas el arbol BST que entra por parametro y resalta los nodos
    que se encuentren en la lista nodosX
       
    Args:
        estructura: estructura de datos que debe ser una lista enlazada
        nodosX: lista de valores de nodos que se resaltan (pintan en otro color)
    
    Returns:
        -
    '''
    try:
        nodos = estructura.getNodeValues('Preorder')
    except:
        e = '\tProblema en el método getNodeValues()'
        raise Exception(e)
    
    bst = BST_grafico()
    for i in nodos:
        bst.addNode_byValue(i)
    dot = bst.visualize(nodosX)
    
    dot2 = Digraph()
    dot2.graph_attr = {
        'rankdir': 'DT'
    }
    dot2.node(name='Colores', label='', color = 'black', shape="none", height="0.2", width = '0.01', fontsize='8.0')
    dot2.node(name='indicativoLeft', label='', color = 'black', shape="none", height="0.2", width = '0.01', fontsize='8.0')
    dot2.edge('indicativoLeft', 'Colores', color=colorLeft, minlen = '0.01', label='left', fontsize='8.0')

    dot3 = Digraph()
    dot3.node(name='Colores2', label='', color = 'black', shape="none", height="0.01", width = '0.01', fontsize='8.0')
    dot3.node(name='indicativoRight', label='', color = 'black', shape="none", height="0.01", width = '0.01', fontsize='8.0')
    dot3.edge('indicativoRight', 'Colores2', color=colorRight, minlen = '0.01', label='right', fontsize='8.0')

    dot2.subgraph(dot3)
    display(dot2)
    display(dot)


### Métodos Gráficos - Grafos

In [13]:
def displayGraph(estructura, tipo, label, nodosX=list(), edgesX=list(), nodeY=None):
    '''
    Grafica en el Canvas el grafo que entra por parametro, del tipo dado, y resalta los nodos
    que se encuentren en la lista nodosX, resalta los arcos que se encuentren en la lista de edgesX,
    y si hay un valor para nodeY lo resalta de un color diferente
       
    Args:
        estructura: estructura de datos que debe ser un grafo
        tipo: tipo de la estructura (4: Dirigido, 2: No Dirigido)
        label: indica si se muestran o no los pesos de los arcos
        nodosX: lista de valores de nodos que se resaltan (pintan en otro color)
        edgesX: lista de valores de arcos (tuplas) que se resaltan (pintan en otro color)
        nodeY: nodo que se resalta en un color diferente a los anteriores
    
    Returns:
        -
    '''
    nodes = estructura.getNodeValues()
    edges = estructura.getEdgeValues()
    edgesAux = estructura.getEdgeValues()
    for i,j,k in edges:
        if j not in nodes:
            edgesAux.remove((i,j,k))    
    edges = edgesAux
    if tipo == 4:
        dot = Digraph()
    else:
        dot = Graph()
    
    dot.graph_attr = {
        'rankdir': 'TB',
        'center': 'true',
        'size':'14,5',
        'ratio':'auto',
        'layout': 'neato',
        'mode': 'sgd',
    }
    
    for i in nodes:
        if i in nodosX:
            dot.node(name=i, label=i, shape='circle',color='red')
        elif i == nodeY:
            dot.node(name=i, label=i, shape='circle',color='blue')
        else:
            dot.node(name=i, label=i, shape='circle',color='black')
    if tipo == 4:
        for i,j,k in edges:
            if (i,j) in edgesX:
                if label:
                    dot.edge(i, j, label=str(round(k,2)), color='red', fontsize='8.0')
                else:
                    dot.edge(i, j, color='red')
            else:
                if label:
                    dot.edge(i, j, label=str(round(k,2)), color='black', fontsize='8.0')
                else:
                    dot.edge(i, j, color='black')
    else:
        tuples = list()
        for i,j,k in edges:
            inv = (j, i)
            if inv not in tuples:
                if (i,j) in edgesX or inv in edgesX:
                    if label:
                        dot.edge(i, j, label=str(round(k,2)), color='red', fontsize='8.0')
                    else:
                        dot.edge(i, j, color='red')
                else:
                    if label:
                        dot.edge(i, j, label=str(round(k,2)), color='black', fontsize='8.0')
                    else:
                        dot.edge(i, j, color='black')
            tuples.append((i, j))
    display(dot)


### Métodos Gráficos - Arreglos

In [14]:
from graphviz import Digraph


def displayArreglo(estructura):
    '''
    Grafica en el Canvas la lista que entra por parametro
       
    Args:
        estructura: estructura de datos que debe ser un arreglo
    
    Returns:
        -
    '''
    try:
        nodos = estructura.getNodeValues()
    except:
        e = '\tProblema en el método getNodeValues()'
        raise Exception(e)
    
    dot = Digraph('structs', engine="dot", filename='structs_revisited.gv', node_attr={'shape': 'record'})
    
    nodo = ""
    cont = 1
    if len(nodos) > 0:
        for i in range(len(nodos)-1):
            nodo += "[" + str(cont) + "]\\n" + str(nodos[i]) + "|"
            cont += 1
        nodo += "[" + str(cont) + "]\\n" + str(nodos[-1])

        dot.node("struct1", nodo)
    else:
        dot.node("struct1", "Lista Vacia")

    display(dot)

### Métodos Gráficos - Hash LP

In [15]:
from graphviz import Digraph


def displayHashLP(estructura):
    '''
    Grafica en el Canvas la Tabla Hash LP que entra por parametro
       
    Args:
        estructura: estructura de datos debe ser una tabla hash
    
    Returns:
        -
    '''
    try:
        nodos = estructura.getNodeValues()
    except:
        e = '\tProblema en el método getNodeValues()'
        raise Exception(e)
    
    dot = Digraph('structs', engine="dot", filename='structs_revisited.gv', node_attr={'shape': 'record'})
    
    nodo = ""

    if len(nodos) > 0:
        for i in range(len(nodos)-1):
            if nodos[i] == "__EMPTY__":
                nodo += "[" + str(i+1) + ']\\n' + "Empty" + "|"
            else:
                nodo += "[" + str(i+1) + ']\\n' + str(nodos[i]) + "|"
        if nodos[-1] == "__EMPTY__":
            nodo += "[" + str(len(nodos)) + ']\\n' + "Empty"
        else:
            nodo += "[" + str(len(nodos)) + ']\\n' + str(nodos[-1])
        dot.node("struct1", nodo)
    else:
        dot.node("struct1", "TH LP Vacia")

    display(dot)

### Metodos Graficos - Separate Chaining

In [16]:
from graphviz import Digraph


def displaySC(estructura):
    '''
    Grafica en el Canvas la Tabla Hash SC que entra por parametro
       
    Args:
        estructura: estructura de datos debe ser una tabla hash
    
    Returns:
        -
    '''
    try:
        nodos = estructura.getNodeValues()
    except:
        e = '\tProblema en el método getNodeValues()'
        raise Exception(e)
    
    dot = Digraph('structs', engine="dot", filename='structs_revisited.gv', node_attr={'shape': 'record'})
    
    nodo = ""

    if len(nodos) > 0:
        for i in range(len(nodos)-1):
            nodo += "[" + str(i+1) + ']\\n' + str(nodos[i]) + "|"
        nodo += "[" + str(len(nodos)) + ']\\n' + str(nodos[-1])

        dot.node("struct1", nodo)
    else:
        dot.node("struct1", "TH SC Vacia")

    display(dot)

### Metodos Graficos - RBT

In [17]:
class RBT_grafico():
    '''
    Clase auxiliar para graficar un arbol RBT, usando la libreria graphviz
    adaptada de https://www.evamariakiss.de/apps/bstlearner_v1.php
    '''

    def __init__(self):
        self.estructura = omap.newMap(
            omaptype='RBT', comparefunction=defaultfunction)

    def addNode_byValue(self, infoNodo):
        self.estructura = omap.put(self.estructura, infoNodo, infoNodo)

    def addNode_byPosition(self, infoNodo, color):
        '''
        NEW FUNCTION
        Se agrega elemento en su posicion adecuada (sin necesidad de balancear) 
        y con su color respectivo.
        '''
        self.estructura = omap.put_by_position(self.estructura, infoNodo, infoNodo, color)

    def getNodeValues(self):
        lst = list()
        if not omap.isEmpty(self.estructura):
            iter = lt.iterator(tr.preorder(self.estructura))
            for i in iter:
                lst.append(i)
        return lst

    def visualize(self, lst=[]):
        tree = self.getNodeValues()
        if len(tree) > 0:
            tree = omap.get(self.estructura, tree[0])
            # Recursively add nodes and edges

            def add_nodes_edges(tree, dot=None):
                col = "black"
                # Create Graphviz Digraph
                if dot is None:
                    dot = Digraph()
                    dot.graph_attr = {
                        'rankdir': 'TB',
                        'center': 'true',
                        'size': '14,5',
                        'ratio': 'auto'
                    }
                    dot.node(name='root', label='root', color='white',
                             shape="circle", fixedsize="True", width="0.4")
                    if (lst != [] and tree['value'] in lst):
                        dot.node(name=str(tree['value']), label=str(tree['value']), color=colorHigh, shape="circle",
                                 fixedsize="True", width="0.4", style="filled", fillcolor=returnColor(tree), fontcolor="white")
                    else:
                        dot.node(name=str(tree['value']), label=str(tree['value']), color=col, shape="circle",
                                 fixedsize="True", width="0.4", style="filled", fillcolor=returnColor(tree), fontcolor="white")
                    dot.edge('root', str(tree['value']), color=colorPointer)

                # Add nodes recursively
                if tree['left'] != None:
                    col = "black"
                    if (lst != [] and tree['left']['value'] in lst):
                        col = "red"
                    dot.node(name=str(tree['left']['value']), label=str(tree['left']['value']),
                             color=col, shape="circle", fixedsize="True", width="0.4", style="filled", fillcolor=returnColor(tree["left"]), fontcolor="white")
                    #col = "black"
                    dot.edge(str(tree['value']), str(
                        tree['left']['value']), color=returnColor(tree["left"]))
                    dot = add_nodes_edges(tree['left'], dot=dot)
                else:
                    aux = get_random_string()
                    dot.node(name=aux, label='',
                             color='white', shape="circle", fixedsize="True", width="0.4")
                    dot.edge(str(tree['value']), aux, color=col)
                    #dot = add_nodes_edges(tree['left'], dot=dot)

                if tree['right'] != None:
                    col = "black"
                    if (lst != [] and tree['right']['value'] in lst):
                        col = "red"
                    dot.node(name=str(tree['right']['value']), label=str(tree['right']['value']),
                             color=col, shape="circle", fixedsize="True", width="0.4", style="filled", fillcolor=returnColor(tree["right"]), fontcolor="white")
                    #col = "black"
                    dot.edge(str(tree['value']), str(
                        tree['right']['value']), color=returnColor(tree["right"]))
                    dot = add_nodes_edges(tree['right'], dot=dot)
                else:
                    aux = get_random_string()
                    dot.node(name=aux, label='',
                             color='white', shape="circle", fixedsize="True", width="0.4")
                    dot.edge(str(tree['value']), aux, color=col)
                    #dot = add_nodes_edges(tree['left'], dot=dot)
                return dot
            return add_nodes_edges(tree)

        else:
            dot = Digraph()
            dot.node(name='root', label='root', color='white',
                     shape="circle", fixedsize="True", width="0.4")
            dot.node(name='-1', label='', shape="square", color='white')
            dot.edge('root', str(-1), color=colorPointer)
            return dot


def returnColor(node):
    if node["color"] == 0:
        return "red"
    return "black"


def displayRBT(estructura, nodosX=[]):
    '''
    Grafica en el Canvas el arbol RBT que entra por parametro y resalta los nodos
    que se encuentren en la lista nodosX

    Args:
        estructura: estructura de datos que debe ser un RBT
        nodosX: lista de valores de nodos que se resaltan (pintan en otro color)

    Returns:
        -
    '''
    try:
        nodos = estructura.getNodeValues('Preorder_with_color')
    except:
        e = '\tProblema en el método getNodeValues()'
        raise Exception(e)

    rbt = RBT_grafico()
    for i in nodos:
        # agregar cada llave en su posicion correcta (sin necesidad de balanceo) y color respectivo
        rbt.addNode_byPosition(i[0], i[1])  
    dot = rbt.visualize(nodosX)

    dot2 = Digraph()
    dot2.graph_attr = {
        'rankdir': 'DT'
    }
    dot2.node(name='Colores', label='', color='black', shape="none",
              height="0.2", width='0.01', fontsize='8.0')
    dot2.node(name='indicativoLeft', label='', color='black',
              shape="none", height="0.2", width='0.01', fontsize='8.0')
    dot2.edge('indicativoLeft', 'Colores', color=colorLeft,
              minlen='0.01', label='left', fontsize='8.0')

    dot3 = Digraph()
    dot3.node(name='Colores2', label='', color='black', shape="none",
              height="0.01", width='0.01', fontsize='8.0')
    dot3.node(name='indicativoRight', label='', color='black',
              shape="none", height="0.01", width='0.01', fontsize='8.0')
    dot3.edge('indicativoRight', 'Colores2', color=colorRight,
              minlen='0.01', label='right', fontsize='8.0')

    dot2.subgraph(dot3)
    #display(dot2)
    display(dot)

    # dot2.view()

    #dot.view()

## 3. Validación de Operaciones

En esta sección se implementan métodos que ejecutan en las estructuras de datos referencia las mismas operaciones que se realizan en la estructura de datos de prueba y determinan si el resultados de las estructuras de datos de prueba fue valido.

### Validacion - Listas Enlazadas

In [18]:
def validar_lista_crear(nodos, st_nodos, tipo):
    '''
    Valida la operacion de crear una lista enlazada
       
    Args:
        nodos: nodos que se añadieron al crear la lista enlazada
        st_nodos: nodos que la estructura tiene
        tipo: tipo de lista enlazada (1: sencilla, 2: doble)
               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    txt_nodos = ''
    for i in nodos:
        txt_nodos = txt_nodos + str(i) + ', '
    
    txt_st = ''
    for i in st_nodos:
        txt_st = txt_st + str(i) + ', '
        
    if len(nodos) == len(st_nodos):
        if nodos == st_nodos:
            if len(nodos) != 0:
                comment = 'Elementos: ' + txt_nodos[:-2]
            else:
                comment = 'Elementos: []'
            state_val = VALIDATION_STATES[1]
        else:
            nodos = sorted(nodos, key=functools.cmp_to_key(defaultfunction))
            st_nodos = sorted(st_nodos, key=functools.cmp_to_key(defaultfunction))
            
            if nodos == st_nodos:
                comment = 'Se añadieron los elementos pero en un orden diferente'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[0]
            else:
                comment = 'Se añadieron elementos diferentes a los indicados'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Se añadieron más elementos de los esperados'
        comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
        state_val = VALIDATION_STATES[-1]
        
    return state_val, comment

def validar_lista_anadir(init_test, end_test, nodo, tipo):
    '''
    Valida la operacion de añadir un elemento a la lista enlazada. 
    Se espera que el elemento se añada al final (contrato de los metodos de la estructura)
       
    Args:
        init_test: lista de elementos de la lista enlazada antes de ejecutar la operacion
        end_test: lista de elementos de la lista enlazada despues de ejecutar la operacion
        nodo: nodo que se añade a la lista enlazada
        tipo: tipo de lista enlazada (1: sencilla, 2: doble)
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''
    structure_ref = referenciaLista(tipo)
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    structure_ref.addNode_byValue(nodo)     #Ejecutar Funcion de Prueba
    end_val = structure_ref.getNodeValues() #Resultado de validacion
    
    if len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break                       
        if sameOrder:
            comment = 'El elemento "'+ str(nodo) + '" se añadió Bien'
            state_val = VALIDATION_STATES[1]
        else:
            test = set(end_test)
            val = set(end_val)
            if test == val:
                comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente, pero no en la posición esperada'
                state_val = VALIDATION_STATES[0]
            else:
                comment = 'El elemento "'+ str(nodo) + '" NO se añadió, hay un elemento adicional pero no corresponde al elemento esperado'
                state_val = VALIDATION_STATES[-1]
    elif len(end_test) == len(end_val) - 1: # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se añadió el elemento "' + str(nodo) + '"'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'El elemento "'+ str(nodo) + '" pudo haberse añadido, pero reemplazó a un elemento de la lista'
            state_val = VALIDATION_STATES[-1]
    elif len(end_test) < len(init_test): # Se eliminaron elementos
        comment = 'Se eliminaron elementos de la lista, hay menos de los que habian antes de ejecutar la operación de añadir'
        state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    
    return state_val, end_val, comment

def validar_enlaces_anteriores(init_test, end_test, end_test_reverse, operacion, nodo, tipo):
    '''
    CORRECCION Nueva funcion para lista doblemente encadenada
    Valida el contenido de los elementos de atras hacia adelante
    Args:
        init_test: lista de elementos de la lista enlazada antes de ejecutar la operacion
        end_test: lista de elementos de la lista enlazada despues de ejecutar la operacion hacia adelante
        end_test_reverse: lista de elementos de la lista enlazada despues de ejecutar la operacion hacia atras
    '''
    num_elementos_adelante = len(end_test)
    num_elementos_atras = len(end_test_reverse)

    state_val = VALIDATION_STATES[1]
    comment = 'OK recorridos hacia adelante y hacia atras \n'
      
    structure_ref = referenciaLista(tipo)
    for i in init_test:            #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)

    if operacion == 'addNodeLast':
        structure_ref.addNode_byValue(nodo)     #Ejecutar Funcion de Referencia agregar al final
    elif operacion == 'addNodeFirst':
        structure_ref.addNode_byValueFirst(nodo)     #Ejecutar Funcion de Referencia agregar al principio
    elif operacion == 'deleteNode':
        structure_ref.deleteNode_byValue(nodo)  #Ejecutar Funcion de Referencia eliminar nodo

    end_val = structure_ref.getNodeValues()                   #Resultado de validacion hacia adelante
    end_val_anteriores = structure_ref.getNodeValuesReverse() #Resultado de validacion hacia atras

    if num_elementos_adelante != num_elementos_atras:
        state_val = VALIDATION_STATES[-1]
        comment = 'Problema enlaces al interior de la lista resultante.\n'
        # comment += str(estructura.estructura) + '\n'            # contenido de la lista a probar (estudiante)
        comment += 'Diferencia en numero de elementos: ' + str(num_elementos_adelante) + ' elementos recorriendo hacia adelante y ' + str(num_elementos_atras) + ' recorriendo hacia atras\n'
        # comment += 'Recorrido elementos del primero al ultimo: ' + str(end_test) + '\n'
        # comment += 'Recorrido elementos del ultimo al primero: ' + str(end_test_reverse)
        return state_val, end_val, end_val_anteriores, comment

    if state_val == VALIDATION_STATES[1]:
        for i in range(len(end_test)):
            if defaultfunction(end_test[i], end_test_reverse[num_elementos_atras-1-i]) != 0:
                state_val = VALIDATION_STATES[-1]
                comment = 'Problema enlaces al interior lista resultante.\n'
                comment += 'Diferencia en los elementos recorriendo hacia adelante y recorriendo hacia atras'
                comment += 'El elemento hacia adelante en posicion ' + str(i) + ' deberia ser igual al elemento hacia atras en posicion ' + str(num_elementos_atras-1-i)
                comment += 'Elemento hacia adelante ' + str(end_test[i]) + ' NO es igual al elemento hacia atras ' + str(end_test_reverse[num_elementos_atras-1-i])
                return state_val, end_val, end_val_anteriores, comment

    # complemento al resultado cuando funciona bien
    # comment += str(estructura.estructura) + '\n'                 # contenido de la lista a probar (estudiante)
    # comment += str(num_elementos_adelante) + ' elementos recorriendo hacia adelante y ' + str(num_elementos_atras) + ' recorriendo hacia atras\n'
    # comment += 'Recorrido elementos del primero al ultimo: ' + str(end_test) + '\n'
    # comment += 'Recorrido elementos del ultimo al primero: ' + str(end_test_reverse)

    return state_val, end_val, end_val_anteriores, comment    

def validar_lista_anadir_first(init_test, end_test, nodo, tipo):
    '''
    Valida la operacion de añadir un elemento a la lista enlazada. 
    Se espera que el elemento se añada al final (contrato de los metodos de la estructura)
       
    Args:
        init_test: lista de elementos de la lista enlazada antes de ejecutar la operacion
        end_test: lista de elementos de la lista enlazada despues de ejecutar la operacion
        nodo: nodo que se añade a la lista enlazada
        tipo: tipo de lista enlazada (1: sencilla, 2: doble)
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''
    structure_ref = referenciaLista(tipo)
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    structure_ref.addNode_byValueFirst(nodo)     #Ejecutar Funcion de Prueba
    end_val = structure_ref.getNodeValues() #Resultado de validacion
    
    if len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break                       
        if sameOrder:
            comment = 'El elemento "'+ str(nodo) + '" se añadió Bien'
            state_val = VALIDATION_STATES[1]
        else:
            test = set(end_test)
            val = set(end_val)
            if test == val:
                comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente, pero no en la posición esperada'
                state_val = VALIDATION_STATES[0]
            else:
                comment = 'El elemento "'+ str(nodo) + '" NO se añadió, hay un elemento adicional pero no corresponde al elemento esperado'
                state_val = VALIDATION_STATES[-1]
    elif len(end_test) == len(end_val) - 1: # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se añadió el elemento "' + str(nodo) + '"'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'El elemento "'+ str(nodo) + '" pudo haberse añadido, pero reemplazó a un elemento de la lista'
            state_val = VALIDATION_STATES[-1]
    elif len(end_test) < len(init_test): # Se eliminaron elementos
        comment = 'Se eliminaron elementos de la lista, hay menos de los que habian antes de ejecutar la operación de añadir'
        state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    return state_val, end_val, comment

def validar_lista_eliminar(init_test, end_test, nodo, tipo, ans):
    '''
    Valida la operacion de eliminar un elemento de la lista enlazada. 
    Se espera que el elemento se elimine de la lista.
       
    Args:
        init_test: lista de elementos de la lista enlazada antes de ejecutar la operacion
        end_test: lista de elementos de la lista enlazada despues de ejecutar la operacion
        nodo: nodo que se elimina de la lista enlazada
        tipo: tipo de lista enlazada (1: sencilla, 2: doble)
        ans: resultado (True/False) de ejecutar el metodo deleteNode_byValue() en la estructura de prueba
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''
    structure_ref = referenciaLista(tipo)
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    ans2 = structure_ref.deleteNode_byValue(nodo)  #Ejecutar accion
    end_val = structure_ref.getNodeValues() #Resultado de validacion

    if len(end_test) != len(end_val):         # numero de elementos diferentes despues de la operacion
        detail = 'El numero de elementos resultado de la operacion es diferente. Se esperaba ' + str(len(end_val)) + ' y se obtuvo ' + str(len(end_test))
        comment = 'Problema en eliminacion de "'+ str(nodo) + '" \n' + detail
        state_val = VALIDATION_STATES[-1]
    elif ans == ans2 and ans:   # key aparecia en Lista y se elimino
        comment = 'El elemento "'+ str(nodo) + '" se eliminó satisfactoriamente'
        state_val = VALIDATION_STATES[1]
    elif ans == ans2:         # key No aparece en Lista y No se elimino
        comment = 'El elemento "'+ str(nodo) + '" NO esta en la Lista y NO se eliminó'
        state_val = VALIDATION_STATES[0]
    else:
        detail = 'El resultado de return del método es diferente al esperado. Se esperaba ' + str(ans2) + ' y se obtuvo ' + str(ans)
        comment = 'Problema en eliminacion de "'+ str(nodo) + '" \n' + detail
        state_val = VALIDATION_STATES[-1]

    '''
    # SIMPLIFICACION de validacion
    if nodo not in init_test:
        comment = "El elemento " + str(nodo) + " no se encuentra en la lista, por lo tanto no se puede eliminar"
        state_val = VALIDATION_STATES[0]
    elif len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break                       
        if sameOrder:
            comment = 'El elemento "'+ str(nodo) + '" se eliminó satisfactoriamente'
            state_val = VALIDATION_STATES[1]
        else:
            end_test = sorted(end_test, key=functools.cmp_to_key(defaultfunction))
            end_val = sorted(end_val, key=functools.cmp_to_key(defaultfunction))
            if(end_test == end_val):
                comment = 'Se eliminó una de las instancias del elemento "'+ str(nodo) + '"'
                state_val = VALIDATION_STATES[1]
            else:
                comment = 'El elemento "'+ str(nodo) + '" No se eliminó satisfactoriamente, se eliminó un elemento diferente'
                state_val = VALIDATION_STATES[-1]
    elif len(end_test) == len(init_test): # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != init_test[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se eliminó el elemento "' + str(nodo) + '". La lista no presenta cambios'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'No se eliminó el elemento "' + str(nodo) + '". La lista presenta cambios no esperados'
            state_val = VALIDATION_STATES[-1]     
    elif len(end_test) < len(end_val): # Se eliminaron mas elementos
        test = set(end_test)
        val = set(end_val)
        if nodo not in end_test:
            comment = 'Se eliminaron todas las instancias del elemento "' + str(nodo) +'" cuando solo una de ellas se debia eliminar'
            state_val = VALIDATION_STATES[0]
        elif test == val:
            comment = 'Se eliminaron más instancias del elemento "' + str(nodo) +'" cuando solo una de ellas se debia eliminar'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'Se eliminaron más elementos de los esperados'
            state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    
    if ans != ans2:
        ans2 = 'El resultado de return del método es diferente al esperado. Se esperaba ' + str(ans2) + ' y se obtuvo ' + str(ans)
        comment = comment + '\n' + ans2   
    '''
    return state_val, end_val, comment

def validar_lista_encontrar(nodos, nodo, ans):
    '''
    Valida la operacion de encontrar un elemento en la lista enlazada.
       
    Args:
        nodos: lista de elementos de la lista enlazada antes de ejecutar la operacion
        nodo: nodo que se busca en la lista enlazada
        ans: resultado (True/False) de ejecutar el metodo isNodeValue() en la estructura de prueba
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        ans_val: resultado (True/False) al ejecutar el metodo isNodeValue() en la estructura de referencia
        comment: mensaje informativo para el usuario
    '''
    if nodo in nodos:
        ans_val = True
    else:
        ans_val = False
    
    if ans == ans_val:
        if ans:
            comment = 'El elemento "'+ str(nodo) + '" SI se encuentra en la lista'
        else:
            comment = 'El elemento "'+ str(nodo) + '" NO se encuentra en la lista'
        state_val = VALIDATION_STATES[1]
    elif ans_val:
        comment = 'El elemento "'+ str(nodo) + '" SI se encuentra en la lista pero el método isNodeVale() reporta que NO se encuentra'
        state_val = VALIDATION_STATES[-1]
    else:
        comment = 'El elemento "'+ str(nodo) + '" NO se encuentra en la lista pero el método isNodeVale() reporta que SI se encuentra'
        state_val = VALIDATION_STATES[-1]
    
    return state_val, ans_val, comment

def validar_lista_adyacentes(init_test, listaAdj, nodo, tipo):
    '''
    Valida la operacion de encontrar los valores de los nodos adyacentes a un 
    elemento en la lista enlazada.
       
    Args:
        init_test: lista de elementos de la lista enlazada antes de ejecutar la operacion
        listaAdj: lista de valores de nodos adyacentes obtenida por la estructura de prueba
        nodo: nodo al cual se le buscan los adyacentes
        tipo: tipo de lista enlazada (1: sencilla, 2: doble)
               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        listaAdj_val: lista de los elementos adyacentes obtenidos por la estructura de referencia
        comment: mensaje informativo para el usuario
        exists: indica si el nodo existe en la lista
        
    '''
    structure_ref = referenciaLista(tipo)
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)

    listaAdj_val = structure_ref.findAdjacentNode(nodo)
    exists = structure_ref.isNodeValue(nodo)
    
    ''' CORRECCION Borrar
    listaAdj_val = sorted(listaAdj_val, key=functools.cmp_to_key(defaultfunction))
    listaAdj = sorted(listaAdj, key=functools.cmp_to_key(defaultfunction))
    '''

    if len(listaAdj) != len(listaAdj_val) or listaAdj != listaAdj_val:
        comment = 'Diferencias en los adyacentes\n'
        comment += 'Se esperaban los adyacentes: ' + str(listaAdj_val) + '\n'
        comment += 'Se obtuvieron los adyacentes: ' + str(listaAdj)
        state_val = VALIDATION_STATES[-1]
    else:
        comment = 'El elemento "'+ str(nodo)+ '" tiene '+str(len(listaAdj_val))+' adyacente(s): ' + str(listaAdj)
        state_val = VALIDATION_STATES[1]

    '''
    txt = ''
    for i in listaAdj_val:
        txt = txt + str(i) + ', '
        
    if listaAdj == listaAdj_val:
        comment = 'El elemento "'+ str(nodo)+ '" tiene '+str(len(listaAdj_val))+' adyacente(s): ' + txt[:-2]
        state_val = VALIDATION_STATES[1]
    else:
        comment = 'No se encontraron todos los adyacentes del elemento\nSe esperaban los elementos: ' + txt[:-2]
        txt = ''
        for i in listaAdj:
            txt = txt + str(i) + ', '
        comment = comment + '\nSe obtuvo: ' + txt[:-2]
        state_val = VALIDATION_STATES[-1]
    '''
    return state_val, listaAdj_val, comment, exists


### Validación - Árboles BST

In [19]:
def validar_bst_crear(nodos, end_test):
    '''
    Valida la operacion de crear un arbol BST
       
    Args:
        nodos: llaves que se añadieron al crear el BST
        end_test: llaves que tiene el BST 
               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: llaves en la estructura de referencia ("Preorder")
        comment: mensaje informativo para el usuario
        
    '''
    state_val = VALIDATION_STATES[1]
    end_val = list()
    comment = 'Sin validacion en el BST'
    # return state_val, end_val, comment

    structure_ref = referenciaBST()
    for key in nodos:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(key)
    
    end_val = structure_ref.getNodeValues("Preorder")
    
    if len(end_test) != len(end_val):         # numero de elementos diferentes despues de la operacion
        detail = 'El numero de llaves resultado de la operacion es diferente. Se esperaba ' + str(len(end_val)) + ' y se obtuvo ' + str(len(end_test))
        comment = 'Problema en crear BST ' + '\n' + detail
        state_val = VALIDATION_STATES[-1]

    else: # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if (end_test[i] != end_val[i]):
                sameOrder = False
                break
        if sameOrder:
            comment = 'Creación de BST satisfactoria'
            state_val = VALIDATION_STATES[1]
        else:
            comment = 'Creación de BST con llaves en orden diferente'
            state_val = VALIDATION_STATES[-1]

    return state_val, end_val, comment

    ''' SIMPLIFICACION DE LA VALIDACION
    txt_nodos = ''
    for i in nodos:
        txt_nodos = txt_nodos + str(i) + ', '
    
    txt_st = ''
    for i in st_nodos:
        txt_st = txt_st + str(i) + ', '
        
    if len(nodos) == len(st_nodos):
        if nodos == st_nodos:
            if len(nodos) != 0:
                comment = 'Elementos: ' + txt_nodos[:-2]
            else:
                comment = 'Elementos: []'
            state_val = VALIDATION_STATES[1]
        else:
            nodos = sorted(nodos, key=functools.cmp_to_key(defaultfunction))
            st_nodos = sorted(st_nodos, key=functools.cmp_to_key(defaultfunction))
            if nodos == st_nodos:
                comment = 'Se añadieron los elementos pero en un orden diferente'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[0]
            else:
                comment = 'Se añadieron elementos diferentes a los indicados'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Se añadieron más elementos de los esperados'
        comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
        state_val = VALIDATION_STATES[-1]
    '''

def validar_bst_anadir(init_test, end_test, nodo):
    '''
    Valida la operacion de añadir un elemento al BST. 
    Se espera que el elemento se añada en la posicion correcta
       
    Args:
        init_test: lista de elementos del BST antes de ejecutar la operacion
        end_test: lista de elementos del BST despues de ejecutar la operacion
        nodo: nodo que se añade al arbol
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos en Preorder de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''
    state_val = VALIDATION_STATES[1]
    end_val = list()
    comment = 'No se hace validacion.'
    # return state_val, end_val, comment

    structure_ref = referenciaBST()
    for key in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(key)
    structure_ref.addNode_byValue(nodo)     #Ejecutar accion
    end_val = structure_ref.getNodeValues("Preorder") #Resultado de validacion

    if len(end_test) != len(end_val):         # numero de llaves diferentes despues de la operacion
        detail = 'El numero de llaves resultado de la operacion es diferente.' + '\n'
        detail += 'Se esperaba ' + str(len(end_val)) + ' y se obtuvo ' + str(len(end_test))
        comment = 'Problema entre BST creado y BST de validación' + '\n' + detail
        state_val = VALIDATION_STATES[-1]

    else: # Hay la cantidad esperada de llaves
        sameOrder = True
        for i in range(len(end_test)):
            if (end_test[i] != end_val[i]):
                sameOrder = False
                break
        if sameOrder:
            comment = 'Validación de BST satisfactoria'
            state_val = VALIDATION_STATES[1]
        else:
            comment = 'BST de validación con llaves en orden diferente'
            state_val = VALIDATION_STATES[-1]

    return state_val, end_val, comment

    ''' SIMPLIFICACION DE LA VALIDACION    
    if len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        sameOrder = True
        if len(init_test) == len(end_val):
            comment = 'El elemento "'+ str(nodo) + '" ya se encontraba en el bst'
            state_val = VALIDATION_STATES[1]
        else:
            for i in range(len(end_test)):
                if end_test[i] != end_val[i]:
                    sameOrder = False
                    break                       
            if sameOrder:
                comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente'
                state_val = VALIDATION_STATES[1]
            else:
                test = set(end_test)
                val = set(end_val)
                if test == val:
                    comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente, pero no en la posición esperada'
                    state_val = VALIDATION_STATES[0]
                else:
                    comment = 'El elemento "'+ str(nodo) + '" NO se añadió, hay un elemento adicional pero no corresponde al elemento esperado'
                    state_val = VALIDATION_STATES[-1]
    elif len(end_test) == len(end_val) - 1: # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se añadió el elemento "' + str(nodo) + '"'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'El elemento "'+ str(nodo) + '" pudo haberse añadido, pero reemplazó a un elemento de la lista'
            state_val = VALIDATION_STATES[-1]
    elif len(end_test) < len(init_test): # Se eliminaron elementos
        comment = 'Se eliminaron elementos de la lista, hay menos de los que habian antes de ejecutar la operación de añadir'
        state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    '''

def validar_bst_eliminar(init_test, end_test, nodo, ans):
    '''
    Valida la operacion de eliminar un elemento del BST. 
       
    Args:
        init_test: lista de elementos del BST antes de ejecutar la operacion
        end_test: lista de elementos del BST despues de ejecutar la operacion
        nodo: nodo que se elimina del BST
        ans: resultado (True/False) de ejecutar el metodo deleteNode_byValue() en la estructura de prueba
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''
    state_val = VALIDATION_STATES[1]
    end_val = list()
    comment = 'No se hace validacion.'
    # return state_val, end_val, comment

    structure_ref = referenciaBST()
    for key in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(key)
    ans2 = structure_ref.deleteNode_byValue(nodo)  #Ejecutar accion
    end_val = structure_ref.getNodeValues("Preorder") #Resultado de validacion
    # return state_val, end_val, comment

    if len(end_test) != len(end_val):         # numero de elementos diferentes despues de la operacion
        detail = 'El numero de llaves resultado de la operacion es diferente.' + '\n'  
        detail += 'Se esperaba ' + str(len(end_val)) + ' y se obtuvo ' + str(len(end_test))
        comment = 'Problema en eliminacion de "'+ str(nodo) + '" \n' + detail
        state_val = VALIDATION_STATES[-1]
    elif ans != ans2:
        detail = 'El resultado de return de la función es diferente al esperado.' + '\n'
        detail += 'Se esperaba ' + str(ans2) + ' y se obtuvo ' + str(ans)
        comment = 'Problema en eliminacion de "'+ str(nodo) + '" \n' + detail
        state_val = VALIDATION_STATES[-1]
    elif ans == ans2 and ans:   # key aparecia en arbol BST y se elimino
        sameOrder = True
        for i in range(len(end_test)):
            if (end_test[i] != end_val[i]):
                sameOrder = False
                break
        if sameOrder:
            comment = 'Validación de BST satisfactoria'
            state_val = VALIDATION_STATES[1]
        else:
            comment = 'Validación de BST con llaves en orden diferente'
            state_val = VALIDATION_STATES[-1]
    elif ans == ans2:         # key No aparecia en arbol BST y No se elimino
        comment = 'La llave "'+ str(nodo) + '" NO esta en el arbol BST y NO se eliminó'
        state_val = VALIDATION_STATES[0]

    return state_val, end_val, comment

    ''' SIMPLIFICACION VALIDACION 

    if nodo not in init_test:
        comment = "El elemento " + str(nodo) + " no se encuentra en el arbol, por lo tanto no se puede eliminar"
        state_val = VALIDATION_STATES[0]
      
    elif len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break                       
        if sameOrder and end_test != init_test:
            comment = 'El elemento "'+ str(nodo) + '" se eliminó satisfactoriamente'
            state_val = VALIDATION_STATES[1]
        else:
            comment = 'El elemento "'+ str(nodo) + '" No se pertenece al BST'
            state_val = VALIDATION_STATES[0]
    elif len(end_test) == len(init_test): # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != init_test[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se eliminó el elemento "' + str(nodo) + '". La estructura no presenta cambios'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'No se eliminó el elemento "' + str(nodo) + '". La estructura presenta cambios no esperados'
            state_val = VALIDATION_STATES[-1]     
    elif len(end_test) < len(end_val): # Se eliminaron mas elementos
        test = set(end_test)
        val = set(end_val)
        if nodo not in end_test:
            comment = 'Se eliminaron todas las instancias del elemento "' + str(nodo) +'" cuando solo una de ellas se debia eliminar'
            state_val = VALIDATION_STATES[0]
        elif test == val:
            comment = 'Se eliminaron más instancias del elemento "' + str(nodo) +'" cuando solo una de ellas se debia eliminar'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'Se eliminaron más elementos de los esperados'
            state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    
    if ans != ans2:
        ans2 = 'El resultado de return del método es diferente al esperado. Se esperaba ' + str(ans2) + ' y se obtuvo ' + str(ans)
        comment = comment + '\n' + ans2   
    '''

def validar_bst_encontrar(nodos, nodo, ans):
    '''
    Valida la operacion de encontrar un elemento en el BST.
       
    Args:
        nodos: lista de elementos del BST antes de ejecutar la operacion
        nodo: nodo que se busca en el BST
        ans: resultado (True/False) de ejecutar el metodo isNodeValue() en la estructura de prueba
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_ans: resultado (True/False) de ejecutar el metodo isNodeValue() en la estructura de referencia
        end_val: lista de llaves en la estructura de referencia
        comment: mensaje informativo para el usuario
    '''

    state_val = VALIDATION_STATES[1]
    end_val = list()
    end_ans = False
    comment = 'No se hace validacion.'
    # return state_val, end_ans, end_val, comment

    structure_ref = referenciaBST()
    for key in nodos:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(key)
    end_ans = structure_ref.isNodeValue(nodo)  #Ejecutar accion
    end_val = structure_ref.getNodeValues("Preorder") #Resultado de validacion

    if len(nodos) != len(end_val):         # numero de elementos diferentes despues de la operacion
        detail = 'Se esperaba BST de validación con ' + str(len(nodos)) + ' y se obtuvo ' + str(len(end_val))
        comment = 'Problema en inicializacion del BST de validación. '+ '\n' + detail
        state_val = VALIDATION_STATES[-1]
        return state_val, end_ans, end_val, comment

    if ans == end_ans:
        if ans:
            comment = 'El elemento "'+ str(nodo) + '" SI se encuentra en el BST'
            state_val = VALIDATION_STATES[1]
        else:
            comment = 'El elemento "'+ str(nodo) + '" NO se encuentra en el BST'
            state_val = VALIDATION_STATES[0]
    elif end_ans:
        comment = 'El elemento "'+ str(nodo) + '" SI se encuentra en el BST pero el método isNodeValue() reporta que NO se encuentra'
        state_val = VALIDATION_STATES[-1]
    else:
        comment = 'El elemento "'+ str(nodo) + '" NO se encuentra en el BST pero el método isNodeValue() reporta que SI se encuentra'
        state_val = VALIDATION_STATES[-1]
    
    return state_val, end_ans, end_val, comment

def validar_bst_adyacentes(init_test, existe, listaAdj, nodo):
    '''
    Valida la operacion de encontrar los valores de los nodos adyacentes a un 
    elemento en el BST.
       
    Args:
        init_test: lista de elementos del BST antes de ejecutar la operacion
        existe: si el nodo se encuentra o No en el BST
        listaAdj: lista de valores de nodos adyacentes obtenida por la estructura de prueba
        nodo: nodo al cual se le buscan los adyacentes
               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        existe_val: existe nodo en la estructura de referencia
        listaAdj_val: lista de los elementos adyacentes obtenidos por la estructura de referencia
        comment: mensaje informativo para el usuario        
    '''
    state_val = VALIDATION_STATES[1]
    existe_val = False
    listaAdj_val = list()
    comment = 'No se hace validacion.'
    # return state_val, existe_val, listaAdj_val, comment

    print("No se hace validacion.", existe_val, listaAdj_val)

    structure_ref = referenciaBST()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    existe_val, listaAdj_val = structure_ref.findAdjacentNode(nodo)   
    
    print("validacion", existe_val, listaAdj_val)

    ''' SIMPLIFICACION VALIDACION
    listaAdj_val = sorted(listaAdj_val, key=functools.cmp_to_key(defaultfunction))
    listaAdj = sorted(listaAdj, key=functools.cmp_to_key(defaultfunction))
    txt = ''
    for i in listaAdj_val:
        txt = txt + str(i) + ', '
    '''

    print("caso validacion 1")
    if existe != existe_val:
        detail = 'Se esperaba existencia de la llave ' + str(existe_val) + ' y se obtuvo ' + str(existe)
        comment = 'Problema en la existencia de la llave '+ '\n' + detail
        state_val = VALIDATION_STATES[-1]
        return state_val, existe_val, listaAdj_val, comment

    print("caso validacion 2")
    if len(listaAdj) != len(listaAdj_val):         # numero de elementos diferentes despues de la operacion
        detail = 'Se esperaban ' + str(len(listaAdj_val)) + ' adyacentes y se obtuvieron ' + str(len(listaAdj))
        comment = 'Problema en el numero de adyacentes. '+ '\n' + detail
        state_val = VALIDATION_STATES[-1]
        return state_val, existe_val, listaAdj_val, comment

    print("caso validacion 3")
    if listaAdj == listaAdj_val:
        if existe_val:
            comment = 'La llave "'+ str(nodo) + '" existe y tiene ' + str(len(listaAdj_val)) + ' adyacente(s)'
            state_val = VALIDATION_STATES[1]
        else:
            comment = 'La llave "'+ str(nodo) + '" NO existe y tiene ' + str(len(listaAdj_val)) + ' adyacente(s)'
            state_val = VALIDATION_STATES[0]
    else:
        comment = 'Hay diferencia en los adyacentes de la llave "' + str(nodo) + '"'
        state_val = VALIDATION_STATES[-1]
     
    return state_val, existe_val, listaAdj_val, comment

def validar_bst_darNodos(init_test, nodos, orden):
    '''
    Valida la operacion de encontrar listar los valores de los nodos de un arbol dado un orden
       
    Args:
        init_test: lista de elementos del arbol en preorder
        nodos: lista de valores de nodos obtenida de listar los nodos en el orden dado
        orden: orden en el cual se listan los nodos (preorder, inorder, postorder)
               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        nodos_val: lista de los elementos en el orden dado, en la estructura de referencia
        comment: mensaje informativo para el usuario
        
    '''
    state_val = VALIDATION_STATES[1]
    nodos_val = list()
    comment = 'No se hace validacion.'
    # return state_val, nodos_val, comment

    structure_ref = referenciaBST()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    
    nodos_val = structure_ref.getNodeValues(orden)
    
    '''
    txt_val = ''
    for i in nodos_val:
        txt_val = txt_val + str(i) + ', '
        
    txt_test = ''
    for i in nodos:
        txt_test = txt_test + str(i) + ', '
    '''
    if len(nodos) != len(nodos_val):         # numero de elementos diferentes despues de la operacion
        detail = 'El numero de llaves resultado del recorrido es diferente. Se esperaba ' + str(len(nodos_val)) + ' y se obtuvo ' + str(len(nodos))
        comment = 'Problema en recorrido ' + orden + '\n' + detail
        state_val = VALIDATION_STATES[-1]

    else: # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(nodos)):
            if (nodos[i] != nodos_val[i]):
                sameOrder = False
                break
        if sameOrder:
            comment = 'Recorrido ' + orden + ' satisfactorio'
            state_val = VALIDATION_STATES[1]
        else:
            comment = 'Recorrido ' + orden + ' en orden diferente'
            state_val = VALIDATION_STATES[-1]

    return state_val, nodos_val, comment


### Validación - Grafos

In [20]:
def validar_graph_crear(nodos, edges, nodos_test, edges_test, tipo):
    '''
    Valida la operacion de crear un grafo dado un tipo
       
    Args:
        nodos: nodos que se añadieron al crear el grafo
        edges: arcos que se añadieron al crear el grafo
        nodos_test: nodos que la estructura tiene
        edges_test: arcos que tiene el grafo
        tipo: tipo del grafo (4: Dirigido, 5: No dirigido)
               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    state_val = VALIDATION_STATES[1]
    detail = 'numero nodos: ' + str(len(nodos_test)) + '\n' + 'numero arcos:' + str(len(edges_test))
    comment = 'Sin Validacion de Creacion de Grafo - ' + str(tipo) + '\n' + detail
    #return state_val, comment

    if tipo == 4:
        structure_ref = referenciaGrafo('Directed')
    else:
        structure_ref = referenciaGrafo('Undirected')
    for i in nodos:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)

    nodos = structure_ref.getNodeValues()
    
    for i,j,k in edges:                     #Inicializacion de la estructura de prueba
        structure_ref.addEdge_byValue(i,j,k)

    edges = structure_ref.getEdgeValues()
    
    if len(nodos) != len(nodos_test):
        comment = 'Numero de vertices esperado: ' + str(len(nodos)) + ' diferente a vertices actuales: ' + str(len(nodos_test))
        state_val = VALIDATION_STATES[-1]
    elif len(edges) != len(edges_test):
        comment = 'Numero de arcos esperado: ' + str(len(edges)) + ' diferente a arcos actuales: ' + str(len(edges_test))
        state_val = VALIDATION_STATES[-1]
    else:
        detail = 'Numero vertices: ' + str(len(nodos_test)) + '\n' + 'Numero arcos:' + str(len(edges_test))
        if tipo == 4:
            comment = 'El Grafo Dirigido se creó satisfactoriamente' + '\n' + detail
        else:
            comment = 'El Grafo No Dirigido se creó satisfactoriamente' + '\n' + detail
        state_val = VALIDATION_STATES[1]
    return state_val, comment   

    ''' SIMPLIFICACION VALIDACION
    txt_nodos_ref = ''
    for i in nodos:
        txt_nodos_ref = txt_nodos_ref + str(i) + ', '
    txt_nodos_test = ''
    for i in nodos_test:
        txt_nodos_test = txt_nodos_test + str(i) + ', '
        
    validation = 0    
    state_val = VALIDATION_STATES[0]
    comment = ''
    
    if len(nodos) == len(nodos_test):
        nodos = sorted(nodos, key=functools.cmp_to_key(defaultfunction))
        nodos_test = sorted(nodos_test, key=functools.cmp_to_key(defaultfunction))
        if nodos == nodos_test:
            validation = validation + 1
        else:
            comment = 'No se tienen los vertices que se esperan tener'
            comment = comment + '\nSe esperaba: ' + txt_nodos_ref[:-2]
            comment = comment + '\n  Se obtuvo: ' + txt_nodos_test[:-2]
            state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Se tiene una cantidad diferente de vertices a los esperados'
        comment = comment + '\nSe esperaba: ' + txt_nodos_ref[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_nodos_test[:-2]
        state_val = VALIDATION_STATES[-1]
        
    txt_edges_ref = ''
    for i in edges:
        txt_edges_ref = txt_edges_ref + str(i) + ', '
    txt_edges_test = ''
    for i in edges_test:
        txt_edges_test = txt_edges_test + str(i) + ', '
        
    if len(edges) == len(edges_test):
        edges.sort()
        edges_test.sort()
        if edges == edges_test:
            validation = validation + 1
        else:
            comment = comment + '\nNo se tienen los arcos que se esperan tener'
            comment = comment + '\nSe esperaba: ' + txt_nodos_ref[:-2]
            comment = comment + '\n  Se obtuvo: ' + txt_nodos_test[:-2]
            state_val = VALIDATION_STATES[-1]
    else:
        comment = comment + '\nSe tiene una cantidad diferente de arcos a los esperados'
        comment = comment + '\nSe esperaba: ' + txt_nodos_ref[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_nodos_test[:-2]
        state_val = VALIDATION_STATES[-1]
        
    if validation == 2:
        comment = 'El grafo se creó satisfactoriamente'
        state_val = VALIDATION_STATES[1]
    '''


def validar_graph_anadir(init_test, end_test, tipo, nodo):
    '''
    Valida la operacion de un nodo al grafo
       
    Args:
        init_test: nodos que la estructura tiene antes de añadir el nuevo nodo
        end_test: nodos que la estructura tiene despues de añadir el nuevo nodo
        tipo: tipo del grafo (4: Dirigido, 5: No dirigido)
        nodo: nodo que se añade al grafo               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    if tipo == 4:
        structure_ref = referenciaGrafo('Directed')
    else:
        structure_ref = referenciaGrafo()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    structure_ref.addNode_byValue(nodo)
    end_ref = structure_ref.getNodeValues()
    
    txt_nodos_ref = ''
    for i in end_ref:
        txt_nodos_ref = txt_nodos_ref + str(i) + ', '
    txt_nodos_test = ''
    for i in end_test:
        txt_nodos_test = txt_nodos_test + str(i) + ', '

    if len(end_ref) != len(end_test):
        comment = 'Se tiene una cantidad diferente de vertices a los esperados'
        state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Adición de vértice satisfactoria'
        state_val = VALIDATION_STATES[1]

    return state_val, comment, end_ref
    '''
    # SIMPLIFICACION VALIDACION
    if len(end_ref) == len(end_test):
        end_ref = sorted(end_ref, key=functools.cmp_to_key(defaultfunction))
        end_test = sorted(end_test, key=functools.cmp_to_key(defaultfunction))
        init_test = sorted(init_test, key=functools.cmp_to_key(defaultfunction))
        if end_test == end_ref and end_ref != init_test:
            comment = "El vertice se añadió correctamente \n El grafo tiene " + str(len(end_test)) + ' vertices.'
            state_val = VALIDATION_STATES[1]
        elif end_test == end_ref:
            comment = 'El vertice ya se encuentra en el grafo'
            state_val = VALIDATION_STATES[0]
        else:
            comment = 'No se tienen los vertices que se esperan tener'
            comment = comment + '\nSe esperaba: ' + txt_nodos_ref[:-2]
            comment = comment + '\n  Se obtuvo: ' + txt_nodos_test[:-2]
            state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Se tiene una cantidad diferente de vertices a los esperados'
        comment = comment + '\nSe esperaba: ' + txt_nodos_ref[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_nodos_test[:-2]
        state_val = VALIDATION_STATES[-1]
    '''

def validar_graph_eliminar(init_test, end_test, tipo, nodo):
    '''
    Valida la operacion de un eliminar un nodo del grafo
       
    Args:
        init_test: nodos que la estructura tiene antes de eliminar el nodo
        end_test: nodos que la estructura tiene despues de eliminar el nodo
        tipo: tipo del grafo (4: Dirigido, 5: No dirigido)
        nodo: nodo que se elimina del grafo               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    if tipo == 4:
        print('structure ref dirrected')
        structure_ref = referenciaGrafo('Directed')
    else:
        structure_ref = referenciaGrafo()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    structure_ref.deleteNode_byValue(nodo)
    end_ref = structure_ref.getNodeValues()

    txt_nodos_ref = ''
    for i in end_ref:
        txt_nodos_ref = txt_nodos_ref + str(i) + ', '
    txt_nodos_test = ''
    for i in end_test:
        txt_nodos_test = txt_nodos_test + str(i) + ', '
        
    state_val = VALIDATION_STATES[0]
    comment = ''
    
    if nodo not in init_test:
        comment = 'El vertice ' + str(nodo) + ' no existe en el grafo, por lo tanto no se puede eliminar'
        
    elif len(end_ref) == len(end_test):
        end_ref = sorted(end_ref, key=functools.cmp_to_key(defaultfunction))
        end_test = sorted(end_test, key=functools.cmp_to_key(defaultfunction))
        init_test = sorted(init_test, key=functools.cmp_to_key(defaultfunction))
        if end_test == end_ref and end_ref != init_test:
            comment = 'El vertice se eliminó correctamente'
            state_val = VALIDATION_STATES[1]
        elif end_test == end_ref:
            comment = 'El vertice NO se encuentra en el grafo'
            state_val = VALIDATION_STATES[0]
        else:
            comment = 'No se tienen los vertices que se esperan tener'
            comment = comment + '\nSe esperaba: ' + txt_nodos_ref[:-2]
            comment = comment + '\n  Se obtuvo: ' + txt_nodos_test[:-2]
            state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Se tiene una cantidad diferente de vertices a los esperados'
        comment = comment + '\nSe esperaba: ' + txt_nodos_ref[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_nodos_test[:-2]
        state_val = VALIDATION_STATES[-1]
    
    return state_val, comment   

def validar_graph_encontrar(init_test, existe_test, nodo):
    '''
    Valida la operacion de un verificar si un nodo pertenece al grafo
       
    Args:
        init_test: nodos que tiene la estructura de prueba 
        existe_test: resultado de la verificacion por parte de la estructura de prueba
        nodo: nodo que se verifica la pertenencia al grafo               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    existe_ref = str(nodo) in init_test
    
    state_val = VALIDATION_STATES[0]
    comment = ''
    
    if existe_test == existe_ref:
        if existe_test:
            comment = 'El vertice "' + str(nodo) + '" SI se encuentra en el grafo'
        else:
            comment = 'El vertice "' + str(nodo) + '" NO se encuentra en el grafo'
        state_val = VALIDATION_STATES[1]
    elif existe_ref:
        comment = 'El vertice SI se encuentra en el grafo, pero el método isNodeValue() indica que NO'
        state_val = VALIDATION_STATES[-1]
    else:
        comment = 'El vertice NO se encuentra en el grafo, pero el método isNodeValue() indica que SI'
        state_val = VALIDATION_STATES[-1]
    
    return state_val, comment   

def validar_graph_anadirEdge(nodes, init_test, end_test, tipo, origen, destino, peso):
    '''
    Valida la operacion de un añadir un arco al grafo
       
    Args:
        nodes: nodos de la estructura de prueba
        init_test: arcos que la estructura de prueba tiene antes de añadir el arco
        end_test: arcos que la estructura de prueba tiene despues de añadir el arco
        tipo: tipo del grafo (4:dirigido, 5:no dirigido)
        origen: nodo origen del arco
        destino: nodo destino del arco
        peso: peso del arco
        
    Returns:
        state_val: Estado final de la validacion (FAILED, WARNING, SUCCESSFUL)
        comment: mensaje informativo para el usuario
        end_ref: arcos resultantes en el grafo
        
    '''
    if tipo == 4:
        structure_ref = referenciaGrafo('Directed')
    else:
        structure_ref = referenciaGrafo('Undirected')
    for i in nodes:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    for i,j,k in init_test:
        structure_ref.addEdge_byValue(i,j,k)
        
    structure_ref.addEdge_byValue(origen,destino,peso)  # funcion de prueba
    end_ref = structure_ref.getEdgeValues()

    if len(end_ref) != len(end_test):
        comment = 'Se tiene una cantidad diferente de Arcos a los esperados'
        state_val = VALIDATION_STATES[-1]
        return state_val, comment, end_ref

    ''' SIMPLIFICACION VALIDACION
    if len(end_ref) == len(end_test):
        end_ref.sort()
        end_test.sort()
        if end_test == end_ref:
            arco = '(' + str(origen) + ' -> ' + str(destino) + ',' + str(peso) + ')'
            comment = 'Se añadió el arco "' + arco + '" correctamente' + ' \n Hay ' + str(len(end_test)) + ' Arcos en el grafo'
            state_val = VALIDATION_STATES[1]
        else:
            comment = 'No se tienen los arcos que se esperan tener'
            comment = comment + '\nSe esperaba: ' + txt_edges_ref[:-2]
            comment = comment + '\n  Se obtuvo: ' + txt_edges_test[:-2]
            state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Se tiene una cantidad diferente de Arcos a los esperados'
        comment = comment + '\nSe esperaba: ' + txt_edges_ref[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_edges_test[:-2]
        state_val = VALIDATION_STATES[-1]
    '''
    if tipo == 4:
        comment = 'Agregar arco ' + str(origen) + ' -> ' + str(destino) + ' weight: ' + str(peso)
    else:
        comment = 'Agregar arco ' + str(origen) + ' <-> ' + str(destino) + ' weight: ' + str(peso)
    state_val = VALIDATION_STATES[1]

    return state_val, comment, end_ref

def validar_graph_adj(nodes, edges, adjNodes, nodo, tipo):
    '''
    Valida la operacion de un encontrar los adyacentes de u nodo en el grafo
       
    Args:
        nodes: nodos de la estructura de prueba
        edges: arcos de la estructura de prueba
        adjNodes: nodos que la estructura de prueba determina como adyacentes de nodo
        nodo: nodo del que se buscan los adyacentes
        tipo: tipo del grafo (4:dirigido, 5:no dirigido)
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    #---------Inicializacion de la estructura de prueba
    if tipo == 4:
        structure_ref = referenciaGrafo('Directed')
        print('Directed')
    else:
        structure_ref = referenciaGrafo()
        print('No Directed')
    for i in nodes:                     
        structure_ref.addNode_byValue(i)
    for i,j,k in edges:
        structure_ref.addEdge_byValue(i,j,k)    

    adjNodes_ref = structure_ref.findAdjacentNode(nodo)
    aux = structure_ref.findAdjacentNode(nodo)
    #---------
    if nodo not in nodes:
        comment = 'El nodo "' +str(nodo) + '" no se encuentra en el grafo'
        state_val = VALIDATION_STATES[0]
    else:
        adjNodes = list(dict.fromkeys(adjNodes))
        adjNodes_ref = list(dict.fromkeys(adjNodes_ref))
        
        adjText = ''
        for i in adjNodes:
            adjText = adjText + str(i) + ', '
        adjText_ref = ''
        for i in adjNodes_ref:
            adjText_ref = adjText_ref + str(i) + ', '
        
        if len(adjNodes) == len(adjNodes_ref):
            adjNodes.sort()
            adjNodes_ref.sort()
            if adjNodes == adjNodes_ref:
                comment = 'El nodo "' + str(nodo) + '" tiene ' + str(len(adjNodes)) + ' adyacentes: ' + adjText[:-2]
                state_val = VALIDATION_STATES[1]
            else:
                comment = 'Los adyacentes del elementos no son los esperados'
                comment = comment + '\nSe esperaba: ' + adjText_ref[:-2]
                comment = comment + '\n  Se obtuvo: ' + adjText[:-2]
                state_val = VALIDATION_STATES[-1]
        else:
            for i in adjNodes_ref:
                if i not in nodes:
                    aux.remove(i)
            adjNodes_ref = aux
            
            adjNodes.sort()
            adjNodes_ref.sort()
            if adjNodes == adjNodes_ref:
                comment = 'El nodo "' + str(nodo) + '" tiene ' + str(len(adjNodes)) + ' adyacentes: ' + adjText[:-2]
                state_val = VALIDATION_STATES[1]
            else:
                comment = 'Los adyacentes de los elementos no son los esperados'
                comment = comment + '\nSe esperaba: ' + adjText_ref[:-2]
                comment = comment + '\n  Se obtuvo: ' + adjText[:-2]
                state_val = VALIDATION_STATES[-1]

    return state_val, comment 

def validar_graph_todos(nodes, edges, tipo):
    '''
    Valida la operacion de un encontrar todos los nodos de un grafo
       
    Args:
        nodes: nodos de la estructura de prueba
        tipo: tipo del grafo (4:dirigido, 5:no dirigido)
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    #---------Inicializacion de la estructura de prueba
    if tipo == 4:
        structure_ref = referenciaGrafo('Directed')
    else:
        structure_ref = referenciaGrafo()
    for i in nodes:                     
        structure_ref.addNode_byValue(i)
    
    for o,d,k in edges:
        structure_ref.addEdge_byValue(o,d,k)
    
    nodes_ref = structure_ref.getNodeValues()
    edges_ref = structure_ref.getEdgeValues()
    
    txt_nodos_ref = ''
    for i in nodes_ref:
        txt_nodos_ref = txt_nodos_ref + str(i) + ', '
    txt_nodos_test = ''
    for i in nodes:
        txt_nodos_test = txt_nodos_test + str(i) + ', '
        
    nodes_ref.sort()
    nodes.sort()
    if nodes_ref == nodes and len(edges_ref) == len(edges):
        comment = 'Hay ' + str(len(nodes)) + ' Vértices en el grafo: ' + txt_nodos_ref[:-2]
        comment += '\n Hay ' + str(len(edges_ref)) + ' Arcos en el grafo: ' + str(edges_ref)
        state_val = VALIDATION_STATES[1]
    else:
        comment = 'No se encontraron todos los Elementos del grafo'
        comment = comment + '\n Se esperaban: ' + txt_nodos_ref[:-2]
        comment = comment + '\n    Se obtuvo: ' + txt_nodos_test[:-2]
        state_val = VALIDATION_STATES[-1]
    return state_val, comment

def validarRecorridosGrafo(estructura, tipo, tipo_inData, recorrido, result_test, nodo=None): #TODO
    '''
    Valida los resultados de ejecutar algoritmos sobre el grafo de prueba
       
    Args:
        estructura: grafo de prueba
        tipo: tipo del grafo (4:dirigido, 5:no dirigido)
        tipo_inData: tipo de la salida del argoritmo
        recorrido: algoritmo que se aplica
        result_test: resultado de ejecutar el algoritmo
        nodo: nodo de inicio del algoritmo
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    state_val = ''
    comment = ''
    posError = ' '
    nodos = estructura.getNodeValues()
    edges = estructura.getEdgeValues()
    # Inicializacion de la estructura
    if tipo == 4:
        structure_ref = referenciaGrafo('Directed')
    else:
        structure_ref = referenciaGrafo()
    for i in nodos:                     
        structure_ref.addNode_byValue(i)
    for i,j,k in edges:
        structure_ref.addEdge_byValue(i,j,k)  
    # ---
    
    # Aplicar el recorrido
    result_ref = structure_ref.algorithms(recorrido, nodo)
    refText = ''
    testText = ''
    if tipo_inData == 'lista_nodos':
        if recorrido == "DepthFirstSearch" or recorrido == 'BreadhtFirstSearch':
            for i in result_ref[1]:
                refText = refText + str(i) + ', '
            for i in result_test:
                testText = testText + str(i) + ', ' 
            refText = refText[:-2]     
            testText = testText[:-2] 
        elif recorrido == 'DepthFirstOrder':
            depthFirstOrderRefText = False
            dfoList = ""
            for i in result_ref:
                if str(i) == 'Pre':
                    refText = refText + str(i) + ': '
                elif str(i) == 'Post':
                    refText = refText + "\n" + str(i) + ':'
                elif str(i) == "Reverse":
                    refText = refText + "\n" + str(i) + ":"
                    depthFirstOrderRefText = True
                elif str(i) != "Pre" and str(i) != "Post" and str(i) != "Reverse" and depthFirstOrderRefText == False:
                    refText = refText + str(i) + ', '
                elif str(i) != "Pre" and str(i) != "Post" and str(i) != "Reverse" and depthFirstOrderRefText == True:
                    dfoList = str(i) + "," + dfoList 
                else:
                    refText = refText + str(i) + ', '
            refText = refText + dfoList
            for i in result_test:
                testText = testText + str(i) + ', ' 
            refText = refText[:-2]     
            testText = testText[:-2]
        else:
            for i in result_ref:
                refText = refText + str(i) + ', '
            for i in result_test:
                testText = testText + str(i) + ', ' 
            refText = refText[:-2]     
            testText = testText[:-2]    
    
    if tipo_inData == 'tupla':
        testEdges = result_test[0]
        testWeight = result_test[1]
        
        refEdges = result_ref[0]
        refWeight = result_ref[1]
        
        for i in refEdges:
            refText = refText + '(' + i[0] + '->' + i[1] + '), '
        for i in testEdges:
            testText = testText + '(' + i[0] + '->' + i[1] + '), '
        
        refText = refText[:-2] + '\n  COSTO: ' + str(refWeight) + '\n'
        testText = testText[:-2] + '\n  COSTO: ' + str(testWeight)
        
        
        
    if tipo_inData == 'edges':
        for i in result_ref:
            refText = refText + '(' + i[0] + '->' + i[1] + '), '
        for i in result_test:
            testText = testText + '(' + i[0] + '->' + i[1] + '), '
        refText = refText[:-2]     
        testText = testText[:-2]    
    
    if tipo_inData == 'dicts':
        a = dict()
        for aux_dict in result_ref:
            a[nodo,aux_dict['node']] = aux_dict['cost']
            refText = refText + '\n* "'+nodo+'"' +'-->' +'"'+str(aux_dict['node'])+'"' + ' Costo: ' + str(aux_dict['cost']) + '\n  Path:'
            for i,j in aux_dict['path']:
                refText = refText + '(' + i + '->' + j + '), '
            if len(aux_dict['path'])>0:
                refText = refText[:-2]   
        b = dict()
        for aux_dict in result_test:
            b[nodo,aux_dict['node']] = aux_dict['cost']
            testText = testText + '\n* "'+nodo+'"' +'-->' +'"'+str(aux_dict['node'])+'"' + ' Costo: ' + str(aux_dict['cost']) + '\n  Path:'
            for i,j in aux_dict['path']:
                testText = testText + '(' + i + '->' + j + '), '
            if len(aux_dict['path'])>0:
                testText = testText[:-2] 
        for i,j  in b.keys():
            for x,y in a.keys():
                if i==x and y==j:
                    if a[x,y] != b[i,j]:
                        posError = 'Hay un error en la definicion del origen del nodo'
        

    if tipo_inData == 'single_dict':
        for lista in result_ref.values():
            refText = refText + '\n* '
            for i in lista:
                refText = refText + i + ', '
            refText = refText[:-2]

        for lista in result_test.values():
            testText = testText + '\n* '
            for i in lista:
                testText = testText + i + ', '
            testText = testText[:-2]
                
    if nodo != None:
        comment = 'El algoritmo ' + recorrido + ' se ha ejecutado desde el elemento "'+ nodo + '"\n'
    else:
        comment = 'El algoritmo ' + recorrido + ' se ha ejecutado:'
    if posError != " ":
        comment = posError
    else:
        comment = comment + '\n '+ posError +'\n Respuesta: \n' + refText
    #comment = comment + '\n\nSe obtuvo: ' + testText    
    return state_val, comment


### Validación - Arreglos

In [21]:
#Validar la creacion de un arreglo
def validar_arreglo_crear(nodos, st_nodos):
    '''
    Valida la operacion de crear un arreglo
       
    Args:
        nodos: nodos que se añadieron al crear el arrelgo
        st_nodos: nodos que la estructura tiene
               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario
        
    '''
    txt_nodos = ''
    for i in nodos:
        txt_nodos = txt_nodos + str(i) + ', '
    
    txt_st = ''
    for i in st_nodos:
        txt_st = txt_st + str(i) + ', '
        
    if len(nodos) == len(st_nodos):
        if nodos == st_nodos:
            if len(nodos) != 0:
                comment = 'Elementos: ' + txt_nodos[:-2]
            else:
                comment = 'Elementos: []'
            state_val = VALIDATION_STATES[1]
        else:
            nodos = sorted(nodos, key=functools.cmp_to_key(defaultfunction))
            st_nodos = sorted(st_nodos, key=functools.cmp_to_key(defaultfunction))
            
            if nodos == st_nodos:
                comment = 'Se añadieron los elementos pero en un orden diferente'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[0]
            else:
                comment = 'Se añadieron elementos diferentes a los indicados'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Se añadieron más elementos de los esperados'
        comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
        state_val = VALIDATION_STATES[-1]
        
    return state_val, comment

def validar_arreglo_anadir(init_test, end_test, nodo):
    '''
    Valida la operacion de añadir un elemento al arreglo.
    Se espera que el elemento se añada al final (contrato de los metodos de la estructura)
       
    Args:
        init_test: lista de elementos de la lista enlazada antes de ejecutar la operacion
        end_test: lista de elementos de la lista enlazada despues de ejecutar la operacion
        nodo: nodo que se añade a la lista enlazada
        tipo: tipo de lista enlazada (1: sencilla, 2: doble)
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''
    structure_ref = refereneciaArreglo()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    structure_ref.addNode_byValue(nodo)     #Ejecutar accion
    end_val = structure_ref.getNodeValues() #Resultado de validacion
    
    if len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break                       
        if sameOrder:
            comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente'
            state_val = VALIDATION_STATES[1]
        else:
            test = set(end_test)
            val = set(end_val)
            if test == val:
                comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente, pero no en la posición esperada'
                state_val = VALIDATION_STATES[0]
            else:
                comment = 'El elemento "'+ str(nodo) + '" NO se añadió, hay un elemento adicional pero no corresponde al elemento esperado'
                state_val = VALIDATION_STATES[-1]
    elif len(end_test) == len(end_val) - 1: # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se añadió el elemento "' + str(nodo) + '"'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'El elemento "'+ str(nodo) + '" pudo haberse añadido, pero reemplazó a un elemento de la lista'
            state_val = VALIDATION_STATES[-1]
    elif len(end_test) < len(init_test): # Se eliminaron elementos
        comment = 'Se eliminaron elementos de la lista, hay menos de los que habian antes de ejecutar la operación de añadir'
        state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    return state_val, end_val, comment

def validar_arreglo_anadir_first(init_test, end_test, nodo):
    '''
    Valida la operacion de añadir un elemento al arreglo.
    Se espera que el elemento se añada al final (contrato de los metodos de la estructura)
       
    Args:
        init_test: lista de elementos de la lista enlazada antes de ejecutar la operacion
        end_test: lista de elementos de la lista enlazada despues de ejecutar la operacion
        nodo: nodo que se añade a la lista enlazada
        tipo: tipo de lista enlazada (1: sencilla, 2: doble)
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''
    structure_ref = refereneciaArreglo()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    structure_ref.addNode_byValueFirst(nodo)     #Ejecutar accion
    end_val = structure_ref.getNodeValues() #Resultado de validacion
    
    if len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break                       
        if sameOrder:
            comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente'
            state_val = VALIDATION_STATES[1]
        else:
            test = set(end_test)
            val = set(end_val)
            if test == val:
                comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente, pero no en la posición esperada'
                state_val = VALIDATION_STATES[0]
            else:
                comment = 'El elemento "'+ str(nodo) + '" NO se añadió, hay un elemento adicional pero no corresponde al elemento esperado'
                state_val = VALIDATION_STATES[-1]
    elif len(end_test) == len(end_val) - 1: # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se añadió el elemento "' + str(nodo) + '"'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'El elemento "'+ str(nodo) + '" pudo haberse añadido, pero reemplazó a un elemento de la lista'
            state_val = VALIDATION_STATES[-1]
    elif len(end_test) < len(init_test): # Se eliminaron elementos
        comment = 'Se eliminaron elementos de la lista, hay menos de los que habian antes de ejecutar la operación de añadir'
        state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    return state_val, end_val, comment

def validar_arreglo_eliminar(init_test, end_test, nodo, ans):
    '''
    Valida la operacion de eliminar un elemento del arreglo. 
    Se espera que el elemento se elimine del arreglo.
       
    Args:
        init_test: lista de elementos del arreglo antes de ejecutar la operacion
        end_test: lista de elementos del arreglo despues de ejecutar la operacion
        nodo: nodo que se elimina del arreglo
        ans: resultado (True/False) de ejecutar el metodo deleteNode_byValue() en la estructura de prueba
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''
    structure_ref = refereneciaArreglo()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    ans2 = structure_ref.deleteNode_byValue(nodo)  #Ejecutar accion
    end_val = structure_ref.getNodeValues() #Resultado de validacion

    if len(end_test) != len(end_val):         # numero de elementos diferentes despues de la operacion
        detail = 'El numero de elementos resultado de la operacion es diferente. Se esperaba ' + str(len(end_val)) + ' y se obtuvo ' + str(len(end_test))
        comment = 'Problema en eliminacion de "'+ str(nodo) + '" \n' + detail
        state_val = VALIDATION_STATES[-1]
    elif ans == ans2 and ans:   # key aparecia en Arreglo y se elimino
        comment = 'El elemento "'+ str(nodo) + '" se eliminó satisfactoriamente'
        state_val = VALIDATION_STATES[1]
    elif ans == ans2:         # key No aparecia en Arreglo y No se elimino
        comment = 'El elemento "'+ str(nodo) + '" NO esta en el Arreglo y NO se eliminó'
        state_val = VALIDATION_STATES[0]
    else:
        detail = 'El resultado de return del método es diferente al esperado. Se esperaba ' + str(ans2) + ' y se obtuvo ' + str(ans)
        comment = 'Problema en eliminacion de "'+ str(nodo) + '" \n' + detail
        state_val = VALIDATION_STATES[-1]

    '''
    # SIMPLIFICACION de validacion
    if nodo not in init_test:
        comment = "El elemento " + str(nodo) + " no se encuentra en la estructura, por lo tanto no se puede eliminar"
        state_val = VALIDATION_STATES[0]

    elif len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break                       
        if sameOrder:
            comment = 'El elemento "'+ str(nodo) + '" se eliminó satisfactoriamente'
            state_val = VALIDATION_STATES[1]
        else:
            end_test = sorted(end_test, key=functools.cmp_to_key(defaultfunction))
            end_val = sorted(end_val, key=functools.cmp_to_key(defaultfunction))
            if(end_test == end_val):
                comment = 'Se eliminó una de las instancias del elemento "'+ str(nodo) + '"'
                state_val = VALIDATION_STATES[1]
            else:
                comment = 'El elemento "'+ str(nodo) + '" No se eliminó satisfactoriamente, se eliminó un elemento diferente'
                state_val = VALIDATION_STATES[-1]
    elif len(end_test) == len(init_test): # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != init_test[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se eliminó el elemento "' + str(nodo) + '". La lista no presenta cambios'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'No se eliminó el elemento "' + str(nodo) + '". La lista presenta cambios no esperados'
            state_val = VALIDATION_STATES[-1]     
    elif len(end_test) < len(end_val): # Se eliminaron mas elementos
        test = set(end_test)
        val = set(end_val)
        if nodo not in end_test:
            comment = 'Se eliminaron todas las instancias del elemento "' + str(nodo) +'" cuando solo una de ellas se debia eliminar'
            state_val = VALIDATION_STATES[0]
        elif test == val:
            comment = 'Se eliminaron más instancias del elemento "' + str(nodo) +'" cuando solo una de ellas se debia eliminar'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'Se eliminaron más elementos de los esperados'
            state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    
    if ans != ans2:
        ans2 = 'El resultado de return del método es diferente al esperado. Se esperaba ' + str(ans2) + ' y se obtuvo ' + str(ans)
        comment = comment + '\n' + ans2   
    '''

    return state_val, end_val, comment

def validar_arreglo_encontrar(nodos, nodo, ans):
    '''
    Valida la operacion de encontrar un elemento en el arreglo.
       
    Args:
        nodos: lista de elementos del arreglo antes de ejecutar la operacion
        nodo: nodo que se busca en el arreglo
        ans: resultado (True/False) de ejecutar el metodo isNodeValue() en la estructura de prueba
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        ans_val: resultado (True/False) si se encuentra el elemento en la estructura de referencia
        comment: mensaje informativo para el usuario
    '''
    if nodo in nodos:
        ans_val = True
    else:
        ans_val = False
    
    if ans == ans_val:
        if ans:
            comment = 'El elemento "'+ str(nodo) + '" SI se encuentra en el arreglo'
        else:
            comment = 'El elemento "'+ str(nodo) + '" NO se encuentra en el arreglo'
        state_val = VALIDATION_STATES[1]
    elif ans_val:
        comment = 'El elemento "'+ str(nodo) + '" SI se encuentra en el arreglo pero el método isNodeValue() reporta que NO se encuentra'
        state_val = VALIDATION_STATES[-1]
    else:
        comment = 'El elemento "'+ str(nodo) + '" NO se encuentra en el arreglo pero el método isNodeValue() reporta que SI se encuentra'
        state_val = VALIDATION_STATES[-1]

    return state_val, ans_val, comment

def validar_arreglo_adyacentes(init_test, listaAdj, nodo):
    '''
    Valida la operacion de encontrar los valores de los nodos adyacentes a un 
    elemento en la lista enlazada.
       
    Args:
        init_test: lista de elementos del arreglo antes de ejecutar la operacion
        listaAdj: lista de valores de nodos adyacentes obtenida por la estructura de prueba
        nodo: nodo al cual se le buscan los adyacentes
               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        listaAdj_val: lista de los elementos adyacentes obtenidos por la estructura de referencia
        comment: mensaje informativo para el usuario
        exists: indica si el nodo existe en la lista
        
    '''
    structure_ref = refereneciaArreglo()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    
    listaAdj_val = structure_ref.findAdjacentNode(nodo)
    exists = structure_ref.isNodeValue(nodo)
    
   # listaAdj_val = sorted(listaAdj_val, key=functools.cmp_to_key(defaultfunction))
    #listaAdj = sorted(listaAdj, key=functools.cmp_to_key(defaultfunction))
    """    CORRECCION Borrar
    txt = ''
    for i in listaAdj_val:
        txt = txt + str(i) + ', '
    """
    #if listaAdj == listaAdj_val:

    if len(listaAdj) != len(listaAdj_val) or listaAdj != listaAdj_val:
        comment = 'Diferencias en los adyacentes\n'
        comment += 'Se esperaban los adyacentes: ' + str(listaAdj_val) + '\n'
        comment += 'Se obtuvieron los adyacentes: ' + str(listaAdj)
        state_val = VALIDATION_STATES[-1]
    else:
        comment = 'El elemento "'+ str(nodo)+ '" tiene '+str(len(listaAdj_val))+' adyacente(s): ' + str(listaAdj)
        state_val = VALIDATION_STATES[1]

    # exists = structure_ref.isNodeValue(nodo)  CORRECCION
        
    return state_val, listaAdj_val, comment, exists


### Validación - Hash LP

In [22]:
def validar_linear_crear(nodos, end_test):
    '''
    Valida la operacion de crear una Hash LP
       
    Args:
        nodos: keys que se añadieron al crear la Hash LP
        end_test: keys que tiene la estructura 
                
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
               
    '''

    print("keys:", nodos)

    state_val = VALIDATION_STATES[1]
    end_val = list()
    comment = 'Sin validacion en la tabla LP'
    # return state_val, end_val, comment
        
    limitfactor = 0.7
    numelements = len(nodos)
    structure_ref = {}
    if numelements == 0:
        structure_ref = referenciaLinear()
    else:
        structure_ref = referenciaLinear(numelements)
        for key in nodos:
            structure_ref.addNode_byValue(int(key), int(key))

    end_val = structure_ref.getNodeValues()

    if len(end_test) != structure_ref.capacity():
        details1 = 'Tabla LP a probar con ' + str(len(end_test)) + ":" + str(end_test)
        details2 = 'Tabla LP de referencia con ' + str(len(end_val)) + ":" + str(end_val)
        details3 = 'Tabla LP de referencia con "'+ str(structure_ref.capacity()) + '" y Tabla LP a probar con "' + str(len(end_test)) + '"'
        comment = 'Problema en el tamaNo de la Tabla Hash LP a probar.' + '\n' + details1 + '\n' + details2 + '\n' + details3
        state_val = VALIDATION_STATES[-1]
        return state_val, end_val, comment

    print(end_test)
    print(end_val)

    if len(end_test) != len(end_val):         # numero de elementos diferentes despues de la operacion
        detail = 'El numero de llaves resultado de la operacion es diferente. Se esperaba ' + str(len(end_val)) + ' y se obtuvo ' + str(len(end_test))
        comment = 'Problema en crear TH LP ' + '\n' + detail
        state_val = VALIDATION_STATES[-1]

    else: # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if (end_test[i] == 'EMPTY' and end_val[i] != '__EMPTY__') or \
               (end_test[i] != 'EMPTY' and end_val[i] == '__EMPTY__'): # Caso Especial
                sameOrder = False
                break
            elif (end_test[i] != end_val[i]) and end_test[i] != 'EMPTY':
                sameOrder = False
                break
        if sameOrder:
            comment = 'Creación de TH LP satisfactoria'
            state_val = VALIDATION_STATES[1]
        else:
            comment = 'Creación de TH LP con llaves en orden diferente'
            state_val = VALIDATION_STATES[-1]

    return state_val, end_val, comment
    '''
    # SIMPLIFICACION DE LA VALIDACION
    txt_nodos = ''
    for i in sorted(nodos):
        txt_nodos = txt_nodos + str(i) + ', '
    
    txt_st = ''
    for i in sorted(st_nodos):
        txt_st = txt_st + str(i) + ', '
        
    if len(nodos) == len(st_nodos):
        if sorted(nodos) == sorted(st_nodos):
            if len(nodos) != 0:
                comment = 'Elementos: ' + txt_nodos[:-2]
            else:
                comment = 'Elementos: []'
            state_val = VALIDATION_STATES[1]
        else:
            nodos = sorted(nodos, key=functools.cmp_to_key(defaultfunction))
            st_nodos = sorted(st_nodos, key=functools.cmp_to_key(defaultfunction))
            
            if nodos == st_nodos:
                comment = 'Se añadieron los elementos pero en un orden diferente'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[0]
            else:
                comment = 'Se añadieron elementos diferentes a los indicados'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Hay diferencias en los elementos esperados'
        comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
        state_val = VALIDATION_STATES[-1]
    '''

def validar_linear_anadir(init_test, end_test, nodo):
    '''
    Valida la operacion de añadir un elemento a la Hash LP. 
       
    Args:
        init_test: lista de elementos de la Hash LP antes de ejecutar la operacion
        end_test: lista de elementos de la Hash LP despues de ejecutar la operacion
        nodo: nodo que se añade al arreglo
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''
    # IMPORTANTE: Se requiere garantizar que la Tabla Hash LP inicie con el mismo tamaNo y contenido que la estructura a validar
    limitfactor = 0.7
    numelements = int(len(init_test)*limitfactor)
    structure_ref = referenciaLinear(numelements)

    state_val = VALIDATION_STATES[1]
    end_val = list()
    comment = 'No se hace validacion.'
    # return state_val, end_val, comment
    
    if len(init_test) != structure_ref.capacity():
        details = 'Se esperaba Tabla LP de referencia con "'+ str(len(init_test)) + '" y se tiene "' + str(structure_ref.capacity()) + '"'
        comment = 'Problema en la inicializacion en el tamaNo de la Tabla Hash LP de referencia.' + '\n' + details
        state_val = VALIDATION_STATES[-1]
        for i in range(structure_ref.capacity()):
            if i < len(init_test):
                structure_ref.addNode_byPosition(init_test[i], init_test[i], i+1) # 1 <= posicion <= capacidad
        end_val = structure_ref.getNodeValues() #Resultado de validacio
        return state_val, end_val, comment

    '''
    # Terminacion sin hacer validacion
    comment += ' ' + str(len(init_test)) + ' igual a ' + str(structure_ref.capacity())
    return state_val, end_val, comment
    '''

    '''
    # SIMPLIFICACION Inicio del contenido de la Tabla Hash LP con el tamaNo y contenido de la estructura a validar
    end_initAux = list()
    for i in init_test:
        if str(i) != "None" and str(i) != "__EMPTY__":
            end_initAux.append(i)
    init_test = end_initAux

    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i,i)
    '''

    for i in range(len(init_test)):
        structure_ref.addNode_byPosition(init_test[i], init_test[i], i+1) # 1 <= posicion <= capacidad 


    structure_ref.addNode_byValue(nodo,nodo)     #Ejecutar accion
    end_val = structure_ref.getNodeValues() #Resultado de validacion

    if len(end_test) != structure_ref.capacity():
        details = 'Se tiene Tabla LP de referencia con "'+ str(structure_ref.capacity()) + '" y la respuesta tiene "' + str(len(end_test)) + '"'
        comment = 'Problema en el tamaNo de la Tabla Hash LP de respuesta.' + '\n' + details
        state_val = VALIDATION_STATES[-1]
        return state_val, end_val, comment

    print(end_test)
    print(end_val)
    
    if len(end_test) != len(end_val):         # numero de elementos diferentes despues de la operacion
        detail = 'El numero de elementos resultado de la operacion es diferente. Se esperaba ' + str(len(end_val)) + ' y se obtuvo ' + str(len(end_test))
        comment = 'Problema en agregar de "'+ str(nodo) + '" \n' + detail
        state_val = VALIDATION_STATES[-1]
    else:
        sameOrder = True
        for i in range(len(end_test)):
            if (end_test[i] == 'EMPTY' and end_val[i] != '__EMPTY__') or \
               (end_test[i] != 'EMPTY' and end_val[i] == '__EMPTY__'): # Caso Especial
                sameOrder = False
                break
            elif (end_test[i] != end_val[i]) and end_test[i] != 'EMPTY':
                sameOrder = False
                break                       
        if sameOrder:
            comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente'
            state_val = VALIDATION_STATES[1]
        else:
            comment = 'El elemento "'+ str(nodo) + '" NO se adiciono o se adiciono de forma incorrecta'
            state_val = VALIDATION_STATES[-1]

    return state_val, end_val, comment

    '''
    # SIMPLIFICACION de validacion
    end_testAux = list()
    for i in end_test:
        if str(i) != "None" and str(i) != "__EMPTY__":
            end_testAux.append(i)
    
    end_valaux = list()
    for i in end_val:
        if str(i) != "None" and str(i) != "__EMPTY__":
            end_valaux.append(i)

    #end_test = sorted(end_testAux)
    #end_val = sorted(end_valaux)

    end_test = end_testAux
    end_val = end_valaux

    print('end_val')
    print(end_val)


    print('end test')
    print(end_test)

    if len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        """
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break                       
        if sameOrder:
            comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente'
            state_val = VALIDATION_STATES[1]
        else:
            test = set(end_test)
            val = set(end_val)
            if test == val:
                comment = 'El elemento "'+ str(nodo) + '" se añadió pero NO en la posición esperada'
                state_val = VALIDATION_STATES[-1]
            else:
                comment = 'El elemento "'+ str(nodo) + '" NO se añadió, hay un elemento adicional pero no corresponde al elemento esperado'
                state_val = VALIDATION_STATES[-1]
        """
        comment = 'El elemento "'+ str(nodo) + '" se añadió (revisión del orden de elementos)'
        state_val = VALIDATION_STATES[1]
    elif len(end_test) == len(end_val) - 1: # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se añadió el elemento "' + str(nodo) + '"'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'El elemento "'+ str(nodo) + '" pudo haberse añadido, pero reemplazó a un elemento de la Hash LP'
            state_val = VALIDATION_STATES[-1]
    elif len(end_test) < len(init_test): # Se eliminaron elementos
        comment = 'Se eliminaron elementos de la Hash LP, hay menos de los que habian antes de ejecutar la operación de añadir'
        state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    '''



def validar_linear_eliminar(init_test, end_test, nodo, ans):
    '''
    Valida la operacion de eliminar un elemento de la Hash LP. 
    Se espera que el elemento se elimine de la Hash LP.
       
    Args:
        init_test: lista de elementos de la Hash LP antes de ejecutar la operacion
        end_test: lista de elementos de la Hash LP despues de ejecutar la operacion
        nodo: nodo que se elimina de la Hash LP
        ans: resultado (True/False) de ejecutar el metodo deleteNode_byValue() en la estructura de prueba
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''

    # IMPORTANTE: Se requiere garantizar que la Tabla Hash LP inicie con el mismo tamaNo y contenido que la estructura a validar
    limitfactor = 0.7
    numelements = int(len(init_test)*limitfactor)
    structure_ref = referenciaLinear(numelements)

    state_val = VALIDATION_STATES[1]
    end_val = list()
    comment = 'No se hace validacion.'
    # return state_val, end_val, comment
    
    if len(init_test) != structure_ref.capacity():
        details = 'Se esperaba Tabla LP de referencia con "'+ str(len(init_test)) + '" y se tiene "' + str(structure_ref.capacity()) + '"'
        comment = 'Problema en la inicializacion en el tamaNo de la Tabla Hash LP de referencia.' + '\n' + details
        state_val = VALIDATION_STATES[-1]
        for i in range(structure_ref.capacity()):
            if i < len(init_test):
                structure_ref.addNode_byPosition(init_test[i], init_test[i], i+1) # 1 <= posicion <= capacidad
        end_val = structure_ref.getNodeValues() #Resultado de validacio
        return state_val, end_val, comment

    '''
    # Terminacion sin hacer validacion
    comment += ' ' + str(len(init_test)) + ' igual a ' + str(structure_ref.capacity())
    return state_val, end_val, comment
    '''

    '''
    # SIMPLIFICACION Inicio del contenido de la Tabla Hash LP con el tamaNo y contenido de la estructura a validar
    auxInit = list()
    for i in init_test:
        if str(i) != "None" and str(i) != "__EMPTY__":
            auxInit.append(i)

    init_test = auxInit
       
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i,i)
    '''
    
    for i in range(len(init_test)):
        structure_ref.addNode_byPosition(init_test[i], init_test[i], i+1) # 1 <= posicion <= capacidad 

    ans2 = structure_ref.deleteNode_byValue(nodo)  #Ejecutar accion

    end_val = structure_ref.getNodeValues() #Resultado de validacio

    print(end_test)
    print(end_val)
    
    if len(end_test) != len(end_val):         # numero de elementos diferentes despues de la operacion
        detail = 'El numero de elementos resultado de la operacion es diferente. Se esperaba ' + str(len(end_val)) + ' y se obtuvo ' + str(len(end_test))
        comment = 'Problema en eliminacion de "'+ str(nodo) + '" \n' + detail
        state_val = VALIDATION_STATES[-1]
    elif ans == ans2 and ans:   # key aparecia en Tabla LP y se elimino
        comment = 'El elemento "'+ str(nodo) + '" se eliminó satisfactoriamente'
        state_val = VALIDATION_STATES[1]
    elif ans == ans2:         # key No aparecia en Tabla LP y No se elimino
        comment = 'El elemento "'+ str(nodo) + '" No esta en la Tabla LP y NO se eliminó'
        state_val = VALIDATION_STATES[0]
    else:
        detail = 'El resultado de return del método es diferente al esperado. Se esperaba ' + str(ans2) + ' y se obtuvo ' + str(ans)
        comment = 'Problema en eliminacion de "'+ str(nodo) + '" \n' + detail
        state_val = VALIDATION_STATES[-1]

    print(state_val, end_val)
    print(comment)
    return state_val, end_val, comment

    '''
    # SIMPLIFICACION de validacion
    if len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break                       
        if sameOrder:
            comment = 'El elemento "'+ str(nodo) + '" se eliminó satisfactoriamente'
            state_val = VALIDATION_STATES[1]
        else:
            comment = 'El elemento "'+ str(nodo) + '" NO se eliminó'
            state_val = VALIDATION_STATES[0]
        else:

            end_test2 = []
            for i in end_test:
                if str(i) != "None" and str(i) != "__EMPTY__":
                    end_test2.append(i)
            
            end_val2 = []
            for i in end_val:
                if str(i) != "None" and str(i) != "__EMPTY__":
                    end_val2.append(i)


            end_test = sorted(end_test2, key=functools.cmp_to_key(defaultfunction))
            end_val = sorted(end_val2, key=functools.cmp_to_key(defaultfunction))
            if(end_test == end_val):
                if ans2 == True:
                    comment = 'El elemento "'+ str(nodo) + '" se eliminó de la Hash LP'
                    state_val = VALIDATION_STATES[1]
                else:
                    comment = 'NO se eliminó el elemento "'+ str(nodo) + '"'
                    state_val = VALIDATION_STATES[-1]
            else:
                comment = 'El elemento "'+ str(nodo) + '" NO se eliminó satisfactoriamente'
                state_val = VALIDATION_STATES[-1]
    elif len(end_test) == len(init_test): # no hay cambios en la cantidad de elementos

        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != init_test[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'NO se eliminó el elemento "' + str(nodo) + '". La Hash LP NO presenta cambios'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'NO se eliminó el elemento "' + str(nodo) + '". La Hash LP presenta cambios no esperados'
            state_val = VALIDATION_STATES[-1]     
    elif len(end_test) < len(end_val): # Se eliminaron mas elementos

        test = set(end_test)
        val = set(end_val)
        if nodo not in end_test:
            comment = 'Se eliminaron todas las instancias del elemento "' + str(nodo) +'" cuando solo una de ellas se debia eliminar'
            state_val = VALIDATION_STATES[0]
        elif test == val:
            comment = 'Se eliminaron más instancias del elemento "' + str(nodo) +'" cuando solo una de ellas se debia eliminar'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'Se eliminaron más elementos de los esperados'
            state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados

        # CORRECCION Caso a revisar
        # cant = len(end_test) - len(end_val)
        # comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        comment = "OK eliminación (caso especial por revisar)"
        state_val = VALIDATION_STATES[1]
    
    if ans != ans2:
        ans2 = 'El resultado de return del método es diferente al esperado. Se esperaba ' + str(ans2) + ' y se obtuvo ' + str(ans)
        comment = comment + '\n' + ans2  
    '''

def validar_linear_encontrar(keys, key, ans):
    '''
    Valida la operacion de encontrar un elemento en la Tabla LP.
       
    Args:
        keys: lista de elementos de la Tabla LP antes de ejecutar la operacion
        key: llave que se busca en la Tabla LP
        ans: resultado (True/False) de ejecutar el metodo isNodeValue() en la estructura de prueba
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_ans: resultado (True/False) de ejecutar el metodo isNodeValue() en la estructura de referencia
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''

    # IMPORTANTE: Se requiere garantizar que la Tabla Hash LP inicie con el mismo tamaNo y contenido que la estructura a validar
    limitfactor = 0.7
    numelements = int(len(keys)*limitfactor)
    structure_ref = referenciaLinear(numelements)

    state_val = VALIDATION_STATES[1]
    end_val = list()
    end_ans = False
    comment = 'No se hace validacion.'
    # return state_val, end_ans, end_val, comment
    
    if len(keys) != structure_ref.capacity():
        details = 'Se esperaba Tabla LP de referencia con "'+ str(len(keys)) + '" y se tiene "' + str(structure_ref.capacity()) + '"'
        comment = 'Problema en la inicializacion en el tamaNo de la Tabla Hash LP de referencia.' + '\n' + details
        state_val = VALIDATION_STATES[-1]
        return state_val, end_ans, end_val, comment

    for i, keyi in enumerate(keys):
        structure_ref.addNode_byPosition(keyi, keyi, i+1) # 1 <= posicion <= capacidad 

    end_ans = structure_ref.isNodeValue(key) # Ejecutar Prueba
    end_val = structure_ref.getNodeValues()

    if ans == end_ans:
        if ans:
            comment = 'El elemento "'+ str(key) + '" SI se encuentra en la Tabla LP'
            state_val = VALIDATION_STATES[1]
        else:
            comment = 'El elemento "'+ str(key) + '" NO se encuentra en la Tabla LP'
            state_val = VALIDATION_STATES[0]
    elif end_ans:
        comment = 'El elemento "'+ str(key) + '" SI se encuentra en la Tabla LP pero el método isNodeValue() reporta que NO se encuentra'
        state_val = VALIDATION_STATES[-1]
    else:
        comment = 'El elemento "'+ str(key) + '" NO se encuentra en la Tabla LP pero el método isNodeValue() reporta que SI se encuentra'
        state_val = VALIDATION_STATES[-1]

    return state_val, end_ans, end_val, comment

### Validación - Separate Chaining

In [23]:
def validar_separate_crear(nodos, end_test):
    '''
    Valida la operacion de crear una tabla SEPARATE CHAINING
       
    Args:
        nodos: llaves que se añadieron al crear la tabla
        end_test: nodos que tiene la estructura 
                
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
        
    '''
    # IMPORTANTE: Se requiere garantizar que la Tabla Hash SC inicie con el mismo tamaNo y contenido que la estructura a validar

    print("keys:", nodos)

    state_val = VALIDATION_STATES[1]
    end_val = []
    comment = 'Sin validacion en la tabla SC'
    # return state_val, end_val, comment
        
    limitfactor = 2.0
    numelements = len(nodos)
    structure_ref = {}
    if numelements == 0:
        structure_ref = referenciaSeparate()
    else:
        structure_ref = referenciaSeparate(numelements)
        for key in nodos:
            structure_ref.addNode_byValue(int(key), int(key))

    end_val = structure_ref.getNodeValues()

    if len(end_test) != structure_ref.capacity():
        details1 = 'Tabla SC a probar con ' + str(len(end_test)) + ":" + str(end_test)
        details2 = 'Tabla SC de referencia con ' + str(len(end_val)) + ":" + str(end_val)
        details3 = 'Tabla SC de referencia con "'+ str(structure_ref.capacity()) + '" y Tabla SC a probar con "' + str(len(end_test)) + '"'
        comment = 'Problema en el tamaNo de la Tabla Hash SC a probar.' + '\n' + details1 + '\n' + details2 + '\n' + details3
        state_val = VALIDATION_STATES[-1]
        return state_val, end_val, comment

    print(end_test)
    print(end_val)

    if len(end_test) != len(end_val):         # numero de elementos diferentes despues de la operacion
        detail = 'El numero de buckets resultado de la operacion es diferente. Se esperaba ' + str(len(end_val)) + ' y se obtuvo ' + str(len(end_test))
        comment = 'Problema en crear TH SC ' + '\n' + detail
        state_val = VALIDATION_STATES[-1]

    else: # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'Creación de TH SC satisfactoria'
            state_val = VALIDATION_STATES[1]
        else:
            comment = 'Creación de TH SC con elementos de buckets en orden diferente'
            state_val = VALIDATION_STATES[-1]

    '''
    # SIMPLIFICACION DE LA VALIDACION
    txt_nodos = ''
    for i in nodos:
        txt_nodos = txt_nodos + str(i) + ', '
    
    txt_st = ''
    for i in st_nodos:
        txt_st = txt_st + str(i) + ', '
        
    if len(nodos) == len(st_nodos):
        nodos = sorted(nodos, key=functools.cmp_to_key(defaultfunction))
        st_nodos = sorted(st_nodos, key=functools.cmp_to_key(defaultfunction))
        if nodos == st_nodos:
            if len(nodos) != 0:
                comment = 'Elementos: ' + txt_nodos[:-2]
            else:
                comment = 'Elementos: []'
            state_val = VALIDATION_STATES[1]
        else:
            nodos = sorted(nodos, key=functools.cmp_to_key(defaultfunction))
            st_nodos = sorted(st_nodos, key=functools.cmp_to_key(defaultfunction))
            
            if nodos == st_nodos:
                comment = 'Se añadieron los elementos pero en un orden diferente'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[0]
            else:
                comment = 'Se añadieron elementos diferentes a los indicados'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Se añadieron más elementos de los esperados'
        comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
        state_val = VALIDATION_STATES[-1]
    '''

    return state_val, end_val, comment

def validar_separate_anadir(init_test, end_test, nodo):
    '''
    Valida la operacion de añadir un elemento a la Tabla SC. 
       
    Args:
        init_test: lista de elementos de la Tabla SC antes de ejecutar la operacion
        end_test: lista de elementos de la Tabla SC despues de ejecutar la operacion
        nodo: nodo que se añade a la Tabla SC
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''
    # IMPORTANTE: Se requiere garantizar que la Tabla Hash SC inicie con el mismo tamaNo y contenido que la estructura a validar

    print("init_test:", init_test)
    print("end_test", end_test)

    state_val = VALIDATION_STATES[1]
    end_val = []
    comment = 'Sin validacion en la tabla SC'
    # return state_val, end_val, comment
        
    limitfactor = 2.0
    numelements = int(len(init_test)*limitfactor - 1)
    structure_ref = referenciaSeparate(numelements)
    # print("creacion estructura referencia")

    '''
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i,i)
    '''

    if len(init_test) != structure_ref.capacity():
        details1 = 'init_test: con ' + str(len(init_test)) + ":" + str(init_test)
        details2 = 'structure_ref: con ' + str(structure_ref.capacity()) + ":" + str(structure_ref) 
        comment = 'Problema en la inicializacion en el tamaNo de la Tabla Hash SC de referencia.' + '\n' + details1 + '\n' + details2
        state_val = VALIDATION_STATES[-1]

        for i in range(structure_ref.capacity()):
            if i < len(init_test):
                if init_test[i] != "":
                    keys = init_test[i].split('\\n')
                    for key in keys:
                       structure_ref.addNode_byPosition(int(key), int(key), i+1) # 1 <= posicion <= capacidad
        end_val = structure_ref.getNodeValues() #Resultado de validacio
        return state_val, end_val, comment

    #print("len(init_test) == structure_ref.capacity() ")

    for i in range(len(init_test)):
        if init_test[i] != "":
            keys = init_test[i].split('\\n')
            for key in keys:
                if key != "":
                    structure_ref.addNode_byPosition(int(key), int(key), i+1) # 1 <= posicion <= capacidad 

    #print("Inicializada estructura de referencia")

    structure_ref.addNode_byValue(nodo,nodo)     #Ejecutar accion
    end_val = structure_ref.getNodeValues() #Resultado de validacion

    if len(end_test) != structure_ref.capacity():
        details1 = 'end_test: con ' + str(len(end_test)) + ":" + str(end_test)
        details2 = 'end_val: con ' + str(len(end_val)) + ":" + str(end_val)
        details3 = 'Se tiene Tabla SC de referencia con "'+ str(structure_ref.capacity()) + '" y la respuesta tiene "' + str(len(end_test)) + '"'
        comment = 'Problema en el tamaNo de la Tabla Hash SC de respuesta.' + '\n' + details1 + '\n' + details2 + '\n' + details3
        state_val = VALIDATION_STATES[-1]
        return state_val, end_val, comment

    print(end_test)
    print(end_val)

    if len(end_test) != len(end_val):         # numero de elementos diferentes despues de la operacion
        detail = 'El numero de buckets resultado de la operacion es diferente. Se esperaba ' + str(len(end_val)) + ' y se obtuvo ' + str(len(end_test))
        comment = 'Problema en agregar de "'+ str(nodo) + '" \n' + detail
        state_val = VALIDATION_STATES[-1]

    else: # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente'
            state_val = VALIDATION_STATES[1]
        else:
            comment = 'El elemento "'+ str(nodo) + '" NO se adiciono o se adiciono de forma incorrecta'
            state_val = VALIDATION_STATES[-1]

    '''
    # SIMPLIFICACION de validacion
    if len(end_test) != len(end_val): # Hay la cantidad esperada de elementos
        sameOrder = True

        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break
        if nodo in init_test:
            comment = 'El elemento ya se encuentra en la tabla SC'
            state_val = VALIDATION_STATES[1]
        elif sameOrder:
            comment = 'El elemento "'+ str(nodo) + '" se añadió satisfactoriamente'
            state_val = VALIDATION_STATES[1]
        else:
            end_test_sorted = sorted(end_test, key=functools.cmp_to_key(defaultfunction))
            end_val_sorted = sorted(end_val, key=functools.cmp_to_key(defaultfunction))
            if end_test_sorted == end_val_sorted:
                comment = 'El elemento "'+ str(nodo) + '" se añadió pero NO en la posición esperada'
                state_val = VALIDATION_STATES[0]
            else:
                test = set(end_test_sorted)
                val = set(end_val_sorted)
                if test == val:
                    comment = 'El elemento "'+ str(nodo) + '" se añadió pero NO en la posición esperada'
                    state_val = VALIDATION_STATES[0]
                else:
                    comment = 'El elemento "'+ str(nodo) + '" NO se añadió, hay un elemento adicional pero no corresponde al elemento esperado'
                    state_val = VALIDATION_STATES[-1]
                comment = 'Hay diferencia entre los elementos de la Tabla SC resultante y la Tabla SC esperada'
                state_val = VALIDATION_STATES[-1]
    elif len(end_test) == len(end_val) - 1: # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se añadió el elemento "' + str(nodo) + '"'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'El elemento "'+ str(nodo) + '" pudo haberse añadido, pero reemplazó a un elemento del arreglo'
            state_val = VALIDATION_STATES[-1]
    elif len(end_test) < len(init_test): # Se eliminaron elementos
        comment = 'Se eliminaron elementos de la Tabla SC, hay menos de los que habian antes de ejecutar la operación de añadir'
        state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    '''
    return state_val, end_val, comment


def validar_separate_eliminar(init_test, end_test, nodo, ans):
    '''
    Valida la operacion de eliminar un elemento del arreglo. 
    Se espera que el elemento se elimine del arreglo.
       
    Args:
        init_test: lista de elementos del arreglo antes de ejecutar la operacion
        end_test: lista de elementos del arreglo despues de ejecutar la operacion
        nodo: nodo que se elimina del arreglo
        ans: resultado (True/False) de ejecutar el metodo deleteNode_byValue() en la estructura de prueba
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''
    # IMPORTANTE: Se requiere garantizar que la Tabla Hash SC inicie con el mismo tamaNo y contenido que la estructura a validar
    state_val = VALIDATION_STATES[1]
    end_val = []
    comment = 'Sin validacion en la tabla SC'
    # return state_val, end_val, comment

    limitfactor = 2.0
    numelements = int(len(init_test)*limitfactor - 1)
    structure_ref = referenciaSeparate(numelements)

    if len(init_test) != structure_ref.capacity():
        details1 = 'init_test: con ' + str(len(init_test)) + ":" + str(init_test)
        details2 = 'structure_ref: con ' + str(structure_ref.capacity()) + ":" + str(structure_ref) 
        comment = 'Problema en la inicializacion en el tamaNo de la Tabla Hash SC de referencia.' + '\n' + details1 + '\n' + details2
        state_val = VALIDATION_STATES[-1]

        for i in range(structure_ref.capacity()):
            if i < len(init_test):
                if init_test[i] != "":
                    keys = init_test[i].split('\\n')
                    for key in keys:
                        if key != "":
                            structure_ref.addNode_byPosition(int(key), int(key), i+1) # 1 <= posicion <= capacidad
        end_val = structure_ref.getNodeValues() #Resultado de validacio
        return state_val, end_val, comment

    for i in range(len(init_test)):
        if init_test[i] != "":
            keys = init_test[i].split('\\n')
            for key in keys:
                if key != "":
                    structure_ref.addNode_byPosition(int(key), int(key), i+1) # 1 <= posicion <= capacidad 

    '''
    # Correccion de la construccion de la estructura de referencia
        
    structure_ref = referenciaSeparate()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i,i)
    '''

    eliminado = structure_ref.deleteNode_byValue(nodo)  #Ejecutar accion
    end_val = structure_ref.getNodeValues() #Resultado de validacion

    '''
    # SImplificacion en Validacion
    if len(end_test) == len(end_val): # Hay la cantidad esperada de elementos
        end_test = sorted(end_test, key=functools.cmp_to_key(defaultfunction))
        end_val = sorted(end_val, key=functools.cmp_to_key(defaultfunction))

        if end_test == end_val:
            
            if ans2 == False:
                comment = 'El elemento "'+ str(nodo) + '" no esta en la tabla'
                state_val = VALIDATION_STATES[1]
            else:
                comment = 'El elemento "'+ str(nodo) + '" se eliminó satisfactoriamente'
                state_val = VALIDATION_STATES[1]     
                              
        else:
            end_test = sorted(end_test, key=functools.cmp_to_key(defaultfunction))
            end_val = sorted(end_val, key=functools.cmp_to_key(defaultfunction))
            if(end_test == end_val):
                comment = 'Se eliminó una de las instancias del elemento "'+ str(nodo) + '"'
                state_val = VALIDATION_STATES[1]
            else:
                comment = 'El elemento "'+ str(nodo) + '" No se eliminó satisfactoriamente, se eliminó un elemento diferente'
                state_val = VALIDATION_STATES[-1]
    elif len(end_test) == len(init_test): # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != init_test[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se eliminó el elemento "' + str(nodo) + '". La lista no presenta cambios'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'No se eliminó el elemento "' + str(nodo) + '". La lista presenta cambios no esperados'
            state_val = VALIDATION_STATES[-1]     
    elif len(end_test) < len(end_val): # Se eliminaron mas elementos
        test = set(end_test)
        val = set(end_val)
        if nodo not in end_test:
            comment = 'Se eliminaron todas las instancias del elemento "' + str(nodo) +'" cuando solo una de ellas se debia eliminar'
            state_val = VALIDATION_STATES[0]
        elif test == val:
            comment = 'Se eliminaron más instancias del elemento "' + str(nodo) +'" cuando solo una de ellas se debia eliminar'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'Se eliminaron más elementos de los esperados'
            state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay '+ str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    '''

    if len(end_test) != structure_ref.capacity():
        details1 = 'end_test: con ' + str(len(end_test)) + ":" + str(end_test)
        details2 = 'end_val: con ' + str(len(end_val)) + ":" + str(end_val)
        details3 = 'Se tiene Tabla SC de referencia con "'+ str(structure_ref.capacity()) + '" y la respuesta tiene "' + str(len(end_test)) + '"'
        comment = 'Problema en el tamaNo de la Tabla Hash SC de respuesta.' + '\n' + details1 + '\n' + details2 + '\n' + details3
        state_val = VALIDATION_STATES[-1]
        return state_val, end_val, comment

    print(end_test)
    print(end_val)

    if len(end_test) != len(end_val):         # numero de elementos diferentes despues de la operacion
        detail = 'El numero de buckets resultado de la operacion es diferente. Se esperaba ' + str(len(end_val)) + ' y se obtuvo ' + str(len(end_test))
        comment = 'Problema en eliminar el "'+ str(nodo) + '" \n' + detail
        state_val = VALIDATION_STATES[-1]

    else: # Hay la cantidad esperada de elementos

        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break
        if sameOrder:
            if ans:
                comment = 'El elemento "'+ str(nodo) + '" se eliminó satisfactoriamente'
                state_val = VALIDATION_STATES[1]
            else:
                comment = 'El elemento "'+ str(nodo) + '" NO se eliminó o NO se encontraba en la tabla SC'
                state_val = VALIDATION_STATES[0]

        else:
            comment = 'El elemento "'+ str(nodo) + '" NO se eliminó o se eliminó de forma incorrecta'
            state_val = VALIDATION_STATES[-1]

    if ans != eliminado:
        details = 'El valor de retorno del método es diferente al esperado. Se esperaba ' + str(eliminado) + ' y se obtuvo ' + str(ans)
        comment = comment + '\n' + details   
        if state_val == VALIDATION_STATES[1]:
           state_val = VALIDATION_STATES[-1] 
    
    return state_val, end_val, comment

def validar_separate_encontrar(init_test, nodo, ans):
    '''
    Valida la operacion de encontrar un elemento en la Tabla SC.
       
    Args:
        nodos: lista de elementos de Tabla SC antes de ejecutar la operacion
        nodo: nodo que se busca en la Tabla SC
        ans: resultado (True/False) de ejecutar el metodo isNodeValue() en la estructura de prueba
        
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        ans_val: resultado (True/False) de ejecutar el metodo isNodeValue() en la estructura de referencia
        comment: mensaje informativo para el usuario
    '''

    '''    
    structure_ref = referenciaSeparate()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i,i)
    '''
    # IMPORTANTE: Se requiere garantizar que la Tabla Hash SC inicie con el mismo tamaNo y contenido que la estructura a validar
    state_val = VALIDATION_STATES[1]
    ans_val = False
    comment = 'Sin validacion en la tabla SC'
    # return state_val, ans_val, comment

    limitfactor = 2.0
    numelements = int(len(init_test)*limitfactor - 1)
    structure_ref = referenciaSeparate(numelements)

    if len(init_test) != structure_ref.capacity():
        details1 = 'init_test: con ' + str(len(init_test)) + ":" + str(init_test)
        details2 = 'structure_ref: con ' + str(structure_ref.capacity()) + ":" + str(structure_ref) 
        comment = 'Problema en la inicializacion en el tamaNo de la Tabla Hash SC de referencia.' + '\n' + details1 + '\n' + details2
        state_val = VALIDATION_STATES[-1]

        for i in range(structure_ref.capacity()):
            if i < len(init_test):
                if init_test[i] != "":
                    keys = init_test[i].split('\\n')
                    for key in keys:
                        if key != "":
                            structure_ref.addNode_byPosition(int(key), int(key), i+1) # 1 <= posicion <= capacidad
        end_val = structure_ref.getNodeValues() #Resultado de validacio
        return state_val, end_val, comment

    for i in range(len(init_test)):
        if init_test[i] != "":
            keys = init_test[i].split('\\n')
            for key in keys:
                if key != "":
                    structure_ref.addNode_byPosition(int(key), int(key), i+1) # 1 <= posicion <= capacidad 


    ans_val = structure_ref.isNodeValue(nodo)  #Ejecutar accion
    
    if ans == ans_val:
        if ans:
            comment = 'El elemento "'+ str(nodo) + '" SI se encuentra en la tabla SC'
        else:
            comment = 'El elemento "'+ str(nodo) + '" NO se encuentra en la tabla SC'
        state_val = VALIDATION_STATES[1]
    elif ans_val:
        comment = 'El elemento "'+ str(nodo) + '" SI se encuentra en la tabla SC pero el método isNodeValue() reporta que NO se encuentra'
        state_val = VALIDATION_STATES[-1]
    else:
        comment = 'El elemento "'+ str(nodo) + '" NO se encuentra en la tabla SC pero el método isNodeValue() reporta que SI se encuentra'
        state_val = VALIDATION_STATES[-1]
    
    return state_val, ans_val, comment

### Validacion - RBT

In [24]:
def validar_rbt_crear(nodos, st_nodos):
    '''
    Valida la operacion de crear un arreglo

    Args:
        nodos: nodos que se añadieron al crear el arreglo
        st_nodos: nodos que la estructura tiene

    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        comment: mensaje informativo para el usuario

    '''
    txt_nodos = ''
    for i in nodos:
        txt_nodos = txt_nodos + str(i) + ', '

    txt_st = ''
    for i in st_nodos:
        txt_st = txt_st + str(i) + ', '

    if len(nodos) == len(st_nodos):
        if nodos == st_nodos:
            if len(nodos) != 0:
                comment = 'Nodos: ' + txt_nodos[:-2]
            else:
                comment = 'Nodos: []'
            state_val = VALIDATION_STATES[1]
        else:
            nodos = sorted(nodos, key=functools.cmp_to_key(defaultfunction))
            st_nodos = sorted(
                st_nodos, key=functools.cmp_to_key(defaultfunction))

            if nodos == st_nodos:
                comment = 'Nodos: ' + txt_nodos[:-2]
                state_val = VALIDATION_STATES[1]
            else:
                comment = 'Se añadieron elementos diferentes a los indicados'
                comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
                comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
                state_val = VALIDATION_STATES[-1]
    else:
        comment = 'Se añadieron más elementos de los esperados'
        comment = comment + '\nSe esperaba: ' + txt_nodos[:-2]
        comment = comment + '\n  Se obtuvo: ' + txt_st[:-2]
        state_val = VALIDATION_STATES[-1]

    return state_val, comment


def validar_rbt_anadir(init_test, end_test, nodo):
    '''
    Valida la operacion de añadir un elemento al RBT. 

    Args:
        init_test: lista de llaves del RBT antes de ejecutar la operacion
        end_test: lista de llaves del RBT despues de ejecutar la operacion
        nodo: nodo que se añade al RBT

    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de llaves de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''
    # IMPORTANTE: Se requiere garantizar que el arbol RBT inicie con el mismo tamaNo, estructura y contenido que la estructura a validar
    structure_ref = referenciaRBT()

    state_val = VALIDATION_STATES[1]
    end_val = list()
    comment = 'No se hace validacion.'
    return state_val, end_val, comment

    for i in init_test:  # Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    structure_ref.addNode_byValue(nodo)  # Ejecutar accion
    end_val = structure_ref.getNodeValues()  # Resultado de validacion

    if len(end_test) == len(end_val):  # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'El elemento "' + \
                str(nodo) + '" se añadió satisfactoriamente'
            state_val = VALIDATION_STATES[1]
        else:
            test = set(end_test)
            val = set(end_val)
            if test == val:
                comment = 'El elemento "' + \
                    str(nodo) + '" se añadió satisfactoriamente, pero no en la posición esperada'
                state_val = VALIDATION_STATES[0]
            else:
                comment = 'El elemento "' + \
                    str(nodo) + '" NO se añadió, hay un elemento adicional pero no corresponde al elemento esperado'
                state_val = VALIDATION_STATES[-1]
    elif len(end_test) == len(end_val) - 1:  # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se añadió el elemento "' + str(nodo) + '"'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'El elemento "' + \
                str(nodo) + '" pudo haberse añadido, pero reemplazó a un elemento del arreglo'
            state_val = VALIDATION_STATES[-1]
    elif len(end_test) < len(init_test):  # Se eliminaron elementos
        comment = 'Se eliminaron elementos del arreglo, hay menos de los que habian antes de ejecutar la operación de añadir'
        state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay ' + str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]
    return state_val, end_val, comment


def validar_rbt_eliminar(init_test, end_test, nodo, ans):
    '''
    Valida la operacion de eliminar un elemento del arreglo. 
    Se espera que el elemento se elimine del arreglo.

    Args:
        init_test: lista de elementos del arreglo antes de ejecutar la operacion
        end_test: lista de elementos del arreglo despues de ejecutar la operacion
        nodo: nodo que se elimina del arreglo
        ans: resultado (True/False) de ejecutar el metodo deleteNode_byValue() en la estructura de prueba

    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        end_val: lista de los elementos de la estructura de referencia al ejecutar la operacion en cuestion
        comment: mensaje informativo para el usuario
    '''

    structure_ref = referenciaRBT()
    for i in init_test:  # Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    ans2 = structure_ref.deleteNode_byValue(nodo)  # Ejecutar accion
    end_val = structure_ref.getNodeValues()  # Resultado de validacion
    
    print(end_test)
    print(end_val)

    if len(end_test) == len(end_val):  # Hay la cantidad esperada de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != end_val[i]:
                sameOrder = False
                break
        if sameOrder:
            if ans2 == False:
                comment = 'El elemento "' + \
                    str(nodo) + '" no hace parte del arbol'
                state_val = VALIDATION_STATES[1]
            else:
                comment = 'El elemento "' + \
                    str(nodo) + '" se eliminó satisfactoriamente'
                state_val = VALIDATION_STATES[1]
        else:
            end_test = sorted(
                end_test, key=functools.cmp_to_key(defaultfunction))
            end_val = sorted(
                end_val, key=functools.cmp_to_key(defaultfunction))
            if(end_test == end_val):
                if ans2 == False:
                    comment = 'El elemento "' + \
                        str(nodo) + '" no hace parte del arbol'
                    state_val = VALIDATION_STATES[1]
                else:
                    comment = 'No se eliminó satisfactoriamente, se eliminó un elemento diferente "' + \
                        str(nodo) + '"'
                    state_val = VALIDATION_STATES[1]
            else:
                comment = 'El elemento "' + \
                    str(nodo) + '" No se eliminó satisfactoriamente, se eliminó un elemento diferente'
                state_val = VALIDATION_STATES[-1]
    elif len(end_test) == len(init_test):  # no hay cambios en la cantidad de elementos
        sameOrder = True
        for i in range(len(end_test)):
            if end_test[i] != init_test[i]:
                sameOrder = False
                break
        if sameOrder:
            comment = 'No se eliminó el elemento "' + \
                str(nodo) + '". La lista no presenta cambios'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'No se eliminó el elemento "' + \
                str(nodo) + '". La lista presenta cambios no esperados'
            state_val = VALIDATION_STATES[-1]
    elif len(end_test) < len(end_val):  # Se eliminaron mas elementos
        test = set(end_test)
        val = set(end_val)
        if nodo not in end_test:
            comment = 'No se eliminó satisfactoriamente, se eliminó un elemento diferente "'
            state_val = VALIDATION_STATES[0]
        elif test == val:
            comment = 'No se eliminó satisfactoriamente, se eliminó un elemento diferente "'
            state_val = VALIDATION_STATES[-1]
        else:
            comment = 'Se eliminaron más elementos de los esperados'
            state_val = VALIDATION_STATES[-1]
    else:   # Hay mas elementos de los esperados
        cant = len(end_test) - len(end_val)
        comment = 'Hay ' + str(cant) + ' elementos más de los esperados'
        state_val = VALIDATION_STATES[-1]

    if ans != ans2:
        ans2 = 'El resultado de return del método es diferente al esperado. Se esperaba ' + \
            str(ans2) + ' y se obtuvo ' + str(ans)
        comment = comment + '\n' + ans2

    return state_val, end_val, comment


def validar_rbt_encontrar(nodos, nodo, ans):
    '''
    Valida la operacion de encontrar un elemento en el RBT.

    Args:
        nodos: lista de elementos del RBT antes de ejecutar la operacion
        nodo: nodo que se busca en el RBT
        ans: resultado (True/False) de ejecutar el metodo isNodeValue() en la estructura de prueba

    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        ans_val: resultado (True/False) de ejecutar el metodo isNodeValue() en la estructura de referencia
        comment: mensaje informativo para el usuario
    '''
    if nodo in nodos:
        ans_val = True
    else:
        ans_val = False

    if ans == ans_val:
        if ans:
            comment = 'El elemento "' + \
                str(nodo) + '" SI se encuentra en el RBT'
        else:
            comment = 'El elemento "' + \
                str(nodo) + '" NO se encuentra en el RBT'
        state_val = VALIDATION_STATES[1]
    elif ans_val:
        comment = 'El elemento "' + \
            str(nodo) + '" SI se encuentra en el RBT pero el método isNodeValue() reporta que NO se encuentra'
        state_val = VALIDATION_STATES[-1]
    else:
        comment = 'El elemento "' + \
            str(nodo) + '" NO se encuentra en el RBT pero el método isNodeValue() reporta que SI se encuentra'
        state_val = VALIDATION_STATES[-1]

    return state_val, ans_val, comment

def validar_rbt_adyacentes(init_test, listaAdj, nodo):
    '''
    Valida la operacion de encontrar los valores de los nodos adyacentes a un 
    elemento en el grafo.
       
    Args:
        init_test: lista de elementos del grafo antes de ejecutar la operacion
        listaAdj: lista de valores de nodos adyacentes obtenida por la estructura de prueba
        nodo: nodo al cual se le buscan los adyacentes
               
    Returns:
        state_val: Estado final de la validacion (WARNING, SUCCESSFUL, FAILED)
        listaAdj_val: lista de los elementos adyacentes obtenidos por la estructura de referencia
        comment: mensaje informativo para el usuario
        exists: indica si el nodo existe en la lista
        
    '''
    structure_ref = referenciaRBT()
    for i in init_test:                     #Inicializacion de la estructura de prueba
        structure_ref.addNode_byValue(i)
    listaAdj_val = structure_ref.findAdjacentNode(nodo)   
    
    listaAdj_val = sorted(listaAdj_val, key=functools.cmp_to_key(defaultfunction))
    listaAdj = sorted(listaAdj, key=functools.cmp_to_key(defaultfunction))
    txt = ''
    for i in listaAdj_val:
        txt = txt + str(i) + ', '

    if listaAdj == listaAdj_val:
        comment = 'El elemento "'+ str(nodo)+ '" tiene '+str(len(listaAdj_val))+' adyacente(s): ' + txt[:-2]
        state_val = VALIDATION_STATES[1]
    else:
        comment = 'No se encontraron todos los adyacentes del elemento\nSe esperaban los elementos: ' + txt[:-2]
        txt = ''
        for i in listaAdj:
            txt = txt + str(i) + ', '
        comment = comment + '\nSe obtuvo: ' + txt[:-2]
        state_val = VALIDATION_STATES[-1]
    
    exists = structure_ref.isNodeValue(nodo)
    
    return state_val, listaAdj_val, comment, exists


## 4. Componente de Enlace

En estas secciones es donde se hacen los llamados a los metodos de las estructuras de pruebas (archivos extrenos). Se ejecutan las operaciones, se validan, se hace manejo de errores y se muestran los resultados en el Canvas de la aplicación

### Enlace - Listas Enlazadas

In [25]:
def crearListaEnlazada(tipo, file, init, data={}):
    """
    Crea una lista enlazada

    Args:
        tipo: Sencilla o Doble
        file: Estructura de datos externa
        init: Vacia o Random
        data: JSON con informacion de creacion (nodos)
    Returns:
        La lista enlazada
    Raises:
        Exception
    """
    nuevoTipoError = False
    if tipo == 1: txt = 'Sencilla - ' + init
    else: txt = 'Doble - ' + init
    
    txtNodos = ''
    try:
        estructura = file.listaEnlazada(tipo)
    except:
        e = "\tProblema al crear la lista enlazada, método listaEnlazada()"
        raise Exception(e)
    long = 0
    nodos = list()
    if init == 'Random':
        txtNodos = '\tValores: '
        long = random.randint(5,15)
        nodos = create_n_random(long)
        for i in nodos:
            txtNodos = txtNodos + str(i) + ', '
            try:
                estructura.addNode_byValue(i)
            except:
                e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                raise Exception(e)
        ''' CORRECCION Borrar esta validacion
        for i in range(0,len(nodos)):
            x = estructura.findAdjacentNode(nodos[i])
            if tipo == 2 and len(x) < 2 and i != 0 and i != len(nodos)-1:
                estructura = file.listaEnlazada(tipo)
                nuevoTipoError = True
        '''

    elif init == 'Estática' or init == 'Archivo':
        try:
            txtNodos = '\tValores: '
            nodos = data["valores"]
        except:
            raise Exception("El formato del archivo ingresado no es válido")
        for i in nodos:
            txtNodos = txtNodos + str(i) + ', '
            try:
                estructura.addNode_byValue(i)
            except:
                e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                raise Exception(e)
        ''' CORRECCION Borrar esta validacion
        for i in range(0,len(nodos)):
            x = estructura.findAdjacentNode(nodos[i])
            if tipo == 2 and len(x) < 2 and i != 0 and i != len(nodos)-1:
                estructura = file.listaEnlazada(tipo)
                nuevoTipoError = True
        '''
    displayList(estructura, tipo) 
    
    try:
        st_nodos = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los Elementos, método getNodeValues()'
        raise Exception(e)
    state_val, comment = validar_lista_crear(nodos, st_nodos, tipo)
    
    if nuevoTipoError == False:
        print('Crear Lista Enlazada', txt)
        print(state_val)
        print(comment)
        print('Total elementos: ' + str(len(estructura.getNodeValues())))
    else:
        print('ERROR: La estructura no tiene los dobles enlaces')
    return estructura

def anadirNodoLista(estructura, tipo, nodo):
    """
    Añade un nodo a lista

    Args:
        estructura: lista enlazada
        tipo: tipo de lista (1: sencilla, 2:doble)
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    init_test = list()                                         # CORRECCION Agregar
    end_test = list()                                         # CORRECCION Agregar
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener los nodos hacia adelante, método getNodeValues()'
        raise Exception(e)
    
    if tipo == 2:                                             # CORRECCION Agregar
        try:
            init_test_anteriores = estructura.getNodeValuesReverse()      # CORRECCION Agregar
        except:
            e = '\tProblema al obtener los nodos hacia atras, método getNodeValuesReverse()'
            raise Exception(e)
    
    try:
        estructura.addNode_byValue(nodo)              # METODO DE PRUEBA
        end_test = estructura.getNodeValues()
        if tipo == 2:                                                             # CORRECCION Agregar
            end_test_anteriores = estructura.getNodeValuesReverse()               # CORRECCION Agregar
            state_val, end_val, end_val_anteriores, comment = validar_enlaces_anteriores(init_test, end_test, end_test_anteriores, 'addNodeLast', nodo, tipo)  # CORRECCION Agregar
        if tipo == 1 or (tipo == 2 and state_val == VALIDATION_STATES[1]):        # CORRECCION Agregar
            state_val, end_val, comment = validar_lista_anadir(init_test, end_test, nodo, tipo)
#        if tipo == 2 and len(estructura.findAdjacentNode(nodo)) != 1:  # CORRECCION  Borrar
#            raise Exception('No creo su conexion hacia atras')         # CORRECCION Borrar
    except Exception as e:
#        if e != 'No creo su conexion hacia atras':                     # CORRECCION Borrar
#           e = 'exception original['+str(e)+']\n'+'\tProblema al añadir el elemento "'+str(nodo)+'", método addNode_byValue()'
#           raise Exception(e)
        e = 'Problema original ' + str(e) + '\n'                        # CORRECCION Agregar
        e += '\tProblema al añadir el elemento "'+str(nodo)+'", método addNode_byValue()'
        raise Exception(e)
    
    out.clear_output()
    displayList(estructura, tipo)
    # print('Tipo Lista: ' + str(tipo))
    print('Total elementos: ' + str(len(end_test)))
    print('Añadir elemento al Final')
    
    print(state_val, comment)
    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        print('Recorrido esperado hacia adelante:', end_val)
        print('Recorrido obtenido hacia adelante:  ', end_test)
        if tipo == 2:
            print('Recorrido esperado hacia atras:', end_val_anteriores)
            print('Recorrido obtenido hacia atras:  ', end_test_anteriores)

def anadirNodoListaFirst(estructura, tipo, nodo):
    """
    Añade un nodo a lista

    Args:
        estructura: lista enlazada
        tipo: tipo de lista (1: sencilla, 2:doble)
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """

    init_test = list()                                         # CORRECCION Agregar
    end_test = list()                                         # CORRECCION Agregar
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener los nodos hacia adelante, método getNodeValues()'
        raise Exception(e)
    if tipo == 2:                                             # CORRECCION Agregar
        try:
            init_test_anteriores = estructura.getNodeValuesReverse()      # CORRECCION Agregar
        except:
            e = '\tProblema al obtener los nodos hacia atras, método getNodeValuesReverse()'
            raise Exception(e)
    try:
        estructura.addNode_byValueFirst(nodo)                  # METODO DE PRUEBA
        end_test = estructura.getNodeValues()
        if tipo == 2:                                                             # CORRECCION Agregar
            end_test_anteriores = estructura.getNodeValuesReverse()               # CORRECCION Agregar
            state_val, end_val, end_val_anteriores, comment = validar_enlaces_anteriores(init_test, end_test, end_test_anteriores, 'addNodeFirst', nodo, tipo)  # CORRECCION Agregar
        if tipo == 1 or (tipo == 2 and state_val == VALIDATION_STATES[1]):              # CORRECCION Agregar
            state_val, end_val, comment = validar_lista_anadir_first(init_test, end_test, nodo, tipo) 
    except Exception as e:
        e = 'Problema original ' + str(e) + '\n'                        # CORRECCION Agregar
        e = '\tProblema al añadir el elemento "'+str(nodo)+'", método addNode_byValueFirst()'
        raise Exception(e)
    
    out.clear_output()
    displayList(estructura, tipo)
    print('Total elementos: ' + str(len(end_test)))           # CORRECCION modificar
    print('Añadir elemento al Principio')
    # print('Tipo Lista: ' + str(tipo))

    print(state_val, comment)
    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        print('Recorrido esperado hacia adelante:', end_val)
        print('Recorrido obtenido hacia adelante:', end_test)
        if tipo == 2:
            print('Recorrido esperado hacia atras:', end_val_anteriores)
            print('Recorrido obtenido hacia atras:', end_test_anteriores)

def eliminarNodoLista(estructura, tipo, nodo):
    """
    Elimina un nodo de la lista

    Args:
        estructura: lista enlazada
        tipo: tipo de la lista (1:sencilla, 2:doble)
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    if tipo == 2:                                             # CORRECCION Agregar
        try:
            init_test_anteriores = estructura.getNodeValuesReverse()      # CORRECCION Agregar
        except:
            e = '\tProblema al obtener los nodos hacia atras, método getNodeValuesReverse()'
            raise Exception(e)
    try:
        ans = estructura.deleteNode_byValue(nodo)             # Aplicacion METODO DE PRUEBA
        end_test = estructura.getNodeValues()
        if tipo == 2:                                                             # CORRECCION Agregar
            end_test_anteriores = estructura.getNodeValuesReverse()               # CORRECCION Agregar
            state_val, end_val, end_val_anteriores, comment = validar_enlaces_anteriores(init_test, end_test, end_test_anteriores, 'deleteNode', nodo, tipo)  # CORRECCION Agregar

        if tipo == 1 or (tipo == 2 and state_val == VALIDATION_STATES[1]):              # CORRECCION Agregar
            state_val, end_val, comment = validar_lista_eliminar(init_test, end_test, nodo, tipo, ans) 
    except Exception as e:
        e = 'Problema original ' + str(e) + '\n'                        # CORRECCION Agregar
        e = '\tProblema al eliminar el elemento "'+str(nodo)+'", método deleteNode_byValue()'
        raise Exception(e)                                              # CORRECCION Agregar
    out.clear_output()
    displayList(estructura, tipo)
    print('Total elementos: ' + str(len(end_test)))
    print('Eliminar elemento')
    
    print(state_val, comment)
    if state_val == VALIDATION_STATES[-1]: # Si fue fallido, mostrar los resultados esperados y obtenidos
        print('Recorrido esperado hacia adelante:', end_val)
        print('Recorrido obtenido hacia adelante:', end_test)
        if tipo == 2:
            print('Recorrido esperado hacia atras:', end_val_anteriores)
            print('Recorrido obtenido hacia atras:', end_test_anteriores)

def encontrarNodoLista(estructura, tipo, nodo):
    """
    Encuentra un nodo en la estrcutura

    Args:
        estructura: lista enlazada
        tipo: tipo de la lista (1:sencilla, 2:doble)
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        ans = estructura.isNodeValue(nodo)
        nodos = estructura.getNodeValues()
        state_val, ans_val, comment = validar_lista_encontrar(nodos, nodo, ans) 
    except:
        e = '\tProblema al buscar el elemento "' + str(nodo) + '", método isNodeValue()'
        raise Exception(e)
    out.clear_output()
    lista = list()
    lista.append(nodo)
    if ans_val:
        displayList(estructura, tipo, lista)
    else:    
        displayList(estructura, tipo)

    print("Buscar elemento")
    print(state_val, comment)

def findAdjacentNodeLista(estructura, tipo, nodo):
    """
    Encuentra los adyacentes de un nodo a lista

    Args:
        estructura: lista enlazada
        tipo: tipo de la lista (1:sencilla, 2:doble)
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        listaAdj = estructura.findAdjacentNode(nodo)
        state_val, listaAdj_val, comment, exists = validar_lista_adyacentes(init_test, listaAdj, nodo, tipo)
        # comment = 'El elemento "'+ str(nodo)+ '" tiene '+str(len(listaAdj))+' adyacente(s): ' + str(listaAdj)
        # state_val = VALIDATION_STATES[1]
        # exists = True
    except Exception as e:   # CORRECCION Modificar
        e = 'Problema original ' + str(e) + '\n'                        # CORRECCION Agregar
        e += 'TamaNo: '  + str(estructura.estructura['size']) + ' Elementos en lista: ' + str(init_test)
        e += '\tProblema al buscar los adyacentes del elemento "' + str(nodo) + '" , método findAdjacentNode()\n'
        raise Exception(e)
    
    out.clear_output()
    displayList(estructura, tipo, listaAdj)
    
    print('Encontrar Adyacentes')
    if not exists:
        print('\tEl elemento "'+str(nodo)+ '" NO existe en la lista')
    print(state_val, comment)

def darTodosLosNodos(estructura, tipo):
    """
    Retorna una lista con todos los nodos de la lista

    Args:
        estructura: lista enlazada
        tipo: tipo de la lista (1:sencilla, 2:doble)
    Returns:
        -
    Raises:
        Exception
    """
    try:
        nodos = estructura.getNodeValues()
    except:
        raise Exception('\tProblema al obtener todos los elemento, método getNodeValues()')
    out.clear_output()
    displayList(estructura, tipo)
    txt = ''
    for i in nodos:
        txt = txt + str(i) + ', '
    
    print('Encontrar Todos')
    print('\tTotal Elementos:', str(len(nodos)))
    print('\tElementos:', txt[:-2])


### Enlace - Árboles BST

In [26]:
def crearBST(init, file, data={}):
    """
    Crea un arbol BST

    Args:
        init: Vacia, Random, Estática, Archivo
        file: Estructura de datos externa
        data: JSON con la información de la inicializacion de la estructura

    Returns:
        La estructura de datos creada
    Raises:
        Exception
    """
    try:
        print('Crear BST')
        estructura = file.bst()
    except Exception as e:
        print('Error')
        #e = "\tProblema al crear el arbol BST, método bst()"
        print(e)
        raise Exception(e + "\tProblema al crear el arbol BST, método bst()")
    long = 0
    nodos = list()
    if init == 'Random':
        long = random.randint(5,10)
        nodos = create_n_random(long)
    elif init == 'Estática' or init == 'Archivo':
        try:
            nodos = data["valores"]
        except:
            raise Exception("El formato del archivo ingresado no es válido")
    for key in nodos:
        try:
            estructura.addNode_byValue(key)
        except:
            e = '\tProblema al añadir el elemento "'+str(key)+'", método addNode_byValue()'
            raise Exception(e)

    try:
        end_test = estructura.getNodeValues("Preorder")
    except:
        e = '\tProblema al obtener todos los Elementos, método getNodeValues()'
        raise Exception(e)

    state_val, end_val, comment = validar_bst_crear(nodos, end_test)

    out.clear_output()
    displayBST(estructura) 
    print("Total llaves: " + str(len(end_test)))
    print('Crear BST', init)
    print(state_val, comment)
    print('BST Preorden:', end_test)
    print('Size BST', estructura.size())
    print('Height BST', estructura.height())
    print('minKey BST', estructura.minKey())

    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        print('Se esperaba BST Preorden:', end_val)
        print('Se obtuvo BST Preorden  :', end_test)

    return estructura

def anadirNodoBST(estructura, nodo):
    """
    Añade un nodo al BST

    Args:
        estructura: lista enlazada
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues("Preorder")
    except:
        e = '\tProblema al obtener todos los Elementos, método getNodeValues()'
        raise Exception(e)
    try:
        estructura.addNode_byValue(nodo)
        end_test = estructura.getNodeValues("Preorder")
        state_val, end_val, comment = validar_bst_anadir(init_test, end_test, nodo) 
    except:
        e = '\tProblema al añadir el elemento "'+str(nodo)+'", método addNode_byValue()'
        raise Exception(e)
    
    out.clear_output()
    displayBST(estructura)
    print('Añadir llave ', nodo)
    print(state_val, comment)
    print("BST Preorden:", end_test)
    print('Size BST', estructura.size())
    print('Height BST', estructura.height())
    print('minKey BST', estructura.minKey())
    
    if state_val != VALIDATION_STATES[1]:
        print('Se esperaba BST Preorden:', end_val)
        print('Se obtuvo BST Preorden  :', end_test)

def eliminarNodoBST(estructura, nodo):
    """
    Elimina un nodo al BST

    Args:
        estructura: BST
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    
    try:
        init_test = estructura.getNodeValues("Preorder")
    except:
        e = '\tProblema al obtener las llaves en Preorden, función getNodeValues()'
        raise Exception(e)
    try:
        ans = estructura.deleteNode_byValue(nodo)  # funcion de prueba
    except:
        e = '\tProblema al eliminar el elemento "'+str(nodo)+'", función deleteNode_byValue()'
        raise Exception(e)

    end_test = estructura.getNodeValues("Preorder")
    state_val, end_val, comment = validar_bst_eliminar(init_test, end_test, nodo, ans) 

    out.clear_output()
    displayBST(estructura)
    print('Eliminar llave', nodo, 'resultado', ans)
    print(state_val, comment)
    print("BST Preorden:", end_test)
    print('Size BST', estructura.size())
    print('Height BST', estructura.height())
    print('minKey BST', estructura.minKey())
    
    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        print('Se esperaba BST Preorden:', end_val)
        print('Se obtuvo BST Preorden  :', end_test)

def encontrarNodoBST(estructura, nodo):
    """
    Encuentra un nodo en la estrcutura

    Args:
        estructura: BST
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        ans = estructura.isNodeValue(nodo)
        nodos = estructura.getNodeValues("Preorder")
        state_val, end_ans, end_val, comment = validar_bst_encontrar(nodos, nodo, ans) 
    except:
        e = '\tProblema al buscar la llave"' + str(nodo) + '", función isNodeValue()'
        raise Exception(e)

    out.clear_output()
    lista = list()
    lista.append(nodo)
    if ans:
        displayBST(estructura, lista)
    else:    
        displayBST(estructura)
        
    print('Buscar elemento', nodo, 'resultado', ans)
    print(state_val, comment)

    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        print('Se esperaba BST resultado', end_ans, 'Preorden:', end_val)
        print('Se obtuvo BST resultado  ', ans, 'Preorden:', nodos)
    
def findAdjacentNodoBST(estructura, nodo):
    """
    Encuentra los adyacentes de un nodo en el BST

    Args:
        estructura: arbol BST
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos las llaves, funcion getNodeValues()'
        raise Exception(e)
    try:
        existe, listaAdj = estructura.findAdjacentNode(nodo)
        print("intermedio", init_test, existe, listaAdj, nodo)
        state_val, existe_val, listaAdj_val, comment = validar_bst_adyacentes(init_test, existe, listaAdj, nodo) 
    except:
        e = '\tProblema al buscar los adyacentes del elemento "' + str(nodo) + '" , función findAdjacentNode()'
        raise Exception(e)
    
    out.clear_output()
    displayBST(estructura, listaAdj)
    print('Encontrar Adyacentes', nodo, 'resultado', existe, 'adyacentes', listaAdj)
    print(state_val, comment)
    
    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        print('Se esperaba resultado', existe_val, 'Adyacentes:', listaAdj_val)
        print('Se obtuvo resultado  ', existe, 'Adyacentes:', listaAdj)

def listarNodosBST(estructura, orden):
    """
    Lista todos los nodos del BST en el orden especificado

    Args:
        estructura: BST
        orden: orden en el cual se dan los nodos ('Preorder','Inorder', 'Postorder')
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues('Preorder')
    except:
        e = '\tProblema al obtener todas las llaves, función getNodeValues()'
        raise Exception(e)
    try:
        nodos = estructura.getNodeValues(orden)
        state_val, nodos_val, comment = validar_bst_darNodos(init_test, nodos, orden) 
    except:
        e = '\tProblema al listar todas las llaves, función getNodeValues()'
        raise Exception(e)
    
    out.clear_output()
    displayBST(estructura)
    print('Listar todas las llaves', orden)
    print(state_val, comment)
    print(nodos)
    print('Size BST', estructura.size())
    print('Height BST', estructura.height())
    print('minKey BST', estructura.minKey())
    
    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        print('Se esperaba:', nodos_val)
        print('Se obtuvo:  ', nodos)


### Enlace - Grafos

In [27]:
def crearGraph(init, tipo, file=None, data={},labels=False):
    """
    Crea un grafo

    Args:
        init: tipo de inicializacion de la estructura
        tipo: Dirigido(4) o no Dirigido(5)
        file: estructura de datos externa
        data: JSON con información de inicializacion
        labels: si se muestran o no los labels de los pesos
    Returns:
        La estructura creada
    Raises:
        Exception
    """
    
    if tipo == 4: txt = 'Dirigido - ' + init
    else: txt = 'No Dirigido - ' + init
    
    txtNodos = ''
    txtEdges = ''
    try:
        if tipo == 4:
            estructura = file.grafo(type='Directed')
        else:
            estructura = file.grafo(type='Undirected')
    except:
        e = "\tProblema al crear grafo, método graph()"
        raise Exception(e)
    long = 0
    nodos = list()
    edges = list()
    if init == 'Random':
        long = random.randint(10,15)
        nodos = create_n_random(long)
        edges = create_n_randomEdges(nodos, tipo)
        
        txtNodos = 'Elementos ('+str(len(nodos))+'): '
        if tipo == 5:
            txtEdges = 'Arcos ('+str(len(edges))+'):\n'
        else:
            txtEdges = 'Arcos ('+str(len(edges))+'):\n'
        for i in nodos:
            try:
                estructura.addNode_byValue(str(i))
                txtNodos = txtNodos + str(i) + ', '
            except:
                e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                raise Exception(e)
        for i,j,k in edges:
            try:
                estructura.addEdge_byValue(i,j,k)
                if tipo == 5:
                    arco = '(' + str(i) + ' <-> ' + str(j) + ', ' + str(round(k,2))+ ')'
                else:
                    arco = '(' + str(i) + ' -> ' + str(j) + ', ' + str(round(k,2))+ ')'
                txtEdges = txtEdges + '\t' + arco + '\n'
            except:
                e = '\tProblema al añadir el arco "('+str(i)+ '->'+ str(j)+','+str(k) + ')", método addEdge_byValue()'
                raise Exception(e)
        
    elif init == 'Estática' or init == 'Archivo':
        try:
            nodos = data["nodos"]
            edges = data["edges"]
            txtNodos = 'Elementos ('+str(len(nodos))+'): '
            if tipo == 5:
                txtEdges = 'Arcos ('+str(len(edges))+'):\n'
            else:
                txtEdges = 'Arcos ('+str(len(edges))+'):\n'
                
            for i in nodos:
                try:
                    estructura.addNode_byValue(str(i))
                    txtNodos = txtNodos + str(i) + ', '
                except:
                    e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                    raise Exception(e)
            for edge in edges:
                try:
                    estructura.addEdge_byValue(str(edge[0]),str(edge[1]),edge[2])
                    if tipo == 5:
                        arco = '(' + str(edge[0]) + ' <-> ' + str(edge[1]) + ', ' + str(edge[2])+ ')'
                    else:
                        arco = '(' + str(edge[0]) + ' -> ' + str(edge[1]) + ', ' + str(edge[2])+ ')'
                    txtEdges = txtEdges + '\t' + arco + '\n'
                except:
                    e = '\tProblema al añadir el arco "('+str(edge[0])+ '->'+ str(edge[1])+','+str(edge[2]) + ')", método addEdge_byValue()'
                    raise Exception(e)
        except:
            raise Exception("El formato del archivo ingresado no es válido")
          
    print('Num Nodes:', estructura.sizeNodes())
    print('Num Edges:', estructura.sizeEdges())

    try:
        st_nodos = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los vertices, método getNodeValues()'
        raise Exception(e)
    try:
        st_edges = estructura.getEdgeValues()
    except:
        e = '\tProblema al obtener todos los arcos, método getEdgeValues()'
        raise Exception(e)    
    
    state_val, comment = validar_graph_crear(nodos, edges, st_nodos, st_edges, tipo)
    
    displayGraph(estructura, tipo, label=labels)
     
    print('Crear Grafo', txt)
    #print(txtNodos[:-2])
    print(st_nodos)
    #print(txtEdges)
    print(st_edges)
    print(state_val, comment)
    return estructura

def anadirNodoGraph(estructura, tipo, label, nodo):
    """
    Añade un nodo al grafo

    Args:
        estructura: grafo
        tipo: Dirigido(4) o no Dirigido(5)
        label: si se muestran o no los labels de los pesos
        nodo: valor del nodo

    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los elementos, método getNodeValues()'
        raise Exception(e)
    try:
        estructura.addNode_byValue(nodo)   # Funcion a Probar
        end_test = estructura.getNodeValues()
        state_val, comment, end_ref = validar_graph_anadir(init_test, end_test, tipo, nodo) 
    except:
        e = '\tProblema al añadir el elemento "'+str(nodo)+'", método addNode_byValue()'
        raise Exception(e)
    
    out.clear_output()
    displayGraph(estructura, tipo, label, nodosX=[nodo])
    print('Añadir Vertice')
    print(state_val, comment)

    if state_val != VALIDATION_STATES[1]:
        print("Se esperaba ", len(end_ref), " vértices")
        print("Se tienen ", len(end_test), " vértices")
        print("Se esperaba ", end_ref)
        print("Se tienen ", end_test)

def eliminarNodoGraph(estructura, tipo, label, nodo):
    """
    Elimina un nodo del grafo

    Args:
        estructura: grafo
        tipo: Dirigido(4) o no Dirigido(5)
        label: si se muestran o no los labels de los pesos
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los elementos, método getNodeValues()'
        raise Exception(e)
    try:
        estructura.deleteNode_byValue(nodo)
        end_test = estructura.getNodeValues()
        state_val, comment = validar_graph_eliminar(init_test, end_test, tipo, nodo) 
    except:
        e = '\tProblema al eliminar el elemento "'+str(nodo)+'", método deleteNode_byValue()'
        raise Exception(e)
    
    out.clear_output()
    displayGraph(estructura, tipo, label)
    print(state_val, comment)
    
def existeNodoGraph(estructura, tipo, label, nodo):
    """
    Verifica la existencia de un nodo en el grafo

    Args:
        estructura: grafo
        tipo: Dirigido(4) o no Dirigido(5)
        label: si se muestran o no los labels de los pesos
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los Elementos, método getNodeValues()'
        raise Exception(e)
    try:
        existe_test = estructura.isNodeValue(nodo)
        state_val, comment = validar_graph_encontrar(init_test, existe_test, nodo) 
    except:
        e = '\tProblema al verificar si existe el Elemento "'+str(nodo)+'", método isNodeValue()'
        raise Exception(e)
    
    out.clear_output()
    displayGraph(estructura, tipo, label, nodosX=[nodo])
    print('Buscar elemento')
    print(state_val, comment)
    
def anadirArcoGraph(estructura, tipo, label, origen, destino, peso):
    """
    Añade un arco al grafo

    Args:
        estructura: grafo
        tipo: Dirigido(4) o no Dirigido(5)
        label: si se muestran o no los labels de los pesos
        origen: valor del nodo origen
        destino: valor del nodo destino
        peso: peso del arco entre origen y destino
    Returns:
        -
    Raises:
        Exception
    """
    try:
        nodes = estructura.getNodeValues()
    except Exception as e:
        detail = '\tProblema al obtener los vertices iniciales, método getNodeValues()'
        raise Exception(detail + '\n' + e)
    try:
        init_test = estructura.getEdgeValues()         
    except Exception as e:
        detail = '\tProblema al obtener los arcos iniciales, método getEdgeValues()'
        raise Exception(detail + '\n' + e)

    comment = 'La funcion addEdge_byValue(...) agrego arco.'
    state_val = VALIDATION_STATES[1]

    pre_condition = True
    if (origen not in nodes):
        comment = "NO existe vertice origen " + str(origen) + " en el grafo"
        state_val = VALIDATION_STATES[-1]
        pre_condition = False

    if pre_condition and (destino not in nodes):
        comment = "NO existe vertice destino " + str(destino) + " en el grafo"
        state_val = VALIDATION_STATES[-1]
        pre_condition = False

    if pre_condition:
        if tipo == 5:
            for eds in init_test:
                if ((eds[0] == origen and eds[1] == destino) or (eds[0] == destino and eds[1] == origen)):
                    comment = 'NO se pueden añadir arcos paralelos'
                    state_val = VALIDATION_STATES[-1]
                    pre_condition = False
        else:
            for eds in init_test:
                if ((eds[0] == origen and eds[1] == destino)):
                    comment = 'NO se pueden añadir arcos paralelos'
                    state_val = VALIDATION_STATES[-1]
                    pre_condition = False

    if pre_condition:
        try:
            estructura.addEdge_byValue(origen,destino,peso)   # funcion de prueba
        except:
            #arco = '(' + str(origen) + ' -> ' + str(destino) + ',' + str(peso) + ')'
            #e = '\tProblema al añadir el arco "'+arco+'", método addEdge_byValue()\n\tVerificar la existencia de los vertices'
            comment = 'NO se pudo agregar arco. Falla funcion addEdge_byValue(...)'
            state_val = VALIDATION_STATES[-1]
            pre_condition = False

    if pre_condition:
        try:
            end_test = estructura.getEdgeValues()         
        except Exception as e:
            detail = '\tProblema al obtener los arcos finales, método getEdgeValues()'
            raise Exception(detail + '\n' + e)

    if pre_condition and tipo == 4:
        encontro_orig_dest = False
        for edge in end_test:
            if (edge[0] == origen) and (edge[1] == destino):
                encontro_orig_dest = True
                break
            
        if not encontro_orig_dest:
            comment = 'NO se agregó el arco de ' + str(origen) + ' -> ' + str(destino)
            state_val = VALIDATION_STATES[-1]
            pre_condition = False
            
    if pre_condition and tipo == 5:
        encontro_orig_dest = False
        encontro_dest_orig = False
        pre_comment = 'NO Existe el arco ' + str(origen) + ' -> ' + str(destino) + '\n'
        pre_comment += 'NO Existe el arco ' + str(destino) + ' -> ' + str(origen) + '\n'

        for edge in end_test:
            if (not encontro_orig_dest) and (edge[0] == origen) and (edge[1] == destino):
                pre_comment = 'Existe el arco ' + str(origen) + ' -> ' + str(destino) + '\n'
                encontro_orig_dest = True
                break
            if (not encontro_dest_orig) and (edge[0] == destino) and (edge[1] == origen):
                pre_comment = 'Existe el arco ' + str(destino) + ' -> ' + str(origen) + '\n'
                encontro_dest_orig = True
                break

        if not encontro_orig_dest:
            encontro_orig_dest = estructura.isEdgeValue(origen, destino)
            if not encontro_orig_dest:
                comment = pre_comment + 'La funcion isEdgeValue(...) NO reporta el arco de ' + str(origen) + ' -> ' + str(destino) + ' en grafo No Dirigido'
                state_val = VALIDATION_STATES[-1]
                pre_condition = False

        if pre_condition and (not encontro_dest_orig):
            encontro_dest_orig = estructura.isEdgeValue(destino, origen)
            if not encontro_dest_orig:
                comment = pre_comment + 'La funcion isEdgeValue(...) NO reporta el arco de ' + str(destino) + ' -> ' + str(origen) + ' en grafo No Dirigido'
                state_val = VALIDATION_STATES[-1]
                pre_condition = False

    if pre_condition:
        state_val, comment, end_val = validar_graph_anadirEdge(nodes, init_test, end_test, tipo, origen, destino, peso) 

    out.clear_output()
    displayGraph(estructura, tipo, label, edgesX=[(origen,destino)])
    print('Añadir Arco')
    print(state_val, comment)

    if pre_condition and state_val != VALIDATION_STATES[1]:
        print('Numero arcos esperados: ', len(end_val), 'Numero arcos obtenidos: ', len(end_test))
        print('Se esperaba:', end_val)
        print('Se obtuvo:  ', end_test)
    
def adyacentesNodoGraph(estructura, tipo, label, nodo):
    """
    Encuentra los adyacentes de un nodo en el grafo

    Args:
        estructura: grafo
        tipo: Dirigido(4) o no Dirigido(5)
        label: si se muestran o no los labels de los pesos
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        nodes = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los elementos, método getNodeValues()'
        raise Exception(e)
    try:
        edges = estructura.getEdgeValues()
    except:
        e = '\tProblema al obtener todos los arcos, método getEdgeValues()'
        raise Exception(e)
    try:
        adjNodes = estructura.findAdjacentNode(nodo)
        state_val, comment = validar_graph_adj(nodes, edges, adjNodes, nodo, tipo) 
    except:
        e = '\tProblema al encontrar los adyacentes del elemento "'+str(nodo)+'", método findAdjacentNode()'
        raise Exception(e)
    
    paintEdges = list()
    for i in adjNodes:
        paintEdges.append((nodo,i))
    
    out.clear_output()
    displayGraph(estructura, tipo, label, nodosX=adjNodes, edgesX=paintEdges, nodeY = nodo)
    print('Adyacentes elemento')
    print(state_val, comment)    

def encontrarNodosGraph(estructura, tipo, label):
    """
    Encuentra todos los nodos del grafo

    Args:
        estructura: grafo
        tipo: Dirigido(4) o no Dirigido(5)
        label: si se muestran o no los labels de los pesos
    Returns:
        -
    Raises:
        Exception
    """
    try:
        nodes = estructura.getNodeValues()
        edges = estructura.getEdgeValues()
    except:
        e = '\tProblema al obtener todos los elemento, método getNodeValues() o método getEdgeValues()'
        raise Exception(e)

    state_val, comment = validar_graph_todos(nodes, edges, tipo)

    out.clear_output()
    displayGraph(estructura, tipo, label)
    print('Encontrar todos los elemento:', state_val)
    print(comment)

def recorridosGraph(estructura, tipo, label, recorrido, nodo=None):    
    """
    Ejecuta un algoritmo dado en el grafo

    Args:
        estructura: grafo
        tipo: Dirigido(4) o no Dirigido(5)
        label: si se muestran o no los labels de los pesos
        recorrido: nombre del algoritmo que se va a ejecutar
        nodo: nodo de inicio del algoritmo

    Returns:
        -
    Raises:
        Exception
    """
    if nodo is not None:
        try:
            out.clear_output()
            displayGraph(estructura, tipo, label)
            existe = estructura.isNodeValue(nodo)
        except:
            e = '\tProblema al verificar la existencia del elemento '+ nodo +', en el metodo getNodeValues()'
            raise Exception(e)
        if not existe:
            e = '\tEl elemento "'+ nodo +'" no pertenece al grafo.'
            raise Exception(e)
    rt, cmm = checkAlgoritmGraph(tipo, recorrido)
    if not rt:
        out.clear_output()
        displayGraph(estructura, tipo, label)
        raise Exception(cmm)
    try:
        rtaRecorrido = estructura.algorithms(recorrido, nodo)
    except:
        e = '\tProblema al ejecutar el recorrido ' + recorrido + ', metodo algorithms()'
        raise Exception(e)

    out.clear_output()
    state_val = ''
    comment = ''
    if recorrido == 'DepthFirstSearch' or recorrido == 'BreadhtFirstSearch' or recorrido == 'DepthFirstOrder':
        if recorrido == 'DepthFirstSearch':
            rut = rtaRecorrido[1]
            rut.remove(nodo)
            edges = rtaRecorrido[0]
            state_val, comment = validarRecorridosGrafo(estructura, tipo, 'lista_nodos', recorrido, rut, nodo)
   #         rtaRecorrido.remove(nodo)
            displayGraph(estructura, tipo, label, nodosX=rut, nodeY=nodo,  edgesX=edges)
        elif  recorrido == 'BreadhtFirstSearch':
            rut = rtaRecorrido[1]
            rut.remove(nodo)
            edges = rtaRecorrido[0]
            state_val, comment = validarRecorridosGrafo(estructura, tipo, 'lista_nodos', recorrido, rut, nodo)
            displayGraph(estructura, tipo, label, nodosX=rut, nodeY=nodo,edgesX=edges)
        else:
            state_val, comment = validarRecorridosGrafo(estructura, tipo, 'lista_nodos', recorrido, rtaRecorrido, nodo)
            displayGraph(estructura, tipo, label)
            
    elif recorrido == 'DirectedCycle':
        nodes = getNodesGivenEdges(rtaRecorrido)
        state_val, comment = validarRecorridosGrafo(estructura, tipo, 'edges', recorrido, rtaRecorrido)
        displayGraph(estructura, tipo, label, nodosX=nodes,edgesX=rtaRecorrido)  
        
    elif recorrido == 'Dijkstra' or recorrido == 'Bellman-Ford':
        state_val, comment = validarRecorridosGrafo(estructura, tipo, 'dicts', recorrido, rtaRecorrido, nodo)
        #print(rtaRecorrido[0])
        edgex = list()
        for i in rtaRecorrido:
            aux = i['path']
            for j in aux:
                if j not in edgex:
                    edgex.append(j)
        displayGraph(estructura, tipo, label, edgesX= edgex, nodeY=nodo)
        
    elif recorrido == 'KosarajuSCC':
        state_val, comment = validarRecorridosGrafo(estructura, tipo, 'single_dict', recorrido, rtaRecorrido)
        displayGraph(estructura, tipo, label)
        
    elif recorrido == 'PrimMST':
        state_val, comment = validarRecorridosGrafo(estructura, tipo, 'tupla', recorrido, rtaRecorrido, nodo)
        edges = rtaRecorrido[0]
        nodes = getNodesGivenEdges(edges)
        displayGraph(estructura, tipo, label, nodosX=nodes,edgesX=edges)
    print('Algoritmo ' + recorrido + ':' , state_val)
    print(comment)
    print(state_val)


### Enlace - Arreglo

In [28]:
def crearArreglo(file, init, data={}):
    """
    Crea un arreglo

    Args:
        file: Estructura de datos externa
        init: Vacia o Random
        data: JSON con informacion de creacion (nodos)
    Returns:
        Arreglo
    Raises:
        Exception
    """
    txtNodos = ''
    try:
        estructura = file.Arreglo()
    except:
        e = "\tProblema al crear arreglo, método Arreglo()"
        raise Exception(e)
    long = 0
    nodos = list()
    if init == 'Random':
        txtNodos = '\tValores: '
        long = random.randint(5,15)
        nodos = create_n_random(long)
        for i in nodos:
            txtNodos = txtNodos + str(i) + ', '
            try:
                estructura.addNode_byValue(i)
            except:
                e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                raise Exception(e)
    elif init == 'Estática' or init == 'Archivo':
        try:
            txtNodos = '\tValores: '
            nodos = data["valores"]
        except:
            raise Exception("El formato del archivo ingresado no es válido")
        for i in nodos:
            txtNodos = txtNodos + str(i) + ', '
            try:
                estructura.addNode_byValue(i)
            except:
                e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                raise Exception(e)
    displayArreglo(estructura)
    
    try:
        st_nodos = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los elementos, método getNodeValues()'
        raise Exception(e)
    state_val, comment = validar_arreglo_crear(nodos, st_nodos)
    
    print(state_val)
    print(comment)
    print("Total elementos: " + str(len(estructura.estructura)))
    return estructura

def anadirNodoArreglo(estructura, nodo):
    """
    Añade un elemento al final del arreglo

    Args:
        estructura: arreglo
        nodo: valor del elemento
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues().copy()
    except:
        e = '\tProblema al obtener todos los elementos, método getNodeValues()'
        raise Exception(e)
    try:
        estructura.addNode_byValue(nodo)
        end_test = estructura.getNodeValues()
        state_val, end_val, comment = validar_arreglo_anadir(init_test, end_test, nodo) 
    except:
        e = '\tProblema al añadir el elemento "'+str(nodo)+'", método addNode_byValue()'
        raise Exception(e)
    
    out.clear_output()
    displayArreglo(estructura)
    print('Total elementos: ' + str(len(estructura.getNodeValues())))
    print('Añadir elemento')
    print(state_val, comment)

    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        print('Se esperaba:', end_val)
        print('Se obtuvo:  ', end_test)

def anadirNodoArregloFirst(estructura, nodo):
    """
    Añade un elemento al principio del arreglo

    Args:
        estructura: arreglo
        nodo: valor del elemento
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los elementos, método getNodeValues()'
        raise Exception(e)
    try:
        estructura.addNode_byValueFirst(nodo)
        end_test = estructura.getNodeValues()
        state_val, end_val, comment = validar_arreglo_anadir_first(init_test, end_test, nodo) 
    except:
        e = '\tProblema al añadir el elemento "'+str(nodo)+'", método addNode_byValueFirst()'
        raise Exception(e)
    
    out.clear_output()
    displayArreglo(estructura)
    print('Total elementos: ' + str(len(estructura.getNodeValues())))
    print('Añadir elemento')
    
    print(state_val, comment)
    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        print('Se esperaba:', end_val)
        print('Se obtuvo:  ', end_test)

def eliminarNodoArreglo(estructura, nodo):
    """
    Elimina un elemento del arreglo

    Args:
        estructura: arreglo
        nodo: valor del elemento
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
        ini_test = []
        for i in range(0,len(init_test)):
            ini_test.append(init_test[i])
        init_test = ini_test
    except:
        e = '\tProblema al obtener todos los elementos, método getNodeValues()'
        raise Exception(e)
    try:
        ans = estructura.deleteNode_byValue(nodo)
        end_test = estructura.getNodeValues()
        state_val, end_val, comment = validar_arreglo_eliminar(init_test, end_test, nodo, ans) 
    except:
        e = '\tProblema al eliminar el elemento "'+str(nodo)+'", método deleteNode_byValue()'
    out.clear_output()
    displayArreglo(estructura)
    print('Total elementos: ' + str(len(estructura.getNodeValues())))
    print('Eliminar elemento')
    
    print(state_val, comment)
    if state_val == VALIDATION_STATES[-1]: # Si fue fallido, mostrar los resultados esperados y obtenidos
       print('Se esperaba:', end_val)
       print('Se obtuvo:  ', end_test)

def encontrarNodoArreglo(estructura, nodo):
    """
    Encuentra un elemento en el arreglo

    Args:
        estructura: arreglo
        nodo: valor del elemento
    Returns:
        -
    Raises:
        Exception
    """
    try:
        ans = estructura.isNodeValue(nodo)
        nodos = estructura.getNodeValues()
        state_val, ans_val, comment = validar_arreglo_encontrar(nodos, nodo, ans) 
    except:
        e = '\tProblema al buscar el elemento "' + str(nodo) + '", método isNodeValue()'
        raise Exception(e)
    
    out.clear_output()
    displayArreglo(estructura)
    print("Buscar elemento")
    print(state_val, comment)

def findAdjacentNodeArreglo(estructura, nodo):
    """
    Encuentra los adyacentes de un elemento del arreglo

    Args:
        estructura: arreglo
        nodo: valor del elemento
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los elementos, método getNodeValues()'
        raise Exception(e)
    try:
        listaAdj = estructura.findAdjacentNode(nodo)
        state_val, listaAdj_val, comment, exists = validar_arreglo_adyacentes(init_test, listaAdj, nodo) 
    except:
        e = '\tProblema al buscar los adyacentes del elemento "' + str(nodo) + '" , método findAdjacentNode()'
        raise Exception(e)
    
    out.clear_output()
    displayArreglo(estructura)

    print('Encontrar Adyacentes')
    if not exists:
        print('\tEl elemento "'+str(nodo)+ '" NO existe en el Arreglo')

    print(state_val, comment)


def darTodosLosNodosArreglo(estructura):
    """
    Retorna una lista con todos los elementos del arreglo

    Args:
        estructura: arreglo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        nodos = estructura.getNodeValues()
    except:
        raise Exception('\tProblema al obtener todos los elemento, método getNodeValues()')
    out.clear_output()
    displayArreglo(estructura)
    txt = ''
    for i in nodos:
        txt = txt + str(i) + ', '
    
    print('Encontrar Todos')
    print('\tTotal Elementos:', str(len(nodos)))
    print('\tElementos:', txt)


### Enlace - HashLP

In [29]:
def crearHashLP(file, init, data={}):
    """
    Crea una Hash LP

    Args:
        file: Estructura de datos externa
        init: Vacia o Random
        data: JSON con informacion de creacion (nodos)
    Returns:
        Tabla LP
    Raises:
        Exception
    """
    txtNodos = ''
    long = 0
    nodos = list()
    estructura = {}
    if init == 'Random':
        txtNodos = '\tValores: '
        long = random.randint(5,15)
        nodos = create_n_random(long)
        print(nodos)
    elif init == 'Estática' or init == 'Archivo':
        try:
            txtNodos = '\tValores: '
            nodos = data["valores"]
            long = len(nodos)
            print(nodos)
        except:
            raise Exception("El formato del archivo ingresado no es válido")

    try:
        if long != 0:
            estructura = file.LinearProbing(numelements=long)
        else:
            estructura = file.LinearProbing()
    except:
        e = "\tProblema al crear Hash SC, método LinearProbing()"
        raise Exception(e)

    for key in nodos:
        txtNodos = txtNodos + str(key) + ', '
        try:
            estructura.addNode_byValue(key,key)
        except:
            e = '\tProblema al añadir el elemento "'+str(key)+'", método addNode_byValue()'
            raise Exception(e)

    try:
        end_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los elementos, método getNodeValues()'
        raise Exception(e)
    
    state_val, end_val, comment = validar_linear_crear(nodos, end_test)

    out.clear_output()
    displayHashLP(estructura)
    print(end_test)
    print('Factor de carga actual: ', str(estructura.estructura['currentfactor']))
    print('Factor de carga limite: ', str(estructura.estructura['limitfactor']))
    print("Total Elementos: " + str(estructura.size()))
  
    print(state_val, comment)
    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        print('Se esperaba:', convertirEmptyValuesLP(end_val))
        print('Se obtuvo:  ', end_test)

    return estructura

def anadirNodoHashLP(estructura, nodo):
    """
    Añade un nodo a la Hash LP

    Args:
        estructura: Hash LP
        nodo: valor del nodo a aNadir
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los elementos, método getNodeValues()'
        raise Exception(e)
    try:
        estructura.addNode_byValue(nodo,nodo)
        end_test = estructura.getNodeValues()
        state_val, end_val, comment = validar_linear_anadir(init_test, end_test, nodo) 
    except:
        e = '\tProblema al añadir el elemento "'+str(nodo)+'", método addNode_byValue()'
        raise Exception(e)
    
    out.clear_output()
    displayHashLP(estructura)

    '''
    end_out = list()
    for i in end_test:
        if str(i) != "None" and str(i) != "__EMPTY__":
            end_out.append(i)
    '''
    
    print(state_val, comment)

    print('Factor de carga actual: ', str(estructura.estructura['currentfactor']))
    print('Factor de carga limite: ', str(estructura.estructura['limitfactor']))
    print("Total Elementos: " + str(estructura.size()))

    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        print('Se esperaba:', convertirEmptyValuesLP(end_val))
        print('Se obtuvo:  ', end_test)


def eliminarNodoHashLP(estructura, nodo):
    """
    Elimina un nodo de la Tabla LP

    Args:
        estructura: Tabla LP
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        #comment = "0"
        #state_val = "0"
        #end_val = "0"
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los elementos, método getNodeValues()'
        raise Exception(e)
    try:
        ans = estructura.deleteNode_byValue(nodo)
        end_test = estructura.getNodeValues()
        state_val, end_val, comment = validar_linear_eliminar(init_test, end_test, nodo, ans)
    except:
        e = '\tProblema al eliminar el elemento "'+str(nodo)+'", método deleteNode_byValue()'
    out.clear_output()
    displayHashLP(estructura)
    print('Eliminar elemento')

    print('Factor de carga actual: ', str(estructura.estructura['currentfactor']))
    print('Factor de carga limite: ', str(estructura.estructura['limitfactor']))
    print("Total Elementos: " + str(estructura.size()))
    
    print(state_val, comment)
    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
       print('Se esperaba:', convertirEmptyValuesLP(end_val))
       print('Se obtuvo:  ', end_test)

def encontrarNodoHashLP(estructura, key):
    """
    Encuentra un nodo en la Tabla LP

    Args:
        estructura: Tabla LP
        key: llave de busqueda
    Returns:
        -
    Raises:
        Exception
    """
    try:
        ans = estructura.isNodeValue(key) # Prueba de la funcion
        keys = estructura.getNodeValues()
        state_val, end_ans, end_val, comment = validar_linear_encontrar(keys, key, ans) 
    except:
        e = '\tProblema al buscar el elemento "' + str(key) + '", método isNodeValue()'
        raise Exception(e)
    
    out.clear_output()
  
    displayHashLP(estructura)
    print('Encontrar elemento')

    print('Factor de carga actual: ', str(estructura.estructura['currentfactor']))
    print('Factor de carga limite: ', str(estructura.estructura['limitfactor']))
    print("Total Elementos: " + str(estructura.size()))
    print(state_val, comment)
    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        print('Se esperaba:', end_ans, convertirEmptyValuesLP(end_val))
        print('Se obtuvo:  ', ans, keys)
    

def darTodosLosNodosHashLP(estructura):
    """
    Retorna una lista con las llaves de la Tabla Hash LP

    Args:
        estructura: Tabla LP
    Returns:
        -
    Raises:
        Exception
    """
    try:
        nodos = estructura.getNodeValues()
    except:
        raise Exception('\tProblema al obtener todos los elemento, método getNodeValues()')
    out.clear_output()
    displayHashLP(estructura)
    txt = ''
    for i in nodos:
        txt = txt + str(i) + ', '
    
    print('Encontrar Todos')
    print('\tTotal Elementos: ', str(len(nodos)))
    print('\tElementos:', txt[:-2])

def convertirEmptyValuesLP(keys_ref):
    """
    Retorna la lista de llaves con transformacion de las llaves '__EMPTY__' a 'EMPTY'

    Args:
        keys_ref: lista (Python) con las llaves de la estructura de referencia

    Returns:
        lista con llaves '__EMPTY__' transformadas
    """
    keys_t = list()
    for key in keys_ref:
        if key == '__EMPTY__':
            keys_t.append('EMPTY')
        else:
            keys_t.append(key)
    return keys_t


### Enlace - Separate Chaining

In [30]:
def crearHashSC(file, init, data={}):
    """
    Crea una lista enlazada

    Args:
        file: Estructura de datos externa implementada por estudiante
        init: Vacia o Random
        data: JSON con informacion de creacion (nodos)
    Returns:
        Tabla SC
    Raises:
        Exception
    """
    txtNodos = ''
    long = 0
    nodos = list()
    estructura = {}
    if init == 'Random':
        txtNodos = '\tValores: '
        long = random.randint(5,15)
        nodos = create_n_random(long)
        print(nodos)
    elif init == 'Estática' or init == 'Archivo':
        try:
            txtNodos = '\tValores: '
            nodos = data["valores"]
            long = len(nodos)
            print(nodos)
        except:
            raise Exception("El formato del archivo ingresado no es válido")

    try:
        if long != 0:
            estructura = file.SeparateChaining(numelements=long)
        else:
            estructura = file.SeparateChaining()
    except:
        e = "\tProblema al crear Hash SC, método SeparateChaining()"
        raise Exception(e)

    for key in nodos:
        txtNodos = txtNodos + str(key) + ', '
        try:
            estructura.addNode_byValue(key,key)
        except:
            e = '\tProblema al añadir el elemento "'+str(key)+'", método addNode_byValue()'
            raise Exception(e)

    try:
        end_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los elementos, método getNodeValues()'
        raise Exception(e)
    
    state_val, end_val, comment = validar_separate_crear(nodos, end_test)

    out.clear_output()
    displaySC(estructura)

    listaRespSC = list()
    for nodoSC in estructura.getNodeValues():
        listaRespSC.append(nodoSC.replace("\\n"," "))
    print(listaRespSC)

    print('Factor de carga actual: ', str(estructura.estructura['currentfactor']))
    print('Factor de carga limite: ', str(estructura.estructura['limitfactor']))
    print("Total Elementos: " + str(estructura.size()))
        
    print(state_val, comment)
    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        lista_end_val = list()
        for nodoSC in end_val:
            lista_end_val.append(nodoSC.replace("\\n"," "))
        print('Se esperaba:', lista_end_val)

        lista_end_test = list()
        for nodoSC in end_test:
            lista_end_test.append(nodoSC.replace("\\n"," "))
        print('Se obtuvo:  ', lista_end_test)

    return estructura

def anadirNodoHashSC(estructura, nodo):
    """
    Añade un nodo a lista

    Args:
        estructura: lista enlazada
        tipo: tipo de lista (1: sencilla, 2:doble)
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los elementos, método getNodeValues()'
        raise Exception(e)
    try:
        estructura.addNode_byValue(nodo,nodo)
        end_test = estructura.getNodeValues()
        state_val, end_val, comment = validar_separate_anadir(init_test, end_test, nodo) 
    except:
        e = '\tProblema al añadir el elemento "'+str(nodo)+'", método addNode_byValue()'
        raise Exception(e)
    
    out.clear_output()
    displaySC(estructura)
    listaRespSC = list()
    for nodoSC in estructura.getNodeValues():
        listaRespSC.append(nodoSC.replace("\\n"," "))
    print(listaRespSC)
    print("Añadio el elemento: ", str(nodo))
    print('Factor de carga actual: ', str(estructura.estructura['currentfactor']))
    print('Factor de carga limite: ', str(estructura.estructura['limitfactor']))
    print("Total Elementos: " + str(estructura.size()))
    
    print(state_val, comment)
    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        lista_end_val = list()
        for nodoSC in end_val:
            lista_end_val.append(nodoSC.replace("\\n"," "))
        print('Se esperaba:', lista_end_val)

        lista_end_test = list()
        for nodoSC in end_test:
            lista_end_test.append(nodoSC.replace("\\n"," "))
        print('Se obtuvo:  ', lista_end_test)


def eliminarNodoHashSC(estructura, nodo):
    """
    Elimina un nodo de la lista

    Args:
        estructura: lista enlazada
        tipo: tipo de la lista (1:sencilla, 2:doble)
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los nodos, método getNodeValues()'
        raise Exception(e)
    try:
        ans = estructura.deleteNode_byValue(nodo)
        end_test = estructura.getNodeValues()

        state_val, end_val, comment = validar_separate_eliminar(init_test, end_test, nodo, ans) 
    except:
        e = '\tProblema al eliminar el elemento "'+str(nodo)+'", método deleteNode_byValue()'

    out.clear_output()
    displaySC(estructura)
    print('Eliminar elemento')
    
    print(state_val, comment)
    print('Factor de carga actual: ', str(estructura.estructura['currentfactor']))
    print('Factor de carga limite: ', str(estructura.estructura['limitfactor']))
    print("Total Elementos: " + str(estructura.size()))

    if state_val != VALIDATION_STATES[1]: # Si no fue exitoso, mostrar los resultados esperados y obtenidos
        lista_end_val = list()
        for nodoSC in end_val:
            lista_end_val.append(nodoSC.replace("\\n"," "))
        print('Se esperaba:', lista_end_val)

        lista_end_test = list()
        for nodoSC in end_test:
            lista_end_test.append(nodoSC.replace("\\n"," "))
        print('Se obtuvo:  ', lista_end_test)

def encontrarNodoHashSC(estructura, nodo):
    """
    Encuentra un nodo en la estrcutura

    Args:
        estructura: Tabla SC
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        ans = estructura.isNodeValue(nodo)
        nodos = estructura.getNodeValues()
        state_val, ans_val, comment = validar_separate_encontrar(nodos, nodo, ans) 
    except:
        e = '\tProblema al buscar el elemento "' + str(nodo) + '", método isNodeValue()'
        raise Exception(e)
    
    out.clear_output()

    ''' CORRECCION Borrar
    if ans:
        print('El elemento: ', str(nodo), ' SI se encuentra en la Tabla SC')
    else:    
        print('El elemento: ', str(nodo), ' NO se encuentra en la Tabla SC')
    '''
    
    displaySC(estructura)
    print(state_val, comment)
    lista_nodos = list()
    for nodoSC in nodos:
        lista_nodos.append(nodoSC.replace("\\n"," "))
    print('Tabla SC:  ', lista_nodos)

    #print("Elementos: " + conc)
    


def darTodosLosNodosHashLP(estructura, tipo):
    """
    Retorna una lista con todos los nodos de la lista

    Args:
        estructura: lista enlazada
        tipo: tipo de la lista (1:sencilla, 2:doble)
    Returns:
        -
    Raises:
        Exception
    """
    try:
        listaRespSC = list()
        for nodoSC in estructura.getNodeValues():
            listaRespSC.append(nodoSC.replace("\\n"," "))

    except:
        raise Exception('\tProblema al obtener todos los elemento, método getNodeValues()')
    out.clear_output()
    displaySC(estructura)
    txt = ''
    for i in listaRespSC:
        txt = txt + str(i) + ', '
    
    print('Encontrar Todos')
    print('\tTotal Elementos:', str(len(listaRespSC)))
    print('\tElementos:', txt[:-2])

### Enlace - RBT

In [31]:
from DISClib.ADT import orderedmap as omap
def crearRBT(init, file, data={}):
    """
    Crea un arbol RBT

    Args:
        init: Vacia, Random, Estática, Archivo
        file: Estructura de datos externa
        data: JSON con la información de la inicializacion de la estructura

    Returns:
        La estructura de datos creada
    Raises:
        Exception
    """
    try:
        print('Crear RBT')
        estructura = file.RBT()
    except Exception as e:
        print('Error')
        #e = "\tProblema al crear el arbol BST, método bst()"
        print(e)
        raise Exception(e + "\tProblema al crear el arbol BST, método bst()")
    long = 0
    nodos = list()
    if init == 'Random':
        long = random.randint(5,10)
        nodos = create_n_random(long)
        for i in nodos:
            try:
                estructura.addNode_byValue(i)
            except:
                e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                raise Exception(e)
    elif init == 'Estática' or init == 'Archivo':
        try:
            nodos = data["valores"]
        except:
            raise Exception("El formato del archivo ingresado no es válido")
        for i in nodos:
            try:
                estructura.addNode_byValue(i)
            except:
                e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                raise Exception(e)
    displayRBT(estructura) 

    
    try:
        st_nodos = estructura.getNodeValues("Preorder")
    except:
        e = '\tProblema al obtener las llaves, método getNodeValues()'
        raise Exception(e)
    state_val, comment = validar_rbt_crear(nodos, st_nodos)
    
    print('Crear RBT ' + init + ':')
    print(state_val)    
    print(comment)
    print('Altura del arbol: ', str(omap.height(estructura.estructura)))
    return estructura

def anadirNodoRBT(estructura, nodo):
    """
    Añade un nodo al RBT

    Args:
        estructura: arbol RBT
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues("Preorder")
    except:
        e = '\tProblema al obtener las llaves, método getNodeValues()'
        raise Exception(e)
    try:
        estructura.addNode_byValue(nodo) # Ejecutar funcion de Prueba
        print('Get node values')
        end_test = estructura.getNodeValues("Preorder")
        print('Get node values post')
        print(end_test)
        state_val, end_val, comment = validar_rbt_anadir(init_test, end_test, nodo) 
    except:
        e = '\tProblema al añadir el elemento "'+str(nodo)+'", método addNode_byValue()'
        raise Exception(e)
    
    out.clear_output()
    displayRBT(estructura)
    print('Añadir elemento')
    
    print(state_val + ':', comment)

    print("Llaves: ", end_test)
    print('Altura del arbol: ', str(omap.height(estructura.estructura)))

    if state_val != VALIDATION_STATES[1]:  # Resultado No exitoso
        print('Se esperaba:', end_val)
        print('Se obtuvo:  ', end_test)

def eliminarNodoRBT(estructura, nodo):
    """
    Elimina un nodo al BST

    Args:
        estructura: BST
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues("Preorder")
    except:
        e = '\tProblema al obtener todos los elementos, método getNodeValues()'
        raise Exception(e)
    try:
        ans = estructura.deleteNode_byValue(nodo)
        end_test = estructura.getNodeValues("Preorder")
        state_val, end_val, comment = validar_rbt_eliminar(init_test, end_test, nodo, ans) 
    except:
        e = '\tProblema al eliminar el elemento "'+str(nodo)+'", método deleteNode_byValue()'
    out.clear_output()
    displayRBT(estructura)
    print('Eliminar elemento')
    
    print(state_val + ':', comment)

    nodos = estructura.getNodeValues()
    print("Vertices: ", nodos)
    print('Altura del arbol: ', str(omap.height(estructura.estructura)))

    if state_val == VALIDATION_STATES[-1]:
        print('Se esperaba:', end_val)
        print('Se obtuvo:  ', end_test)

def encontrarNodoRBT(estructura, nodo):
    """
    Encuentra un nodo en la estrcutura

    Args:
        estructura: BST
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        ans = estructura.isNodeValue(nodo)
        nodos = estructura.getNodeValues("Preorder")
        state_val, ans_val, comment = validar_rbt_encontrar(nodos, nodo, ans) 
    except:
        e = '\tProblema al buscar el elemento "' + str(nodo) + '", método isNodeValue()'
        raise Exception(e)
    out.clear_output()
    lista = list()
    lista.append(nodo)
    if ans_val:
        displayRBT(estructura, lista)
    else:    
        displayRBT(estructura)

    print('Buscar Elemento')
    print(state_val, comment)

    # nodos = estructura.getNodeValues()
    # print("Vertices: ", nodos)
    # print('Altura del arbol: ', str(omap.height(estructura.estructura)))
    
def findAdjacentNodoRBT(estructura, nodo):
    """
    Encuentra los adyacentes de un nodo en el bst

    Args:
        estructura: lista enlazada
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        init_test = estructura.getNodeValues()
    except:
        e = '\tProblema al obtener todos los elementos, método getNodeValues()'
        raise Exception(e)
    try:
        listaAdj = estructura.findAdjacentNode(nodo)
        state_val, listaAdj_val, comment, exists = validar_rbt_adyacentes(init_test, listaAdj, nodo) 
    except:
        e = '\tProblema al buscar los adyacentes del elemento "' + str(nodo) + '" , método findAdjacentNode()'
        raise Exception(e)
    
    out.clear_output()
    displayRBT(estructura, listaAdj)
    
    if not exists:
        print('Encontrar Adyacentes\n\tEl elemento "'+str(nodo)+ '" no existe en el bst')
    else:
        print('Encontrar Adyacentes')
        print(state_val, comment)
    
    nodos = estructura.getNodeValues()
    print("Nodos: ", nodos)
    print('Altura del arbol: ', str(omap.height(estructura.estructura)))

def listarNodosRBT(estructura, orden):
    """
    Lista todos los nodos del BST en el orden especificado

    Args:
        estructura: lista enlazada
        orden: orden en el cual se dan los nodos (preorden,inorden, postorden)
    Returns:
        -
    Raises:
        Exception
    """
    try:
        #init_test = estructura.getNodeValues()
        x = 1
    except:
        e = '\tProblema al obtener todos los elementos, método getNodeValues()'
        raise Exception(e)
    try:
        nodos = estructura.getNodeValues(orden)
        #state_val, nodos_val, comment = validar(init_test, nodos, orden) 
    except:
        e = '\tProblema al listar todos los elementos, método getNodeValues()'
        raise Exception(e)
    
    out.clear_output()
    displayRBT(estructura)
    print('Listar todos los vertices')
    print(nodos)
    print('Altura del arbol: ', str(omap.height(estructura.estructura)))
    #print(state_val, comment)


## 5. Interfaz Grafica

### Opciones

En esta seccion se construyen los paneles de las opciones de las operaciones que se soportan por cada estructura de datos. 

Es acá donde se hace el llamado a los metodos de Enlace segun corresponda

#### Opciones - Listas Enlazadas

In [32]:
import this

#listaType = widgets.RadioButtons(
 #   options = ['Sencilla', 'Doble'],
  #  disabled = False
#)

#grafoType = widgets.RadioButtons(
 #   options = ['Dirigido', 'No dirigido'],
  #  disabled = False
#)

grafoLabels = widgets.Checkbox(
    value=False,
    description='mostrar pesos',
    disabled=False,
    indent=False
)

listaInit = widgets.RadioButtons(
    options = ['Vacia', 'Random', 'Estática' ,'Archivo'],
    disabled = False
)

listaOrden = widgets.RadioButtons(
    options = ['Preorder', 'Inorder', 'Postorder'],
    disabled = False
)

listaInputNode = widgets.Text(
    value = '',
    placeholder = 'Valor del elemento',
    disabled = False,
    layout = {'width': '140px', 'justify_content':'center'}
)

btn_crear = Button(description='Crear', width='extended')
def on_button_crear_clicked(b):
    with out:
        out.clear_output()
        this.type = 1
        try:
            if listaInit.value == 'Estática':
                try:
                    name = 'creation files/lista_enlazada_1.json'
                    json_data = open(name)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo estatico " + name
                    raise Exception (e)               
                this.estructura = crearListaEnlazada(this.type, this.file, listaInit.value, data)
            elif listaInit.value == 'Archivo':
                try:
                    root = tk.Tk()
                    file_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Json File", "*.json")])
                    root.destroy()
                    json_data = open(file_path)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + file_path
                    raise Exception (e)
                this.estructura = crearListaEnlazada(this.type, this.file, listaInit.value, data)
            else:
                this.estructura = crearListaEnlazada(this.type, this.file, listaInit.value)
        except Exception as e:
            print('Hubo un problema al intentar crear la lista simplemente encadenada')
            print(e)
btn_crear.on_click(on_button_crear_clicked)

btn_addNode = Button(description='Añadir Último', width='extended')
def on_button_addNode_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([1,2], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    anadirNodoLista(this.estructura, this.type, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar añadir un elemento')
                print(e)
btn_addNode.on_click(on_button_addNode_clicked)

btn_addNode_first = Button(description='Añadir Primero', width='extended')
def on_button_addNode_first_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([1,2], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    anadirNodoListaFirst(this.estructura, this.type, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar añadir un elemento')
                print(e)
btn_addNode_first.on_click(on_button_addNode_first_clicked)

btn_deleteNode = Button(description='Eliminar', width='extended')
def on_button_deleteNode_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([1,2], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    eliminarNodoLista(this.estructura, this.type, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar eliminar un elemento')
                print(e)
btn_deleteNode.on_click(on_button_deleteNode_clicked)

btn_findNode = Button(description='Validar', width='extended')
def on_button_findNode_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([1,2], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:                
                    encontrarNodoLista(this.estructura, this.type, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar un elemento')
                print(e)
btn_findNode.on_click(on_button_findNode_clicked)

btn_findAdjNode = Button(description='Adyacentes', width='extended')
def on_button_findAdjNode_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([1,2], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    findAdjacentNodeLista(this.estructura, this.type, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar los adyacentes de un elemento')
                print(e)
btn_findAdjNode.on_click(on_button_findAdjNode_clicked)

btn_todosNodos = Button(description='Encontrar Todos', width='extended')
def on_button_todosNodos_clicked(b):
    with out:    
        try:
            try:
                this.estructura
                state, comment = validarEstructura([1,2], this.type)
                if not state:
                    print(comment)
            except:
                raise Exception("\tLa estructura no ha sido creada")
            if state:
                darTodosLosNodos(this.estructura, this.type)
        except Exception as e:
            print('Hubo un problema al intentar encontrar todos los elementos')
            print(e)
btn_todosNodos.on_click(on_button_todosNodos_clicked)

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


#### Opciones - Lista Enlazada Doble

In [33]:
btn_crearDoble = Button(description='Crear', width='extended')
def on_button_crearDoble_clicked(b):
    with out:
        out.clear_output()
        this.type = 2
        try:
            if listaInit.value == 'Estática':
                try:
                    name = 'creation files/lista_enlazada_1.json'
                    json_data = open(name)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo estatico " + name
                    raise Exception (e)               
                this.estructura = crearListaEnlazada(this.type, this.file, listaInit.value, data)
            elif listaInit.value == 'Archivo':
                try:
                    root = tk.Tk()
                    file_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Json File", "*.json")])
                    root.destroy()
                    json_data = open(file_path)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + file_path
                    raise Exception (e)
                this.estructura = crearListaEnlazada(this.type, this.file, listaInit.value, data)
            else:
                this.estructura = crearListaEnlazada(this.type, this.file, listaInit.value)
        except Exception as e:
            print('Hubo un problema al intentar crear la lista doblemente encadenada')
            print(e)
btn_crearDoble.on_click(on_button_crearDoble_clicked)

#### Opciones - Árboles BST 

In [34]:
btn_crearArbolOption = Button(description='Crear', width='extended')
def on_button_crearArbolOption_clicked(b):
    with out:
        out.clear_output()
        try:
            this.type = 3
            if listaInit.value == 'Estática':
                try:
                    name = 'creation files/bst_1.json' 
                    json_data = open(name)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + name
                    raise Exception (e)               
                this.estructura = crearBST(listaInit.value, this.file, data)
            elif listaInit.value == 'Archivo':
                try:
                    root = tk.Tk()
                    file_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Json File", "*.json")])
                    root.destroy()
                    json_data = open(file_path)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + file_path
                    raise Exception (e)
                this.estructura = crearBST(listaInit.value, this.file, data)
            else:
                this.estructura = crearBST(listaInit.value, this.file)
        except Exception as e:
            print('Hubo un problema al intentar crear el arbol')
            print(e)
btn_crearArbolOption.on_click(on_button_crearArbolOption_clicked)

btn_addNodeArbolOption = Button(description='Añadir', width='extended')
def on_button_addNodeArbolOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([3], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    anadirNodoBST(this.estructura, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar añadir un elemento')
                print(e)
btn_addNodeArbolOption.on_click(on_button_addNodeArbolOption_clicked)

btn_deleteNodeArbolOption = Button(description='Eliminar', width='extended')
def on_button_deleteNodeArbolOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([3], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    eliminarNodoBST(this.estructura, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar eliminar un elemento')
                print(e)
btn_deleteNodeArbolOption.on_click(on_button_deleteNodeArbolOption_clicked)

btn_findNodeArbolOption = Button(description='Validar', width='extended')
def on_button_findNodeArbolOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([3], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    encontrarNodoBST(this.estructura, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar un elemento')
                print(e)
btn_findNodeArbolOption.on_click(on_button_findNodeArbolOption_clicked)

btn_findAdjNodeArbolOption = Button(description='Adyacentes', width='extended')
def on_button_findAdjNodeArbolOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([3], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    findAdjacentNodoBST(this.estructura, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar los adyacentes de un elemento')
                print(e)
btn_findAdjNodeArbolOption.on_click(on_button_findAdjNodeArbolOption_clicked)

btn_todosNodosArbol = Button(description='Listar nodos', width='extended')
def on_button_todosNodosArbol_clicked(b):
    with out:
        out.clear_output()
        try:
            try:
                this.estructura
                state, comment = validarEstructura([3], this.type)
                if not state:
                    print(comment)
            except:
                raise Exception("\tLa estructura no ha sido creada")
            if state:
                listarNodosBST(this.estructura, listaOrden.value)
        except Exception as e:
            print('Hubo un problema al intentar encontrar los adyacentes de un elemento')
            print(e)
btn_todosNodosArbol.on_click(on_button_todosNodosArbol_clicked)

#### Opciones - Grafos

In [35]:
grafoInputNodeInit = widgets.Text(
    value = '',
    placeholder = 'Vertice Origen',
    disabled = False,
    layout = {'width': '140px', 'justify_content':'center'}
)
grafoInputNodeEnd = widgets.Text(
    value = '',
    placeholder = 'Vertice Destino',
    disabled = False,
    layout = {'width': '140px', 'justify_content':'center'}
)
grafoInputEdgeWeight = widgets.FloatText(
    step=0.5,
    value = 0.00,
    placeholder = 'Peso Arco',
    disabled = False,
    layout = {'width': '140px', 'justify_content':'center'}
)
grafoInputNode = widgets.Text(
    value = '',
    placeholder = 'Vertice de inicio *',
    disabled = False,
    layout = {'width': '140px', 'justify_content':'center'}
)
grafoRecorrido = widgets.RadioButtons(
    options = ['DepthFirstSearch*', 'BreadhtFirstSearch*',  
               'PrimMST*', 'Dijkstra*', 'DepthFirstOrder','KosarajuSCC' ],
    disabled = False
)
grafoRecorridoD = widgets.RadioButtons(
    options = ['DepthFirstSearch*', 'BreadhtFirstSearch*',  
               'Dijkstra*', 'DepthFirstOrder','KosarajuSCC' ],
    disabled = False
)
btn_crearGrafoOption = Button(description='Crear', width='extended')
def on_button_crearGrafoOption_clicked(b):
    with out:
        out.clear_output()
        this.type = 4
        try:
            if listaInit.value == 'Estática':
                try:
                    name = 'creation files/graph_1.json' 
                    json_data = open(name)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo estatico " + name
                    raise Exception (e)               
                this.estructura = crearGraph(init = listaInit.value, 
                                             tipo = this.type, 
                                             data = data, 
                                             file = this.file,
                                             labels=grafoLabels.value)
            elif listaInit.value == 'Archivo':
                try:
                    root = tk.Tk()
                    file_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Json File", "*.json")])
                    root.destroy()
                    json_data = open(file_path)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + file_path
                    raise Exception (e)
                this.estructura = crearGraph(init = listaInit.value, 
                                tipo = this.type, 
                                data = data, 
                                file = this.file,
                                labels=grafoLabels.value)
            else:
                this.estructura = crearGraph(init = listaInit.value, 
                                tipo = this.type,
                                file = this.file, 
                                labels=grafoLabels.value)
        except Exception as e:
            print('Hubo un problema al intentar crear el grafo')
            print(e)
btn_crearGrafoOption.on_click(on_button_crearGrafoOption_clicked)

btn_addNodeGrafoOption = Button(description='Añadir', width='extended')
def on_button_addNodeGrafoOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    this.estructura
                    state, comment = validarEstructura([4,5], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    anadirNodoGraph(this.estructura, this.type, grafoLabels.value, value)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar añadir un Vertice')
                print(e)
btn_addNodeGrafoOption.on_click(on_button_addNodeGrafoOption_clicked)

btn_deleteNodeGrafoOption = Button(description='Eliminar', width='extended')
def on_button_deleteNodeGrafoOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    this.estructura
                    state, comment = validarEstructura([4,5], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    eliminarNodoGraph(this.estructura, this.type, grafoLabels.value, value)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar eliminar un Vertice')
                print(e)
btn_deleteNodeGrafoOption.on_click(on_button_deleteNodeGrafoOption_clicked)

btn_findNodeGrafoOption = Button(description='Validar', width='extended')
def on_button_findNodeGrafoOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    this.estructura
                    state, comment = validarEstructura([4,5], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    existeNodoGraph(this.estructura, this.type, grafoLabels.value, value)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar validar si un Vertice existe')
                print(e)
btn_findNodeGrafoOption.on_click(on_button_findNodeGrafoOption_clicked)

btn_addEdgeOption = Button(description='Añadir', width='extended')
def on_button_addEdgeOption_clicked(b):
    with out:
        origen = grafoInputNodeInit.value.strip() 
        destino = grafoInputNodeEnd.value.strip()
        peso = grafoInputEdgeWeight.value
        if len(origen) > 0 and len(destino) > 0:
            try:
                try:
                    this.estructura
                    state, comment = validarEstructura([4,5], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    anadirArcoGraph(this.estructura, this.type, grafoLabels.value, origen, destino, peso)
                grafoInputNodeInit.value = ''
                grafoInputNodeEnd.value = ''
                grafoInputEdgeWeight.value = 0.00
            except Exception as e:
                print('Hubo un problema al intentar añadir un arco')
                print(e)
btn_addEdgeOption.on_click(on_button_addEdgeOption_clicked)

btn_findAdjNodeGrafoOption = Button(description='Adyacentes', width='extended')
def on_button_findAdjNodeGrafoOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    this.estructura
                    state, comment = validarEstructura([4,5], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    adyacentesNodoGraph(this.estructura, this.type, grafoLabels.value, value)
                listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar los adyacentes de un vertice')
                print(e)
btn_findAdjNodeGrafoOption.on_click(on_button_findAdjNodeGrafoOption_clicked)

btn_todosNodosGrafo = Button(description='Encontrar Todos', width='extended')
def on_button_todosNodosGrafo_clicked(b):
    with out:
        try:
            try:
                this.estructura
                state, comment = validarEstructura([4,5], this.type)
                if not state:
                    print(comment)
            except:
                raise Exception("\tLa estructura no ha sido creada")
            if state:
                encontrarNodosGraph(this.estructura, this.type, grafoLabels.value)
        except Exception as e:
            print('Hubo un problema al intentar encontrar todos los vertices')
            print(e)
btn_todosNodosGrafo.on_click(on_button_todosNodosGrafo_clicked)

btn_recorridosGrafo = Button(description='Ejecutar', width='extended')
def on_button_recorridosGrafo_clicked(b):
    with out:
        try:
            try:
                this.estructura
                state, comment = validarEstructura([4,5], this.type)
                if not state:
                    print(comment)
            except:
                raise Exception("\tLa estructura no ha sido creada")
            if state:
                recorrido = grafoRecorrido.value
                nodo = grafoInputNode.value.strip()
                if '*' in recorrido and len(nodo) > 0:
                    recorrido = recorrido.replace('*', '')
                    print('Grafo labels:')
                    print(grafoLabels.value)
                    recorridosGraph(this.estructura, this.type, grafoLabels.value, recorrido, nodo)
                elif '*' in recorrido:
                    recorrido = recorrido.replace('*', '')
                    #out.clear_output()
                    displayGraph(this.estructura, this.type, grafoLabels.value)
                    raise Exception('ERROR: Para el recorrido ' + recorrido + ' debe ingresar un vertice')
                else:
                    recorridosGraph(this.estructura, this.type, grafoLabels.value, recorrido)
                grafoInputNode.value = ''
        except Exception as e:
            #out.clear_output()
            print('Hubo un problema al intentar ejecutar el recorrido en el grafo')
            print(e)
btn_recorridosGrafo.on_click(on_button_recorridosGrafo_clicked)

#### Opciones Grafos - No Dirigidos

In [36]:
btn_crearGrafoOptionNoDirigidos = Button(description='Crear', width='extended')
def on_button_crearGrafoOptionND_clicked(b):
    with out:
        out.clear_output()
        this.type = 5
        try:
            if listaInit.value == 'Estática':
                try:
                    name = 'creation files/graph_1.json' 
                    json_data = open(name)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo estatico " + name
                    raise Exception (e)               
                this.estructura = crearGraph(init = listaInit.value, 
                                             tipo = this.type, 
                                             data = data, 
                                             file = this.file,
                                             labels=grafoLabels.value)
            elif listaInit.value == 'Archivo':
                try:
                    root = tk.Tk()
                    file_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Json File", "*.json")])
                    root.destroy()
                    json_data = open(file_path)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + file_path
                    raise Exception (e)
                this.estructura = crearGraph(init = listaInit.value, 
                                tipo = this.type, 
                                data = data, 
                                file = this.file,
                                labels=grafoLabels.value)
            else:
                this.estructura = crearGraph(init = listaInit.value, 
                                tipo = this.type,
                                file = this.file, 
                                labels=grafoLabels.value)
        except Exception as e:
            print('Hubo un problema al intentar crear el grafo No Dirigido')
            print(e)
btn_crearGrafoOptionNoDirigidos.on_click(on_button_crearGrafoOptionND_clicked)


#### Opciones - Arreglos

In [37]:
arreglosInit = widgets.RadioButtons(
    options = ['Vacia', 'Random', 'Estática' ,'Archivo'],
    disabled = False
)

arregloInputNode = widgets.Text(
    value = '',
    placeholder = 'Valor del elemento',
    disabled = False,
    layout = {'width': '140px', 'justify_content':'center'}
)

btn_crearArreglo = Button(description='Crear', width='extended')
def on_button_crearArreglo_clicked(b):
    with out:
        out.clear_output()
        try:
            if arreglosInit.value == 'Estática':
                try:
                    name = 'creation files/lista_enlazada_1.json'
                    json_data = open(name)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + name
                    raise Exception (e)               
                this.estructura = crearArreglo(this.file, arreglosInit.value, data)
                this.type = 5
            elif arreglosInit.value == 'Archivo':
                try:
                    root = tk.Tk()
                    file_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Json File", "*.json")])
                    root.destroy()
                    json_data = open(file_path)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + file_path
                    raise Exception (e)
                this.estructura = crearArreglo(this.file, arreglosInit.value, data)
                this.type = 5
            else:
                this.estructura = crearArreglo(this.file, arreglosInit.value)
                this.type = 5
        except Exception as e:
            print('Hubo un problema al intentar crear el arreglo e1')
            print(e)
btn_crearArreglo.on_click(on_button_crearArreglo_clicked)

btn_addNodeArreglo = Button(description='Añadir Último', width='extended')
def on_button_addNodeArreglo_clicked(b):
    with out:
        value = arregloInputNode.value.strip() 
        print("Añadir:", str(value))
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([5],this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    print("Añadir elemento arreglo")
                    anadirNodoArreglo(this.estructura, data)
                    arregloInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar añadir un elemento')
                print(e)
btn_addNodeArreglo.on_click(on_button_addNodeArreglo_clicked)

btn_addNodeArregloFirst = Button(description='Añadir Primero', width='extended')
def on_button_addNodeArregloFirst_clicked(b):
    with out:
        print("Anadir First: val->")
        value = arregloInputNode.value.strip() 
        print(value)
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([5], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    print("Anadir elemento arreglo")
                    anadirNodoArregloFirst(this.estructura, data)
                    arregloInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar añadir un elemento')
                print(e)
btn_addNodeArregloFirst.on_click(on_button_addNodeArregloFirst_clicked)

btn_deleteNodeArreglo = Button(description='Eliminar', width='extended')
def on_button_deleteNodeArreglo_clicked(b):
    with out:
        value = arregloInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([5], this.type)
                    if not state:
                       print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    eliminarNodoArreglo(this.estructura, data)
                    arregloInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar eliminar un elemento')
                print(e)
btn_deleteNodeArreglo.on_click(on_button_deleteNodeArreglo_clicked)

btn_findAdjNodeArreglo = Button(description='Adyacentes', width='extended')
def on_button_findAdjNodeArreglo_clicked(b):
    with out:
        value = arregloInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([5], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:
                    findAdjacentNodeArreglo(this.estructura, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar los adyacentes de un elemento')
                print(e)
btn_findAdjNodeArreglo.on_click(on_button_findAdjNodeArreglo_clicked)


btn_findNodeArreglo = Button(description='Validar', width='extended')
def on_button_findNodeArreglo_clicked(b):
    with out:
        value = arregloInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    state, comment = validarEstructura([5], this.type)
                    if not state:
                        print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                if state:                
                    encontrarNodoArreglo(this.estructura, data)
                    arregloInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar un elemento')
                print(e)
btn_findNodeArreglo.on_click(on_button_findNodeArreglo_clicked)


btn_todosNodosArreglo = Button(description='Encontrar Todos', width='extended')
def on_button_todosNodosArreglo_clicked(b):
    with out:    
        try:
            try:
                this.estructura
                state, comment = validarEstructura([5], this.type)
                if not state:
                    print(comment)
            except:
                raise Exception("\tLa estructura no ha sido creada")
            if state:
                darTodosLosNodosArreglo(this.estructura)
        except Exception as e:
            print('Hubo un problema al intentar encontrar todos los elementos')
            print(e)
btn_todosNodosArreglo.on_click(on_button_todosNodosArreglo_clicked)




#### Opciones - HashLP

In [38]:
HashLPInit = widgets.RadioButtons(
    options = ['Vacia', 'Random', 'Estática' ,'Archivo'],
    disabled = False
)

HashInputNode = widgets.Text(
    value = '',
    placeholder = 'Valor del elemento',
    disabled = False,
    layout = {'width': '140px', 'justify_content':'center'}
)

btn_crearHashLP = Button(description='Crear', width='extended')
def on_button_crearHashLP_clicked(b):
    with out:
        out.clear_output()
        print(HashLPInit.value)
        try:
            if HashLPInit.value == 'Estática':
                try:
                    name = 'creation files/lista_enlazada_1.json'
                    json_data = open(name)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + name
                    raise Exception (e)               
                this.estructura = crearHashLP(this.file, HashLPInit.value, data)
            elif HashLPInit.value == 'Archivo':
                try:
                    root = tk.Tk()
                    file_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Json File", "*.json")])
                    root.destroy()
                    json_data = open(file_path)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + file_path
                    raise Exception (e)
                this.estructura = crearHashLP(this.file, HashLPInit.value, data)
            else:
                this.estructura = crearHashLP(this.file, HashLPInit.value)
        except Exception as e:
            print('Hubo un problema al intentar crear el Hash e1')
            print(e)
btn_crearHashLP.on_click(on_button_crearHashLP_clicked)

btn_addNodeHashLP = Button(description='Añadir', width='extended')
def on_button_addNodeHashLP_clicked(b):
    with out:
        print("Anadir")
        value = HashInputNode.value.strip() 
        print(value)
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    #state, comment = validarEstructura([1,2], this.type)
                    #if not state:
                     #   print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                #if state:
                if True:
                    print("Anadir elemento HashLP")
                    anadirNodoHashLP(this.estructura, data)
                    HashInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar añadir un elemento')
                print(e)
btn_addNodeHashLP.on_click(on_button_addNodeHashLP_clicked)


btn_deleteNodeHashLP = Button(description='Eliminar', width='extended')
def on_button_deleteNodeHashLP_clicked(b):
    with out:
        value = HashInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    #state, comment = validarEstructura([1,2], this.type)
                    #if not state:
                     #   print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                #if state:
                if True:
                    print('Entro a eliminar')
                    eliminarNodoHashLP(this.estructura, data)
                    HashInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar eliminar un elemento')
                print(e)
btn_deleteNodeHashLP.on_click(on_button_deleteNodeHashLP_clicked)

btn_findNodeHashLP = Button(description='Validar', width='extended')
def on_button_findNodeHashLP_clicked(b):
    with out:
        value = HashInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    #state, comment = validarEstructura([5], this.type)
                    #if not state:
                     #   print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                #if state:                
                if True:
                    encontrarNodoHashLP(this.estructura, data)
                    arregloInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar un elemento')
                print(e)
btn_findNodeHashLP.on_click(on_button_findNodeHashLP_clicked)

### Opciones - RBT

In [39]:
btn_crearArbolRBTOption = Button(description='Crear', width='extended')
def on_button_crearArbolRBTOption_clicked(b):
    with out:
        out.clear_output()
        try:
            this.type = 10
            if listaInit.value == 'Estática':
                try:
                    name = 'creation files/bst_1.json' 
                    json_data = open(name)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + name
                    raise Exception (e)               
                this.estructura = crearRBT(listaInit.value, this.file, data)
            elif listaInit.value == 'Archivo':
                try:
                    root = tk.Tk()
                    file_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Json File", "*.json")])
                    root.destroy()
                    json_data = open(file_path)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + file_path
                    raise Exception (e)
                this.estructura = crearRBT(listaInit.value, this.file, data)
            else:
                this.estructura = crearRBT(listaInit.value, this.file)
        except Exception as e:
            print('Hubo un problema al intentar crear el arbol')
            print(e)
btn_crearArbolRBTOption.on_click(on_button_crearArbolRBTOption_clicked)

btn_addNodeArbolRBTOption = Button(description='Añadir', width='extended')
def on_button_addNodeArbolRBTOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                   # state, comment = validarEstructura([3], this.type)
                    #if not state:
                     #   print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                #if state:
                if True:
                    print('Añadir elemento RBT')
                    anadirNodoRBT(this.estructura, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar añadir un elemento')
                print(e)
btn_addNodeArbolRBTOption.on_click(on_button_addNodeArbolRBTOption_clicked)

btn_deleteNodeArbolRBTOption = Button(description='Eliminar', width='extended')
def on_button_deleteNodeArbolRBTOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                   # state, comment = validarEstructura([3], this.type)
                    #if not state:
                     #   print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
               # if state:
                if True:
                    eliminarNodoRBT(this.estructura, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar eliminar un elemento')
                print(e)
btn_deleteNodeArbolRBTOption.on_click(on_button_deleteNodeArbolRBTOption_clicked)

btn_findNodeArbolRBTOption = Button(description='Validar', width='extended')
def on_button_findNodeArbolRBTOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    #state, comment = validarEstructura([3], this.type)
                    #if not state:
                     #   print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                #if state:
                if True:
                    encontrarNodoRBT(this.estructura, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar un elemento')
                print(e)
btn_findNodeArbolRBTOption.on_click(on_button_findNodeArbolRBTOption_clicked)

btn_findAdjNodeArbolRBTOption = Button(description='Adyacentes', width='extended')
def on_button_findAdjNodeArbolRBTOption_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    #state, comment = validarEstructura([3], this.type)
                    #if not state:
                     #   print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                #if state:
                if True:
                    findAdjacentNodoRBT(this.estructura, data)
                    listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar los adyacentes de un elemento')
                print(e)
btn_findAdjNodeArbolRBTOption.on_click(on_button_findAdjNodeArbolRBTOption_clicked)

btn_todosNodosArbolRBT = Button(description='Listar Elementos', width='extended')
def on_button_todosNodosArbolRBT_clicked(b):
    with out:
        out.clear_output()
        try:
            try:
                this.estructura
                #state, comment = validarEstructura([3], this.type)
                #if not state:
                 #   print(comment)
            except:
                raise Exception("\tLa estructura no ha sido creada")
           # if state:
            if True:
                listarNodosRBT(this.estructura, listaOrden.value)
        except Exception as e:
            print('Hubo un problema al intentar encontrar los adyacentes de un elemento')
            print(e)
btn_todosNodosArbolRBT.on_click(on_button_todosNodosArbolRBT_clicked)

In [40]:
HashSCInit = widgets.RadioButtons(
    options = ['Vacia', 'Random', 'Estática', 'Archivo'],
    disabled = False
)

HashInputNode = widgets.Text(
    value = '',
    placeholder = 'Valor del elemento',
    disabled = False,
    layout = {'width': '140px', 'justify_content':'center'}
)

btn_crearHashSC = Button(description='Crear', width='extended')
def on_button_crearHashSC_clicked(b):
    with out:
        out.clear_output()
        print(HashSCInit.value)
        try:
            if HashSCInit.value == 'Estática':
                try:
                    name = 'creation files/lista_enlazada_1.json'
                    json_data = open(name)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + name
                    raise Exception (e)               
                this.estructura = crearHashSC(this.file, HashSCInit.value, data)
            elif HashSCInit.value == 'Archivo':
                try:
                    root = tk.Tk()
                    file_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Json File", "*.json")])
                    root.destroy()
                    json_data = open(file_path)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + file_path
                    raise Exception (e)
                this.estructura = crearHashSC(this.file, HashSCInit.value, data)
            else:
                this.estructura = crearHashSC(this.file, HashSCInit.value)
        except Exception as e:
            print('Hubo un problema al intentar crear el Hash e1')
            print(e)
btn_crearHashSC.on_click(on_button_crearHashSC_clicked)

btn_addNodeHashSC = Button(description='Añadir', width='extended')
def on_button_addNodeHashSC_clicked(b):
    with out:
        print("Anadir")
        value = HashInputNode.value.strip() 
        print(value)
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    #state, comment = validarEstructura([1,2], this.type)
                    #if not state:
                     #   print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                #if state:
                if True:
                    print("Anadir elemento HashSC")
                    anadirNodoHashSC(this.estructura, data)
                    HashInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar añadir un elemento')
                print(e)
btn_addNodeHashSC.on_click(on_button_addNodeHashSC_clicked)


btn_deleteNodeHashSC = Button(description='Eliminar', width='extended')
def on_button_deleteNodeHashSC_clicked(b):
    with out:
        value = HashInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    #state, comment = validarEstructura([1,2], this.type)
                    #if not state:
                     #   print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                #if state:
                if True:
                    print('Entro a eliminar')
                    eliminarNodoHashSC(this.estructura, data)
                    HashInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar eliminar un elemento')
                print(e)
btn_deleteNodeHashSC.on_click(on_button_deleteNodeHashSC_clicked)

btn_findNodeHashSC = Button(description='Validar', width='extended')
def on_button_findNodeHashSC_clicked(b):
    with out:
        value = HashInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    #state, comment = validarEstructura([5], this.type)
                    #if not state:
                     #   print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                #if state:                
                if True:
                    encontrarNodoHashSC(this.estructura, data)
                    arregloInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar un elemento')
                print(e)
btn_findNodeHashSC.on_click(on_button_findNodeHashSC_clicked)

In [41]:
HashLPInit = widgets.RadioButtons(
    options = ['Vacia', 'Random', 'Estática' ,'Archivo'],
    disabled = False
)

HashInputNode = widgets.Text(
    value = '',
    placeholder = 'Valor del elemento',
    disabled = False,
    layout = {'width': '140px', 'justify_content':'center'}
)

btn_crearHashLP = Button(description='Crear', width='extended')
def on_button_crearHashLP_clicked(b):
    with out:
        out.clear_output()
        print(HashLPInit.value)
        try:
            if HashLPInit.value == 'Estática':
                try:
                    name = 'creation files/lista_enlazada_1.json'
                    json_data = open(name)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + name
                    raise Exception (e)               
                this.estructura = crearHashLP(this.file, HashLPInit.value, data)
            elif HashLPInit.value == 'Archivo':
                try:
                    root = tk.Tk()
                    file_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Json File", "*.json")])
                    root.destroy()
                    json_data = open(file_path)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + file_path
                    raise Exception (e)
                this.estructura = crearHashLP(this.file, HashLPInit.value, data)
            else:
                this.estructura = crearHashLP(this.file, HashLPInit.value)
        except Exception as e:
            print('Hubo un problema al intentar crear el Hash e1')
            print(e)
btn_crearHashLP.on_click(on_button_crearHashLP_clicked)

btn_addNodeHashLP = Button(description='Añadir', width='extended')
def on_button_addNodeHashLP_clicked(b):
    with out:
        print("Anadir")
        value = HashInputNode.value.strip() 
        print(value)
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    #state, comment = validarEstructura([1,2], this.type)
                    #if not state:
                     #   print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                #if state:
                if True:
                    print("Anadir elemento HashLP")
                    anadirNodoHashLP(this.estructura, data)
                    HashInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar añadir un elemento')
                print(e)
btn_addNodeHashLP.on_click(on_button_addNodeHashLP_clicked)


btn_deleteNodeHashLP = Button(description='Eliminar', width='extended')
def on_button_deleteNodeHashLP_clicked(b):
    with out:
        value = HashInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    #state, comment = validarEstructura([1,2], this.type)
                    #if not state:
                     #   print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                #if state:
                if True:
                    print('Entro a eliminar')
                    eliminarNodoHashLP(this.estructura, data)
                    HashInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar eliminar un elemento')
                print(e)
btn_deleteNodeHashLP.on_click(on_button_deleteNodeHashLP_clicked)

btn_findNodeHashLP = Button(description='Validar', width='extended')
def on_button_findNodeHashLP_clicked(b):
    with out:
        value = HashInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                    #state, comment = validarEstructura([5], this.type)
                    #if not state:
                     #   print(comment)
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                #if state:                
                if True:
                    encontrarNodoHashLP(this.estructura, data)
                    arregloInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar un elemento')
                print(e)
btn_findNodeHashLP.on_click(on_button_findNodeHashLP_clicked)

#### Opciones - Nuevo

In [42]:
# -------------------------- Cargar Estructura - Nueva --------------------------#
#btn_ejecutarNuevaOpcion = Button(description='Ejecutar Nuevo', width='extended')
#def on_button_ejecutarNuevaOpcion_clicked(b):
#    with out:
#        '''
#        En esta parte se hace el llamado a los metodos de enlace, se validan los inputs, se grafica, etc.
#        '''
#        print('Nueva Opcion')
#btn_ejecutarNuevaOpcion.on_click(on_button_ejecutarNuevaOpcion_clicked)
# ------------------------------------------------------------------------------#

### Layouts

Sección en la que se definen los layout de los componentes (botones y opciones) para cada una de las estructuras de datos

#### Layout - Listas Enlazadas

In [43]:
items_optionsCreateList = [
    Box([Label(value=' ')]),
    Box([Label(value='Crear Lista Enlazada')], layout=form_item_layout),
   # Box([Label(value='Tipo:')]),
    #Box([listaType]),
    Box([Label(value='Inicialización:')]),
    Box([listaInit]),
    Box([btn_crear]),
]
items_optionsCreateListDoble = [
    Box([Label(value=' ')]),
    Box([Label(value='Crear Lista Enlazada')], layout=form_item_layout),
   # Box([Label(value='Tipo:')]),
    #Box([listaType]),
    Box([Label(value='Inicialización:')]),
    Box([listaInit]),
    Box([btn_crearDoble]),
]
items_optionsAddNode = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Elemento Último')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_addNode], layout=form_item_layout),
]
items_optionsAddNodeFirst = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Elemento Primero')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_addNode_first], layout=form_item_layout),
]
items_optionsDeleteNode = [
    Box([Label(value=' ')]),
    Box([Label(value='Eliminar Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_deleteNode], layout=form_item_layout),
]
items_optionsFindNode = [
    Box([Label(value=' ')]),
    Box([Label(value='Existe Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findNode], layout=form_item_layout),
]
items_optionsFindAdjNode = [
    Box([Label(value=' ')]),
    Box([Label(value='Adyacentes Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findAdjNode], layout=form_item_layout),
]
items_optionsFindAllNodes = [
    Box([Label(value=' ')]),
    Box([btn_todosNodos], layout=form_item_layout),
]

layoutOptionsCreate = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddNode = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddNodeFirst = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsDeleteNode = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindNode = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAdjNode = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAllNodes = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)

optionsCreateList = Box(items_optionsCreateList, layout=layoutOptionsCreate)
optionsCreateListDoble = Box(items_optionsCreateListDoble, layout=layoutOptionsCreate)
optionsAddNode = Box(items_optionsAddNode, layout=layoutOptionsAddNode)
optionsAddNodeFirst = Box(items_optionsAddNodeFirst, layout=layoutOptionsAddNodeFirst)
optionsDeleteNode = Box(items_optionsDeleteNode, layout=layoutOptionsDeleteNode)
optionsFindNode = Box(items_optionsFindNode, layout=layoutOptionsFindNode)
optionsFindAdjNode = Box(items_optionsFindAdjNode, layout=layoutOptionsFindAdjNode)
optionsFindAllNodes = Box(items_optionsFindAllNodes, layout=layoutOptionsFindAllNodes)


#### Layout - Árboles BST 

In [44]:
items_optionsCreateArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Crear Árbol Binario')], layout=form_item_layout),
    Box([Label(value='Inicialización:')]),
    Box([listaInit]),
    Box([btn_crearArbolOption]),
]
items_optionsAddNodeArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_addNodeArbolOption], layout=form_item_layout),
]
items_optionsDeleteNodeArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Eliminar Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_deleteNodeArbolOption], layout=form_item_layout),
]
items_optionsFindNodeArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Existe Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findNodeArbolOption], layout=form_item_layout),
]
items_optionsFindAdjNodeArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Adyacentes Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findAdjNodeArbolOption], layout=form_item_layout),
]

items_optionsFindAllNodesArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Listar todos los elementos')], layout=form_item_layout),
    Box([Label(value='Orden:')]),
    Box([listaOrden]),
    Box([btn_todosNodosArbol]),
]

layoutOptionsCreateArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddNodeArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsDeleteNodeArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindNodeArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAdjNodeArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAllNodesArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)

optionsCreateArbol = Box(items_optionsCreateArbol, layout=layoutOptionsCreateArbol)
optionsAddNodeArbol = Box(items_optionsAddNodeArbol, layout=layoutOptionsAddNodeArbol)
optionsDeleteNodeArbol = Box(items_optionsDeleteNodeArbol, layout=layoutOptionsDeleteNodeArbol)
optionsFindNodeArbol = Box(items_optionsFindNodeArbol, layout=layoutOptionsFindNodeArbol)
optionsFindAdjNodeArbol = Box(items_optionsFindAdjNodeArbol, layout=layoutOptionsFindAdjNodeArbol)
optionsFindAllNodesArbol = Box(items_optionsFindAllNodesArbol, layout=layoutOptionsFindAllNodesArbol)

#### Layout - Grafos

In [45]:
items_optionsCreateGrafo = [
    Box([Label(value='Crear Grafo')], layout=form_item_layout),
    Box([grafoLabels]),
    Box([Label(value='Tipo:')]),
    #Box([grafoType]),
    Box([Label(value='Inicialización:')]),
    Box([listaInit]),
    Box([btn_crearGrafoOption]),
]
items_optionsCreateGrafoND = [
    Box([Label(value='Crear Grafo')], layout=form_item_layout),
    Box([grafoLabels]),
    Box([Label(value='Tipo:')]),
    #Box([grafoType]),
    Box([Label(value='Inicialización:')]),
    Box([listaInit]),
    Box([btn_crearGrafoOptionNoDirigidos]),
]
items_optionsAddNodeGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Elemento')], layout=form_item_layout),
    Box([grafoLabels]),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_addNodeGrafoOption], layout=form_item_layout),
]
items_optionsDeleteNodeGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Eliminar Elemento')], layout=form_item_layout),
    Box([grafoLabels]),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_deleteNodeGrafoOption], layout=form_item_layout),
]
items_optionsFindNodeGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Existe Elemento')], layout=form_item_layout),
    Box([grafoLabels]),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findNodeGrafoOption], layout=form_item_layout),
]
items_optionsFindAdjNodeGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Adyacentes Elemento')], layout=form_item_layout),
    Box([grafoLabels]),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findAdjNodeGrafoOption], layout=form_item_layout),
]
items_optionsAddEdgeGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Arco')], layout=form_item_layout),
    Box([grafoLabels]),
    Box([grafoInputNodeInit], layout=form_item_layout),
    Box([grafoInputNodeEnd], layout=form_item_layout),
    Box([grafoInputEdgeWeight], layout=form_item_layout),
    Box([btn_addEdgeOption], layout=form_item_layout),
]
items_optionsAllNodesGrafo = [
    Box([Label(value=' ')]),
    Box([grafoLabels]),
    Box([btn_todosNodosGrafo], layout=form_item_layout),
]
items_optionsRecorridosGrafo = [
    Box([grafoLabels]),
    Box([Label(value='Recorridos Grafo')], layout=form_item_layout),
    Box([grafoInputNode], layout=form_item_layout),
    Box([grafoRecorrido], layout=form_item_layout),
    Box([btn_recorridosGrafo], layout=form_item_layout),
]
items_optionsRecorridosGrafoD = [
    Box([grafoLabels]),
    Box([Label(value='Recorridos Grafo')], layout=form_item_layout),
    Box([grafoInputNode], layout=form_item_layout),
    Box([grafoRecorridoD], layout=form_item_layout),
    Box([btn_recorridosGrafo], layout=form_item_layout),
]

layoutOptionsRecorridosGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAllNodesGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsCreateGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddNodeGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsDeleteNodeGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindNodeGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddEdgeGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAdjNodeGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)

optionsCreateGrafo = Box(items_optionsCreateGrafo, layout=layoutOptionsCreateGrafo)
optionsCreateGrafoND = Box(items_optionsCreateGrafoND, layout=layoutOptionsCreateGrafo)
optionsAddNodeGrafo = Box(items_optionsAddNodeGrafo, layout=layoutOptionsAddNodeGrafo)
optionsDeleteNodeGrafo = Box(items_optionsDeleteNodeGrafo, layout=layoutOptionsDeleteNodeGrafo)
optionsFindNodeGrafo = Box(items_optionsFindNodeGrafo, layout=layoutOptionsFindNodeGrafo)
optionsFindAdjNodeGrafo = Box(items_optionsFindAdjNodeGrafo, layout=layoutOptionsFindAdjNodeGrafo)
optionsAddEdgeGrafo = Box(items_optionsAddEdgeGrafo, layout=layoutOptionsAddEdgeGrafo)
optionsAllNodesGrafo = Box(items_optionsAllNodesGrafo, layout=layoutOptionsAllNodesGrafo)
optionsRecorridosGrafo = Box(items_optionsRecorridosGrafo, layout=layoutOptionsRecorridosGrafo)
optionsRecorridosGrafoD = Box(items_optionsRecorridosGrafoD, layout=layoutOptionsRecorridosGrafo)

#### Layout - Arreglo

In [46]:
items_optionsCreateArreglo = [
    Box([Label(value=' ')]),
    Box([Label(value='Crear Arreglo')], layout=form_item_layout),
    Box([Label(value='Inicialización:')]),
    Box([arreglosInit]),
    Box([btn_crearArreglo]),
]
items_optionsAddNodeArreglo = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Elemento Último')], layout=form_item_layout),
    Box([arregloInputNode], layout=form_item_layout),
    Box([btn_addNodeArreglo], layout=form_item_layout),
]
items_optionsAddNodeArregloFirst = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Elemento Primero')], layout=form_item_layout),
    Box([arregloInputNode], layout=form_item_layout),
    Box([btn_addNodeArregloFirst], layout=form_item_layout),
]
items_optionsDeleteNodeArreglo = [
    Box([Label(value=' ')]),
    Box([Label(value='Eliminar Elemento')], layout=form_item_layout),
    Box([arregloInputNode], layout=form_item_layout),
    Box([btn_deleteNodeArreglo], layout=form_item_layout),
]
items_optionsFindNodeArreglo = [
    Box([Label(value=' ')]),
    Box([Label(value='Encontrar Elemento')], layout=form_item_layout),
    Box([arregloInputNode], layout=form_item_layout),
    Box([btn_findNodeArreglo], layout=form_item_layout),
]
items_optionsFindAdjNodeArreglo = [
    Box([Label(value=' ')]),
    Box([Label(value='Adyacentes Elemento')], layout=form_item_layout),
    Box([arregloInputNode], layout=form_item_layout),
    Box([btn_findAdjNodeArreglo], layout=form_item_layout),
]
items_optionsFindAllNodesArreglo = [
    Box([Label(value=' ')]),
    Box([btn_todosNodosArreglo], layout=form_item_layout),
]


layoutOptionsCreateArreglo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddNodeArreglo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddNodeArregloFirst = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsDeleteNodeArreglo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindNodeArreglo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAdjNodeArreglo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAllNodesArreglo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)

optionsCreateArreglo = Box(items_optionsCreateArreglo, layout=layoutOptionsCreate)
optionsAddNodeArreglo = Box(items_optionsAddNodeArreglo, layout=layoutOptionsAddNode)
optionsAddNodeArregloFirst = Box(items_optionsAddNodeArregloFirst, layout=layoutOptionsAddNodeArregloFirst)
optionsDeleteNodeArreglo = Box(items_optionsDeleteNodeArreglo, layout=layoutOptionsDeleteNode)
optionsFindNodeArreglo = Box(items_optionsFindNodeArreglo, layout=layoutOptionsFindNodeArreglo)
optionsFindAdjNodeArreglo = Box(items_optionsFindAdjNodeArreglo, layout=layoutOptionsFindAdjNodeArreglo)
optionsFindAllNodesArreglo = Box(items_optionsFindAllNodesArreglo, layout=layoutOptionsFindAllNodesArreglo)


#### Layout - HashLP

In [47]:
items_optionsCreateHashLP = [
    Box([Label(value=' ')]),
    Box([Label(value='Crear Hash LP')], layout=form_item_layout),
    Box([Label(value='Inicialización:')]),
    Box([HashLPInit]),
    Box([btn_crearHashLP]),
]
items_optionsAddNodeHashLP = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Elemento')], layout=form_item_layout),
    Box([HashInputNode], layout=form_item_layout),
    Box([btn_addNodeHashLP], layout=form_item_layout),
]
items_optionsDeleteNodeHashLP = [
    Box([Label(value=' ')]),
    Box([Label(value='Eliminar Elemento')], layout=form_item_layout),
    Box([HashInputNode], layout=form_item_layout),
    Box([btn_deleteNodeHashLP], layout=form_item_layout),
]
items_optionsFindNodeHashLP = [
    Box([Label(value=' ')]),
    Box([Label(value='Encontrar Elemento')], layout=form_item_layout),
    Box([HashInputNode], layout=form_item_layout),
    Box([btn_findNodeHashLP], layout=form_item_layout),
]

layoutOptionsCreateHashLP = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddNodeHashLP = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsDeleteNodeHashLP = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindNodeHashLP = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)

optionsCreateHashLP = Box(items_optionsCreateHashLP, layout=layoutOptionsCreateHashLP)
optionsAddNodeHashLP = Box(items_optionsAddNodeHashLP, layout=layoutOptionsAddNodeHashLP)
optionsDeleteNodeHashLP = Box(items_optionsDeleteNodeHashLP, layout=layoutOptionsDeleteNodeHashLP)
optionsFindNodeHashLP = Box(items_optionsFindNodeHashLP, layout=layoutOptionsFindNodeHashLP)

#### Layout - SC

In [48]:
items_optionsCreateHashSC = [
    Box([Label(value=' ')]),
    Box([Label(value='Crear Hash SCC')], layout=form_item_layout),
    Box([Label(value='Inicialización:')]),
    Box([HashSCInit]),
    Box([btn_crearHashSC]),
]
items_optionsAddNodeHashSC = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Elemento')], layout=form_item_layout),
    Box([HashInputNode], layout=form_item_layout),
    Box([btn_addNodeHashSC], layout=form_item_layout),
]
items_optionsDeleteNodeHashSC = [
    Box([Label(value=' ')]),
    Box([Label(value='Eliminar Elemento')], layout=form_item_layout),
    Box([HashInputNode], layout=form_item_layout),
    Box([btn_deleteNodeHashSC], layout=form_item_layout),
]
items_optionsFindNodeHashSC = [
    Box([Label(value=' ')]),
    Box([Label(value='Encontrar Elemento')], layout=form_item_layout),
    Box([HashInputNode], layout=form_item_layout),
    Box([btn_findNodeHashSC], layout=form_item_layout),
]

layoutOptionsCreateHashSC = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddNodeHashSC = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsDeleteNodeHashSC = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindNodeHashSC = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)

optionsCreateHashSC = Box(items_optionsCreateHashSC, layout=layoutOptionsCreateHashSC)
optionsAddNodeHashSC = Box(items_optionsAddNodeHashSC, layout=layoutOptionsAddNodeHashSC)
optionsDeleteNodeHashSC = Box(items_optionsDeleteNodeHashSC, layout=layoutOptionsDeleteNodeHashSC)
optionsFindNodeHashSC = Box(items_optionsFindNodeHashSC, layout=layoutOptionsFindNodeHashSC)

### Layout - RBT

In [49]:
items_optionsCreateArbolRBT = [
    Box([Label(value=' ')]),
    Box([Label(value='Crear Árbol RBT')], layout=form_item_layout),
    Box([Label(value='Inicialización:')]),
    Box([listaInit]),
    Box([btn_crearArbolRBTOption]),
]
items_optionsAddNodeArbolRBT = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_addNodeArbolRBTOption], layout=form_item_layout),
]
items_optionsDeleteNodeArbolRBT = [
    Box([Label(value=' ')]),
    Box([Label(value='Eliminar Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_deleteNodeArbolRBTOption], layout=form_item_layout),
]
items_optionsFindNodeArbolRBT = [
    Box([Label(value=' ')]),
    Box([Label(value='Existe Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findNodeArbolRBTOption], layout=form_item_layout),
]
items_optionsFindAdjNodeArbolRBT = [
    Box([Label(value=' ')]),
    Box([Label(value='Adyacentes Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findAdjNodeArbolRBTOption], layout=form_item_layout),
]

items_optionsFindAllNodesArbolRBT = [
    Box([Label(value=' ')]),
    Box([Label(value='Listar todos los elemtos')], layout=form_item_layout),
    Box([Label(value='Orden:')]),
    Box([listaOrden]),
    Box([btn_todosNodosArbolRBT]),
]

layoutOptionsCreateArbolRBT = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddNodeArbolRBT = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsDeleteNodeArbolRBT = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindNodeArbolRBT = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAdjNodeArbolRBT = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAllNodesArbolRBT = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)

optionsCreateArbolRBT = Box(items_optionsCreateArbolRBT, layout=layoutOptionsCreateArbolRBT)
optionsAddNodeArbolRBT = Box(items_optionsAddNodeArbolRBT, layout=layoutOptionsAddNodeArbolRBT)
optionsDeleteNodeArbolRBT = Box(items_optionsDeleteNodeArbolRBT, layout=layoutOptionsDeleteNodeArbolRBT)
optionsFindNodeArbolRBT = Box(items_optionsFindNodeArbolRBT, layout=layoutOptionsFindNodeArbolRBT)
optionsFindAdjNodeArbolRBT = Box(items_optionsFindAdjNodeArbolRBT, layout=layoutOptionsFindAdjNodeArbolRBT)
optionsFindAllNodesArbolRBT = Box(items_optionsFindAllNodesArbolRBT, layout=layoutOptionsFindAllNodesArbolRBT)

#### Layout - Nueva Estructura

In [50]:
# -------------------------- Cargar Estructura - Nueva --------------------------#
#layoutOptionsNuevoBoton = Layout(
#    display='none',
#    flex_flow='column',
#    align_items='stretch',
#    width='auto',
#    justify_content='center'
#)
#
#items_optionsNuevoBoton = [
#    # Insertar los elementos que se desean para el detalle de la nueva opcion
#    Box([Label(value='Nuevo Boton')], layout=form_item_layout),
#    Box([btn_ejecutarNuevaOpcion], layout=form_item_layout),
#]
#
#optionsNuevoBoton = Box(items_optionsNuevoBoton, layout=layoutOptionsNuevoBoton)
#
# ------------------------------------------------------------------------------#



### Contruir Interfaz

En esta sección se unen todos los elementos de la interfaz para posteriormente crearla

#### Contruir Layouts

In [51]:
allOptions = [optionsCreateList,
              optionsCreateListDoble,
            optionsAddNode,
            optionsAddNodeFirst,
            optionsDeleteNode,
            optionsFindNode,
            optionsFindAdjNode,
            optionsFindAllNodes,
            optionsCreateArbol,
            optionsAddNodeArbol,
            optionsDeleteNodeArbol,
            optionsFindNodeArbol,
            optionsFindAdjNodeArbol,
            optionsFindAllNodesArbol,
            optionsCreateGrafo,
            optionsCreateGrafoND,
            optionsAddNodeGrafo,
            optionsDeleteNodeGrafo,
            optionsFindNodeGrafo,
            optionsAddEdgeGrafo,
            optionsFindAdjNodeGrafo,
            optionsAllNodesGrafo,
            optionsRecorridosGrafo,
            optionsRecorridosGrafoD,
            optionsAddNodeArreglo,
            optionsAddNodeArregloFirst,
            optionsFindNodeArreglo,
            optionsFindAdjNodeArreglo,
            optionsFindAllNodesArreglo,
            optionsCreateHashLP,
            optionsAddNodeHashLP,
            optionsDeleteNodeHashLP,
            optionsFindNodeHashLP,
            optionsCreateHashSC,
            optionsAddNodeHashSC,
            optionsDeleteNodeHashSC,
            optionsFindNodeHashSC,
            optionsCreateArbolRBT,
            optionsAddNodeArbolRBT,
            optionsDeleteNodeArbolRBT,
            optionsFindNodeArbolRBT,
            optionsFindAdjNodeArbolRBT,
            optionsFindAllNodesArbolRBT
            # -------------------------- Cargar Estructura - Nueva --------------------------#
            #optionsNuevoBoton
            # ------------------------------------------------------------------------------#
            ]            

def hideShowOptions(option):
    for i in allOptions:
        i.layout.visibility = 'hidden'
        i.layout.display = 'none'
    listaInputNode.value = ''
    grafoInputNodeInit.value = ''
    grafoInputNodeEnd.value = ''
    grafoInputEdgeWeight.value = 0.00
       
    option.layout.display = 'block'
    option.layout.visibility = "visible"

# -----------------------------Panel de Opciones Lista Enlazada-----------------------------
btn_crearLista = Button(description='Crear Lista Enlazada', width='extended')
def on_button_crearLista_clicked(b):
    with out:
        hideShowOptions(optionsCreateList)
btn_crearLista.on_click(on_button_crearLista_clicked)

btn_crearListaDoble = Button(description='Crear Lista Enlazada', width='extended')
def on_button_crearListaDoble_clicked(b):
    with out:
        hideShowOptions(optionsCreateListDoble)
btn_crearListaDoble.on_click(on_button_crearListaDoble_clicked)

btn_addNode = Button(description='Añadir Último', width='extended')
def on_button_addNode_clicked(b):
    with out:
        hideShowOptions(optionsAddNode)
btn_addNode.on_click(on_button_addNode_clicked)

btn_addNode_first = Button(description='Añadir Primero', width='extended')
def on_button_addNode_first_clicked(b):
    with out:
        hideShowOptions(optionsAddNodeFirst)
btn_addNode_first.on_click(on_button_addNode_first_clicked)

btn_deleteNode = Button(description='Eliminar Elemento', width='extended')
def on_button_deleteNode_clicked(b):
    with out:
        hideShowOptions(optionsDeleteNode)
btn_deleteNode.on_click(on_button_deleteNode_clicked)

btn_findNode = Button(description='Existe Elemento', width='extended')
def on_button_findNode_clicked(b):
    with out:
        hideShowOptions(optionsFindNode)
btn_findNode.on_click(on_button_findNode_clicked)

btn_findAdjNode = Button(description='Encontrar Adyacentes', width='extended')
def on_button_findAdjNode_clicked(b):
    with out:
        hideShowOptions(optionsFindAdjNode)
btn_findAdjNode.on_click(on_button_findAdjNode_clicked)

btn_findAllNodes = Button(description='Encontrar Todos', width='extended')
def on_button_findAllNodes_clicked(b):
    with out:
        hideShowOptions(optionsFindAllNodes)
btn_findAllNodes.on_click(on_button_findAllNodes_clicked)

# -----------------------------Panel de Opciones Arbol-----------------------------
btn_crearArbol = Button(description='Crear Árbol Binario', width='extended')
def on_button_crearArbol_clicked(b):
    with out:
        hideShowOptions(optionsCreateArbol)
btn_crearArbol.on_click(on_button_crearArbol_clicked)

btn_addNodeArbol = Button(description='Añadir Elemento', width='extended')
def on_button_addNodeArbol_clicked(b):
    with out:
        hideShowOptions(optionsAddNodeArbol)
btn_addNodeArbol.on_click(on_button_addNodeArbol_clicked)

btn_deleteNodeArbol = Button(description='Eliminar Elemento', width='extended')
def on_button_deleteNodeArbol_clicked(b):
    with out:
        hideShowOptions(optionsDeleteNodeArbol)
btn_deleteNodeArbol.on_click(on_button_deleteNodeArbol_clicked)

btn_findNodeArbol = Button(description='Existe Elemento', width='extended')
def on_button_findNodeArbol_clicked(b):
    with out:
        hideShowOptions(optionsFindNodeArbol)
btn_findNodeArbol.on_click(on_button_findNodeArbol_clicked)

btn_findAdjNodeArbol = Button(description='Encontrar Adyacentes', width='extended')
def on_button_findAdjNodeArbol_clicked(b):
    with out:
        hideShowOptions(optionsFindAdjNodeArbol)
btn_findAdjNodeArbol.on_click(on_button_findAdjNodeArbol_clicked)

btn_findAllNodesArbol = Button(description='Encontrar Todos', width='extended')
def on_button_findAllNodesArbol_clicked(b):
    with out:
        hideShowOptions(optionsFindAllNodesArbol)
btn_findAllNodesArbol.on_click(on_button_findAllNodesArbol_clicked)

# -----------------------------Panel de Opciones Arbol RBT-----------------------------
btn_crearArbolRBT = Button(description='Crear Árbol RBT', width='extended')
def on_button_crearArbolRBT_clicked(b):
    with out:
        hideShowOptions(optionsCreateArbolRBT)
btn_crearArbolRBT.on_click(on_button_crearArbolRBT_clicked)

btn_addNodeArbolRBT = Button(description='Añadir Elemento', width='extended')
def on_button_addNodeArbolRBT_clicked(b):
    with out:
        hideShowOptions(optionsAddNodeArbolRBT)
btn_addNodeArbolRBT.on_click(on_button_addNodeArbolRBT_clicked)

btn_deleteNodeArbolRBT = Button(description='Eliminar Elemento', width='extended')
def on_button_deleteNodeArbolRBT_clicked(b):
    with out:
        hideShowOptions(optionsDeleteNodeArbolRBT)
btn_deleteNodeArbolRBT.on_click(on_button_deleteNodeArbolRBT_clicked)

btn_findNodeArbolRBT = Button(description='Existe Elemento', width='extended')
def on_button_findNodeArbolRBT_clicked(b):
    with out:
        hideShowOptions(optionsFindNodeArbolRBT)
btn_findNodeArbolRBT.on_click(on_button_findNodeArbolRBT_clicked)

btn_findAdjNodeArbolRBT = Button(description='Encontrar Adyacentes', width='extended')
def on_button_findAdjNodeArbolRBT_clicked(b):
    with out:
        hideShowOptions(optionsFindAdjNodeArbolRBT)
btn_findAdjNodeArbolRBT.on_click(on_button_findAdjNodeArbolRBT_clicked)

btn_findAllNodesArbolRBT = Button(description='Encontrar Todos', width='extended')
def on_button_findAllNodesArbolRBT_clicked(b):
    with out:
        hideShowOptions(optionsFindAllNodesArbolRBT)
btn_findAllNodesArbolRBT.on_click(on_button_findAllNodesArbolRBT_clicked)

# -----------------------------Panel de Opciones Grafo-----------------------------
btn_crearGrafo = Button(description='Crear Grafo', width='extended')
def on_button_crearGrafo_clicked(b):
    with out:
        hideShowOptions(optionsCreateGrafo)
btn_crearGrafo.on_click(on_button_crearGrafo_clicked)

btn_crearGrafoND = Button(description='Crear Grafo', width='extended')
def on_button_crearGrafoND_clicked(b):
    with out:
        hideShowOptions(optionsCreateGrafoND)
btn_crearGrafoND.on_click(on_button_crearGrafoND_clicked)

btn_addNodeGrafo = Button(description='Añadir Vertice', width='extended')
def on_button_addNodeGrafo_clicked(b):
    with out:
        hideShowOptions(optionsAddNodeGrafo)
btn_addNodeGrafo.on_click(on_button_addNodeGrafo_clicked)

btn_deleteNodeGrafo = Button(description='Eliminar Vertice', width='extended')
def on_button_deleteNodeGrafo_clicked(b):
    with out:
        hideShowOptions(optionsDeleteNodeGrafo)
btn_deleteNodeGrafo.on_click(on_button_deleteNodeGrafo_clicked)

btn_findNodeGrafo = Button(description='Existe Vertice', width='extended')
def on_button_findNodeGrafo_clicked(b):
    with out:
        hideShowOptions(optionsFindNodeGrafo)
btn_findNodeGrafo.on_click(on_button_findNodeGrafo_clicked)

btn_addEdgeGrafo = Button(description='Añadir Arco', width='extended')
def on_button_addEdgeGrafo_clicked(b):
    with out:
        hideShowOptions(optionsAddEdgeGrafo)
btn_addEdgeGrafo.on_click(on_button_addEdgeGrafo_clicked)

btn_findAdjNodeGrafo = Button(description='Encontrar Adyacentes', width='extended')
def on_button_findAdjNodeGrafo_clicked(b):
    with out:
        hideShowOptions(optionsFindAdjNodeGrafo)
btn_findAdjNodeGrafo.on_click(on_button_findAdjNodeGrafo_clicked)

btn_findAllNodesGrafo = Button(description='Encontrar Todos', width='extended')
def on_button_findAllNodesGrafo_clicked(b):
    with out:
        hideShowOptions(optionsAllNodesGrafo)
btn_findAllNodesGrafo.on_click(on_button_findAllNodesGrafo_clicked)

btn_recorridosOptionGrafo = Button(description='Realizar Recorrido', width='extended')
def on_button_recorridosOptionsGrafo_clicked(b):
    with out:
        hideShowOptions(optionsRecorridosGrafoD)
btn_recorridosOptionGrafo.on_click(on_button_recorridosOptionsGrafo_clicked)

# -----------------------------Panel de Opciones Arreglo-----------------------------
btn_crearArreglo = Button(description='Crear Arreglo', width='extended')
def on_button_crearArreglo_clicked(b):
    with out:
        hideShowOptions(optionsCreateArreglo)
btn_crearArreglo.on_click(on_button_crearArreglo_clicked)

btn_addNodeArregloFirst = Button(description='Añadir Primero', width='extended')
def on_button_addNodeArregloFirst_clicked(b):
    with out:
        hideShowOptions(optionsAddNodeArregloFirst)
btn_addNodeArregloFirst.on_click(on_button_addNodeArregloFirst_clicked)


btn_addNodeArreglo = Button(description='Añadir Último', width='extended')
def on_button_addNodeArreglo_clicked(b):
    with out:
        hideShowOptions(optionsAddNodeArreglo)
btn_addNodeArreglo.on_click(on_button_addNodeArreglo_clicked)


btn_deleteNodeArreglo = Button(description='Eliminar Elemento', width='extended')
def on_button_deleteNodeArreglo_clicked(b):
    with out:
        hideShowOptions(optionsDeleteNodeArreglo)
btn_deleteNodeArreglo.on_click(on_button_deleteNodeArreglo_clicked)

btn_findNodeArreglo = Button(description='Buscar Elemento', width='extended')
def on_button_FindNodeArreglo_clicked(b):
    with out:
        hideShowOptions(optionsFindNodeArreglo)
btn_findNodeArreglo.on_click(on_button_FindNodeArreglo_clicked)

btn_findAdjNodeArreglo = Button(description='Encontrar Adyacentes', width='extended')
def on_button_findAdjNodeArreglo_clicked(b):
    with out:
        hideShowOptions(optionsFindAdjNodeArreglo)
btn_findAdjNodeArreglo.on_click(on_button_findAdjNodeArreglo_clicked)

btn_findAllNodesArreglo = Button(description='Encontrar Todos', width='extended')
def on_button_findAllNodesArreglo_clicked(b):
    with out:
        hideShowOptions(optionsFindAllNodesArreglo)
btn_findAllNodesArreglo.on_click(on_button_findAllNodesArreglo_clicked)

# -----------------------------Panel de Opciones HashLP-----------------------------
btn_crearHashLP = Button(description='Crear Tabla Hash', width='extended')
def on_button_crearHashLP_clicked(b):
    with out:
        hideShowOptions(optionsCreateHashLP)
btn_crearHashLP.on_click(on_button_crearHashLP_clicked)

btn_addNodeHashLP = Button(description='Añadir Elemento', width='extended')
def on_button_addNodeHashLP_clicked(b):
    with out:
        hideShowOptions(optionsAddNodeHashLP)
btn_addNodeHashLP.on_click(on_button_addNodeHashLP_clicked)


btn_deleteNodeHashLP = Button(description='Eliminar Elemento', width='extended')
def on_button_deleteNodeHashLP_clicked(b):
    with out:
        hideShowOptions(optionsDeleteNodeHashLP)
btn_deleteNodeHashLP.on_click(on_button_deleteNodeHashLP_clicked)

btn_FindNodeHashLP = Button(description='Encontrar Elemento', width='extended')
def on_button_FindNodeHashLP_clicked(b):
    with out:
        hideShowOptions(optionsFindNodeHashLP)
btn_FindNodeHashLP.on_click(on_button_FindNodeHashLP_clicked)

# -----------------------------Panel de Opciones HashSC-----------------------------
btn_crearHashSC = Button(description='Crear Tabla Hash', width='extended')
def on_button_crearHashSC_clicked(b):
    with out:
        hideShowOptions(optionsCreateHashSC)
btn_crearHashSC.on_click(on_button_crearHashSC_clicked)

btn_addNodeHashSC = Button(description='Añadir Elemento', width='extended')
def on_button_addNodeHashSC_clicked(b):
    with out:
        hideShowOptions(optionsAddNodeHashSC)
btn_addNodeHashSC.on_click(on_button_addNodeHashSC_clicked)


btn_deleteNodeHashSC = Button(description='Eliminar Elemento', width='extended')
def on_button_deleteNodeHashSC_clicked(b):
    with out:
        hideShowOptions(optionsDeleteNodeHashSC)
btn_deleteNodeHashSC.on_click(on_button_deleteNodeHashSC_clicked)

btn_FindNodeHashSC = Button(description='Encontrar Elemento', width='extended')
def on_button_FindNodeHashSC_clicked(b):
    with out:
        hideShowOptions(optionsFindNodeHashSC)
btn_FindNodeHashSC.on_click(on_button_FindNodeHashSC_clicked)

# -------------------------- Cargar Estructura - Nueva --------------------------#
#btn_nuevoBoton = Button(description='Nuevo Boton', width='extended')
#def on_button_nuevoBoton_clicked(b):
#    with out:
#        hideShowOptions(optionsNuevoBoton)
#btn_nuevoBoton.on_click(on_button_nuevoBoton_clicked)
# ------------------------------------------------------------------------------#

#### Construir Opciones

In [52]:
layout_canvas = Layout(height='590px', border='1px dotted blue', overflow ='auto')
layout_options = Layout(height='auto', width='auto')

buttons_listas = [
    Box([btn_crearLista], layout=form_item_layout),
    Box([btn_addNode], layout=form_item_layout),
    Box([btn_addNode_first], layout=form_item_layout),
    Box([btn_deleteNode], layout=form_item_layout),
    Box([btn_findNode], layout=form_item_layout),
    Box([btn_findAdjNode], layout=form_item_layout),
    Box([btn_findAllNodes], layout=form_item_layout),
]
buttons_listasDoble = [
    Box([btn_crearListaDoble], layout=form_item_layout),
    Box([btn_addNode], layout=form_item_layout),
    Box([btn_addNode_first], layout=form_item_layout),
    Box([btn_deleteNode], layout=form_item_layout),
    Box([btn_findNode], layout=form_item_layout),
    Box([btn_findAdjNode], layout=form_item_layout),
    Box([btn_findAllNodes], layout=form_item_layout),
]
buttons_arboles = [
    Box([btn_crearArbol], layout=form_item_layout),
    Box([btn_addNodeArbol], layout=form_item_layout),
    Box([btn_deleteNodeArbol], layout=form_item_layout),
    Box([btn_findNodeArbol], layout=form_item_layout),
    Box([btn_findAdjNodeArbol], layout=form_item_layout),
    Box([btn_findAllNodesArbol],layout=form_item_layout)
]
buttons_arbolesRBT = [
    Box([btn_crearArbolRBT], layout=form_item_layout),
    Box([btn_addNodeArbolRBT], layout=form_item_layout),
    Box([btn_deleteNodeArbolRBT], layout=form_item_layout),
    Box([btn_findNodeArbolRBT], layout=form_item_layout),
    Box([btn_findAdjNodeArbolRBT], layout=form_item_layout),
    Box([btn_findAllNodesArbolRBT],layout=form_item_layout)
]
buttons_grafos = [
    Box([btn_crearGrafo], layout=form_item_layout),
    Box([btn_addNodeGrafo], layout=form_item_layout),
    Box([btn_deleteNodeGrafo], layout=form_item_layout),
    Box([btn_findNodeGrafo], layout=form_item_layout),
    Box([btn_addEdgeGrafo], layout=form_item_layout),
    Box([btn_findAdjNodeGrafo], layout=form_item_layout),
    Box([btn_findAllNodesGrafo], layout=form_item_layout),
    Box([btn_recorridosOptionGrafo], layout=form_item_layout),
]
buttons_grafosND = [
    Box([btn_crearGrafoND], layout=form_item_layout),
    Box([btn_addNodeGrafo], layout=form_item_layout),
    Box([btn_deleteNodeGrafo], layout=form_item_layout),
    Box([btn_findNodeGrafo], layout=form_item_layout),
    Box([btn_addEdgeGrafo], layout=form_item_layout),
    Box([btn_findAdjNodeGrafo], layout=form_item_layout),
    Box([btn_findAllNodesGrafo], layout=form_item_layout),
    Box([btn_recorridosOptionGrafo], layout=form_item_layout),
]
buttons_arreglos = [
    Box([btn_crearArreglo], layout=form_item_layout),
    Box([btn_addNodeArreglo], layout=form_item_layout),
    Box([btn_addNodeArregloFirst], layout=form_item_layout),
    Box([btn_deleteNodeArreglo], layout=form_item_layout),
    Box([btn_findNodeArreglo], layout=form_item_layout),
    Box([btn_findAdjNodeArreglo], layout=form_item_layout),
    Box([btn_findAllNodesArreglo], layout=form_item_layout),
]

buttons_HashLP = [
    Box([btn_crearHashLP], layout=form_item_layout),
    Box([btn_addNodeHashLP], layout=form_item_layout),
    Box([btn_deleteNodeHashLP], layout=form_item_layout),
    Box([btn_FindNodeHashLP], layout=form_item_layout),
]

buttons_HashSC = [
    Box([btn_crearHashSC], layout=form_item_layout),
    Box([btn_addNodeHashSC], layout=form_item_layout),
    Box([btn_deleteNodeHashSC], layout=form_item_layout),
    Box([btn_FindNodeHashSC], layout=form_item_layout),
]

# Cargar Estructura - Lista Enlazada
btn_cargarEstructuraLista = Button(description='Cargar Lista', width='extended', button_style='warning')
def on_button_cargarEstructuraLista_clicked(b):
    with out:       
        try:
            root = tk.Tk()
            src_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Python File", "*.py")])
            root.destroy()
            spec = importlib.util.spec_from_file_location("",src_path)
            foo = importlib.util.module_from_spec(spec)
  
            spec.loader.exec_module(foo)
            this.file = foo
            out.clear_output()
            print('Estructura Lista Enlazada Cargada Exitosamente')
            print(src_path)
        except:
            out.clear_output
            if src_path == '':
                e = "ERROR Problema al cargar la estructura Lista: No se seleccionó ningún archivo"
            else:
                e = "ERROR Problema al cargar la estructura Lista:\n" + src_path + '\n'
                e += "El archivo puede tener errores en su definicion Python"
            print(e)
        buttonsListas.layout.visibility = 'visible'
        buttonsListas.layout.display = 'block'
btn_cargarEstructuraLista.on_click(on_button_cargarEstructuraLista_clicked)

buttons_cargarEstructurasLista = [
    Box([btn_cargarEstructuraLista], layout=form_item_layout),
]

# Cargar Estructura - Arbol BST
btn_cargarEstructuraBST = Button(description='Cargar BST', width='extended', button_style='warning')
def on_button_cargarEstructuraBST_clicked(b):
    with out:       
        try:
            root = tk.Tk()
            src_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Python File", "*.py")])
            root.destroy()
            spec = importlib.util.spec_from_file_location("",src_path)
            foo = importlib.util.module_from_spec(spec)
  
            spec.loader.exec_module(foo)
            this.file = foo
            out.clear_output()
            print('Estructura Arbol BST Cargada Exitosamente')
            print(src_path)
        except:
            out.clear_output
            if src_path == '':
                e = "ERROR Problema al cargar la estructura Arbol BST: No se seleccionó ningún archivo"
            else:
                e = "ERROR Problema al cargar la estructura Arbol BST:\n" + src_path + '\n'
                e += "El archivo puede tener errores en su definicion Python"
                
            print(e)
        buttonsArboles.layout.visibility = 'visible'
        buttonsArboles.layout.display = 'block'
btn_cargarEstructuraBST.on_click(on_button_cargarEstructuraBST_clicked)

buttons_cargarEstructurasBST = [
    Box([btn_cargarEstructuraBST], layout=form_item_layout),
]

# Cargar Estructura - Arbol RBT
btn_cargarEstructuraRBT = Button(description='Cargar RBT', width='extended', button_style='warning')
def on_button_cargarEstructuraRBT_clicked(b):
    with out:       
        try:
            root = tk.Tk()
            src_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Python File", "*.py")])
            root.destroy()
            spec = importlib.util.spec_from_file_location("",src_path)
            foo = importlib.util.module_from_spec(spec)
  
            spec.loader.exec_module(foo)
            this.file = foo
            out.clear_output()
            print('Estructura Arbol RBT Cargada Exitosamente')
            print(src_path)
        except:
            out.clear_output
            if src_path == '':
                e = "ERROR Problema al cargar la estructura Arbol RBT: No se seleccionó ningún archivo"
            else:
                e = "ERROR Problema al cargar la estructura Arbol RBT:\n" + src_path + '\n'
                e += "El archivo puede tener errores en su definicion Python"

            print(e)
        buttonsArbolesRBT.layout.visibility = 'visible'
        buttonsArbolesRBT.layout.display = 'block'
btn_cargarEstructuraRBT.on_click(on_button_cargarEstructuraRBT_clicked)

buttons_cargarEstructurasRBT = [
    Box([btn_cargarEstructuraRBT], layout=form_item_layout),
]

# Cargar Estructura - Grafo
btn_cargarEstructuraGrafo = Button(description='Cargar Grafo', width='extended', button_style='warning')
def on_button_cargarEstructuraGrafo_clicked(b):
    with out:       
        try:
            root = tk.Tk()
            src_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Python File", "*.py")])
            root.destroy()
            spec = importlib.util.spec_from_file_location("",src_path)
            foo = importlib.util.module_from_spec(spec)
  
            spec.loader.exec_module(foo)
            this.file = foo
            out.clear_output()
            print('Estructura Grafo Cargada Exitosamente')
            print(src_path)
        except:
            out.clear_output
            if src_path == '':
                e = "ERROR Problema al cargar la estructura Grafo: No se seleccionó ningún archivo"
            else:
                e = "ERROR Problema al cargar la estructura Grafo:\n" + src_path + '\n'
                e += "El archivo puede tener errores en su definicion Python"

            print(e)
        buttonsGrafos.layout.visibility = 'visible'
        buttonsGrafos.layout.display = 'block'
btn_cargarEstructuraGrafo.on_click(on_button_cargarEstructuraGrafo_clicked)

buttons_cargarEstructurasGrafo = [
    Box([btn_cargarEstructuraGrafo], layout=form_item_layout),
]

# Cargar Estructura - Arreglo
btn_cargarEstructuraArreglo = Button(description='Cargar Arreglo', width='extended', button_style='warning')
def on_button_cargarEstructuraArreglo_clicked(b):
    with out:       
        try:
            root = tk.Tk()
            src_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Python File", "*.py")])
            root.destroy()
            spec = importlib.util.spec_from_file_location("",src_path)
            foo = importlib.util.module_from_spec(spec)
  
            spec.loader.exec_module(foo)
            this.file = foo
            out.clear_output()
            print('Estructura Arreglo Cargada Exitosamente')
            print(src_path)
        except:
            out.clear_output
            if src_path == '':
                e = "ERROR Problema al cargar la estructura Arreglo: No se seleccionó ningún archivo"
            else:
                e = "ERROR Problema al cargar la estructura Arreglo:\n" + src_path + '\n'
                e += "El archivo puede tener errores en su definicion Python"

            print(e)
        buttonsArreglos.layout.visibility = 'visible'
        buttonsArreglos.layout.display = 'block'
btn_cargarEstructuraArreglo.on_click(on_button_cargarEstructuraArreglo_clicked)

buttons_cargarEstructurasArreglo = [
    Box([btn_cargarEstructuraArreglo], layout=form_item_layout),
]


# Cargar Estructura - HashLP
btn_cargarEstructuraHashLP = Button(description='Cargar Tabla Hash', width='extended', button_style='warning')
def on_button_cargarEstructuraHashLP_clicked(b):
    with out:       
        try:
            root = tk.Tk()
            src_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Python File", "*.py")])
            root.destroy()
            spec = importlib.util.spec_from_file_location("",src_path)
            foo = importlib.util.module_from_spec(spec)
  
            spec.loader.exec_module(foo)
            this.file = foo
            out.clear_output()
            print('Estructura HashLP Cargada Exitosamente')
            print(src_path)
        except:
            out.clear_output
            if src_path == '':
                e = "ERROR Problema al cargar la estructura HashLP: No se seleccionó ningún archivo"
            else:
                e = "ERROR Problema al cargar la estructura HashLP:\n" + src_path + '\n'
                e += "El archivo puede tener errores en su definicion Python"
            print(e)
        buttonsHashLP.layout.visibility = 'visible'
        buttonsHashLP.layout.display = 'block'
btn_cargarEstructuraHashLP.on_click(on_button_cargarEstructuraHashLP_clicked)

buttons_cargarEstructurasHashLP = [
    Box([btn_cargarEstructuraHashLP], layout=form_item_layout),
]

# Cargar Estructura - HashSC
btn_cargarEstructuraHashSC = Button(description='Cargar Tabla Hash', width='extended', button_style='warning')
def on_button_cargarEstructuraHashSC_clicked(b):
    with out:       
        try:
            root = tk.Tk()
            src_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Python File", "*.py")])
            root.destroy()
            spec = importlib.util.spec_from_file_location("",src_path)
            foo = importlib.util.module_from_spec(spec)
  
            spec.loader.exec_module(foo)
            this.file = foo
            out.clear_output()
            print('Estructura HashSC Cargada Exitosamente')
            print(src_path)
        except:
            out.clear_output
            if src_path == '':
                e = "ERROR Problema al cargar la estructura HashSC: No se seleccionó ningún archivo"
            else:
                e = "ERROR Problema al cargar la estructura HashSC:\n " + src_path + '\n'
                e += "El archivo puede tener errores en su definicion Python"

            print(e)
        buttonsHashSC.layout.visibility = 'visible'
        buttonsHashSC.layout.display = 'block'
btn_cargarEstructuraHashSC.on_click(on_button_cargarEstructuraHashSC_clicked)

buttons_cargarEstructurasHashSC = [
    Box([btn_cargarEstructuraHashSC], layout=form_item_layout),
]




buttonsCargarLista = Box(buttons_cargarEstructurasLista, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))
buttonsCargarBST = Box(buttons_cargarEstructurasBST, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))
buttonsCargarRBT = Box(buttons_cargarEstructurasRBT, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))

buttonsCargarGrafo = Box(buttons_cargarEstructurasGrafo, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))

buttonsCargarArreglo = Box(buttons_cargarEstructurasArreglo, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))

buttonsCargarHashLP = Box(buttons_cargarEstructurasHashLP, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))

buttonsCargarHashSC = Box(buttons_cargarEstructurasHashSC, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))

buttonsListas = Box(buttons_listas, layout=Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))
buttonsListasDoble = Box(buttons_listasDoble, layout=Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))
buttonsArboles = Box(buttons_arboles, layout=Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))
buttonsArbolesRBT = Box(buttons_arbolesRBT, layout=Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))
buttonsGrafos = Box(buttons_grafos, layout=Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))
buttonsGrafosND = Box(buttons_grafosND, layout=Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))

buttonsArreglos = Box(buttons_arreglos, layout=Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))

buttonsHashLP = Box(buttons_HashLP, layout=Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))

buttonsHashSC = Box(buttons_HashSC, layout=Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))

canvas = HBox([out], layout=layout_canvas)
optionsListas = VBox([buttonsCargarLista,
                      buttonsListas,
                      optionsCreateList,
                      optionsAddNode,
                      optionsAddNodeFirst,
                      optionsDeleteNode,
                      optionsFindNode,
                      optionsFindAdjNode,
                      optionsFindAllNodes],
                      layout=layout_options)
optionsListasDoble = VBox([buttonsCargarLista,
                      buttonsListas,
                      optionsCreateListDoble,
                      optionsAddNode,
                      optionsAddNodeFirst,
                      optionsDeleteNode,
                      optionsFindNode,
                      optionsFindAdjNode,
                      optionsFindAllNodes],
                      layout=layout_options)
optionsArboles = VBox([buttonsCargarBST,
                       buttonsArboles,
                        optionsCreateArbol,
                        optionsAddNodeArbol,
                        optionsDeleteNodeArbol,
                        optionsFindNodeArbol,
                        optionsFindAdjNodeArbol,
                        optionsFindAllNodesArbol],
                        layout=layout_options)
optionsArbolesRBT = VBox([buttonsCargarRBT,
                       buttonsArbolesRBT,
                        optionsCreateArbolRBT,
                        optionsAddNodeArbolRBT,
                        optionsDeleteNodeArbolRBT,
                        optionsFindNodeArbolRBT,
                        optionsFindAdjNodeArbolRBT,
                        optionsFindAllNodesArbolRBT],
                        layout=layout_options)
optionsGrafos = VBox([buttonsCargarGrafo,
                    buttonsGrafos,            
                    optionsCreateGrafo,
                    optionsAddNodeGrafo,
                    optionsDeleteNodeGrafo,
                    optionsFindNodeGrafo,
                    optionsAddEdgeGrafo,
                    optionsFindAdjNodeGrafo,
                    optionsAllNodesGrafo,
                    optionsRecorridosGrafoD],
                    layout=layout_options)
optionsGrafosND = VBox([buttonsCargarGrafo,
                    buttonsGrafos,            
                    optionsCreateGrafoND,
                    optionsAddNodeGrafo,
                    optionsDeleteNodeGrafo,
                    optionsFindNodeGrafo,
                    optionsAddEdgeGrafo,
                    optionsFindAdjNodeGrafo,
                    optionsAllNodesGrafo,
                    optionsRecorridosGrafo],
                    layout=layout_options)
optionsArreglo = VBox([buttonsCargarArreglo,
                    buttonsArreglos,            
                    optionsCreateArreglo,
                    optionsAddNodeArreglo,
                    optionsAddNodeArregloFirst,
                    optionsDeleteNodeArreglo,
                    optionsFindNodeArreglo,
                    optionsFindAdjNodeArreglo,
                    optionsFindAllNodesArreglo],
                    layout=layout_options)
optionsHashLP = VBox([buttonsCargarHashLP,
                    buttonsHashLP,            
                    optionsCreateHashLP,
                    optionsAddNodeHashLP,
                    optionsDeleteNodeHashLP,
                    optionsFindNodeHashLP],
                    layout=layout_options)
optionsHashSC = VBox([buttonsCargarHashSC,
                    buttonsHashSC,            
                    optionsCreateHashSC,
                    optionsAddNodeHashSC,
                    optionsDeleteNodeHashSC,
                    optionsFindNodeHashSC],
                    layout=layout_options)

# -------------------------- Cargar Estructura - Nueva --------------------------#
#btn_cargarNueva = Button(description='Cargar Nueva', width='extended', button_style='warning')
#def on_button_cargarNueva_clicked(b):
#    with out:       
#        try:
#            root = tk.Tk()
#            src_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Python File", "*.py")])
#            root.destroy()
#            spec = importlib.util.spec_from_file_location("",src_path)
#            foo = importlib.util.module_from_spec(spec)
#  
#            spec.loader.exec_module(foo)
#            this.file = foo
#            out.clear_output()
#            print('Estructura Cargada Exitosamente')
#            print(src_path)
#        except:
#            out.clear_output
#            if src_path == '':
#                e = "Problema al cargar la estructura: No se seleccionó ningún archivo"
#            else:
#                e = "Problema al cargar la estructura: " + src_path
#            print(e)
#        buttonsNueva.layout.visibility = 'visible'
#        buttonsNueva.layout.display = 'block'
#btn_cargarNueva.on_click(on_button_cargarNueva_clicked)
#
#
#buttons_nueva = [
#    Box([btn_nuevoBoton], layout=form_item_layout),
#]
#
#buttons_cargarEstructurasNueva = [
#    Box([btn_cargarNueva], layout=form_item_layout),
#]
#
#buttonsCargarNuevaEstructura = Box(buttons_cargarEstructurasNueva, layout=Layout(
#    display='flex',
#    flex_flow='column',
#    align_items='stretch',
#    width='auto',
#    justify_content='center'
#))
#
#buttonsNueva = Box(buttons_nueva, layout=Layout(
#    display='none',
#    flex_flow='column',
#    align_items='stretch',
#    width='auto',
#    justify_content='center'
#))
#
#optionsNuevaEstructura = VBox([buttonsCargarNuevaEstructura,
#                    buttonsNueva,            
#                    optionsNuevoBoton],
#                    layout=layout_options)
# ------------------------------------------------------------------------------#

#### Construir Aplicación

In [53]:
with out1:
    app = AppLayout(header=None,
          left_sidebar=None,
          center=canvas,
          right_sidebar=optionsArreglo,
          pane_widths=['0%', '80%', '20%'],
          pane_heights=['0%','590px', 1],
          footer=None)
    display(app)
with out2:
    app = AppLayout(header=None,
          left_sidebar=None,
          center=canvas,
          right_sidebar=optionsListas,
          pane_widths=['0%', '80%', '20%'],
          pane_heights=['0%','590px', 1],
          footer=None)
    display(app)
with out3:
    app = AppLayout(header=None,
          left_sidebar=None,
          center=canvas,
          right_sidebar=optionsListasDoble,
          pane_widths=['0%', '80%', '20%'],
          pane_heights=['0%','590px', 1],
          footer=None)
    display(app)
with out4:
    app = AppLayout(header=None,
          left_sidebar=None,
          center=canvas,
          right_sidebar=optionsArboles,
          pane_widths=['0%', '80%', '20%'],
          pane_heights=['0%','590px', 1],
          footer=None)
    display(app)
with out5:
    app = AppLayout(header=None,
          left_sidebar=None,
          center=canvas,
          right_sidebar=optionsGrafos,
          pane_widths=['0%', '80%', '20%'],
          pane_heights=['0%','590px', 1],
          footer=None)
    display(app)
with out6:
    app = AppLayout(header=None,
          left_sidebar=None,
          center=canvas,
          right_sidebar=optionsGrafosND,
          pane_widths=['0%', '80%', '20%'],
          pane_heights=['0%','590px', 1],
          footer=None)
    display(app)
with out7:
    app = AppLayout(header=None,
          left_sidebar=None,
          center=canvas,
          right_sidebar=optionsHashLP,
          pane_widths=['0%', '80%', '20%'],
          pane_heights=['0%','590px', 1],
          footer=None)
    display(app)
with out8:
    app = AppLayout(header=None,
          left_sidebar=None,
          center=canvas,
          right_sidebar=optionsHashSC,
          pane_widths=['0%', '80%', '20%'],
          pane_heights=['0%','590px', 1],
          footer=None)
    display(app)
with out9:
    app = AppLayout(header=None,
          left_sidebar=None,
          center=canvas,
          right_sidebar=optionsArbolesRBT,
          pane_widths=['0%', '80%', '20%'],
          pane_heights=['0%','590px', 1],
          footer=None)
    display(app)

## Mostrar Aplicación

In [57]:
display(tab) 